In [1]:
import os
import pandas as pd
import numpy as np
import random
import sys
import csv

In [2]:
mimic_path = "C:/Users/Mariam Safieldin/Documents/Personal Docs/AUB/Thesis/Methodology/Transfer Learning/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/" # Path containing MIMIC data
output_files = "C:/Users/Mariam Safieldin/Documents/Personal Docs/AUB/Thesis/Methodology/Transfer Learning/mimic-iii-clinical-database-1.4/mimic-iii-clinical-database-1.4/Output Files/" # Path containing MIMIC data

# Patients File

In [3]:
pt=pd.read_csv(mimic_path+'PATIENTS.csv.gz', header=0, index_col=0)
pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46520 entries, 234 to 31844
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SUBJECT_ID   46520 non-null  int64 
 1   GENDER       46520 non-null  object
 2   DOB          46520 non-null  object
 3   DOD          15759 non-null  object
 4   DOD_HOSP     9974 non-null   object
 5   DOD_SSN      13378 non-null  object
 6   EXPIRE_FLAG  46520 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [4]:
pt['GENDER'].tolist().count('F')

20399

In [5]:
pt['GENDER'].tolist().count('M')

26121

In [6]:
len(pt['SUBJECT_ID'].tolist())

46520

In [7]:
demo=pt[['SUBJECT_ID', 'GENDER']]

In [8]:
pt.DOB = pd.to_datetime(pt.DOB)
pt.DOD = pd.to_datetime(pt.DOD)
pt.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
ROW_ID,,,,,,,
234,249,F,2075-03-13,NaT,NaN,NaN,0
235,250,F,2164-12-27,2188-11-22,2188-11-22 00:00:00,NaN,1
236,251,M,2090-03-15,NaT,NaN,NaN,0
237,252,M,2078-03-06,NaT,NaN,NaN,0
238,253,F,2089-11-26,NaT,NaN,NaN,0


# Admissions File

In [9]:
adm=pd.read_csv(mimic_path+"ADMISSIONS.csv.gz", header=0, index_col=0) 
adm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 21 to 58598
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   SUBJECT_ID            58976 non-null  int64 
 1   HADM_ID               58976 non-null  int64 
 2   ADMITTIME             58976 non-null  object
 3   DISCHTIME             58976 non-null  object
 4   DEATHTIME             5854 non-null   object
 5   ADMISSION_TYPE        58976 non-null  object
 6   ADMISSION_LOCATION    58976 non-null  object
 7   DISCHARGE_LOCATION    58976 non-null  object
 8   INSURANCE             58976 non-null  object
 9   LANGUAGE              33644 non-null  object
 10  RELIGION              58518 non-null  object
 11  MARITAL_STATUS        48848 non-null  object
 12  ETHNICITY             58976 non-null  object
 13  EDREGTIME             30877 non-null  object
 14  EDOUTTIME             30877 non-null  object
 15  DIAGNOSIS             58951 non-nul

In [10]:
len(adm['HAS_CHARTEVENTS_DATA'].tolist())

58976

In [11]:
len(adm['HADM_ID'].tolist())

58976

In [12]:
demo1 = adm[['SUBJECT_ID', 'HADM_ID', 'INSURANCE','LANGUAGE','DISCHARGE_LOCATION','ADMISSION_TYPE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']]

In [13]:
def subj_merge(table1, table2):
    return table1.merge(table2, how='inner', left_on=['SUBJECT_ID'], right_on=['SUBJECT_ID'])

In [14]:
adm_pt=subj_merge(adm, pt)
adm_pt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   SUBJECT_ID            58976 non-null  int64         
 1   HADM_ID               58976 non-null  int64         
 2   ADMITTIME             58976 non-null  object        
 3   DISCHTIME             58976 non-null  object        
 4   DEATHTIME             5854 non-null   object        
 5   ADMISSION_TYPE        58976 non-null  object        
 6   ADMISSION_LOCATION    58976 non-null  object        
 7   DISCHARGE_LOCATION    58976 non-null  object        
 8   INSURANCE             58976 non-null  object        
 9   LANGUAGE              33644 non-null  object        
 10  RELIGION              58518 non-null  object        
 11  MARITAL_STATUS        48848 non-null  object        
 12  ETHNICITY             58976 non-null  object        
 13  EDREGTIME       

In [15]:
import datetime as DT
adm_pt['ADMITTIME'] = pd.to_datetime(adm_pt['ADMITTIME']).dt.date
adm_pt['DOB'] = pd.to_datetime(adm_pt['DOB']).dt.date

In [16]:
adm_pt['AGE'] = adm_pt.apply(lambda e: (e['ADMITTIME'] - e['DOB']).days/365, axis=1)


In [17]:
adm_pt

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,AGE
0,22,165315,2196-04-09,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,...,BENZODIAZEPINE OVERDOSE,0,1,F,2131-05-07,NaT,NaN,NaN,0,64.969863
1,23,152223,2153-09-03,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,M,2082-07-17,NaT,NaN,NaN,0,71.178082
2,23,124321,2157-10-18,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,BRAIN MASS,0,1,M,2082-07-17,NaT,NaN,NaN,0,75.304110
3,24,161859,2139-06-06,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,...,INTERIOR MYOCARDIAL INFARCTION,0,1,M,2100-05-31,NaT,NaN,NaN,0,39.041096
4,25,129635,2160-11-02,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,...,ACUTE CORONARY SYNDROME,0,1,M,2101-11-21,NaT,NaN,NaN,0,58.989041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98794,190603,2127-11-07,2127-11-11 14:30:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,...,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,0,1,M,2049-07-29,NaT,NaN,NaN,0,78.326027
58972,98797,105447,2132-12-24,2132-12-25 12:00:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,...,ALTERED MENTAL STATUS,1,1,M,2044-12-27,2132-12-25,2132-12-25 00:00:00,NaN,1,88.049315
58973,98800,191113,2131-03-30,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,...,TRAUMA,0,1,F,2111-11-05,NaT,NaN,NaN,0,19.410959
58974,98802,101071,2151-03-05,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,...,SAH,1,1,F,2067-09-21,2151-03-06,2151-03-06 00:00:00,NaN,1,83.506849


In [18]:
adm_pt.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG', 'AGE'],
      dtype='object')

In [19]:
adm_pt.drop(['ADMITTIME','DISCHTIME','DEATHTIME','EDREGTIME','EDOUTTIME','HOSPITAL_EXPIRE_FLAG','DOB','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'],inplace=True,axis=1)

In [20]:
adm_pt

,SUBJECT_ID,HADM_ID,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,GENDER,AGE
0,22,165315,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,BENZODIAZEPINE OVERDOSE,1,F,64.969863
1,23,152223,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,M,71.178082
2,23,124321,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,1,M,75.304110
3,24,161859,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,INTERIOR MYOCARDIAL INFARCTION,1,M,39.041096
4,25,129635,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,ACUTE CORONARY SYNDROME,1,M,58.989041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98794,190603,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,1,M,78.326027
58972,98797,105447,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,ALTERED MENTAL STATUS,1,M,88.049315
58973,98800,191113,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,TRAUMA,1,F,19.410959
58974,98802,101071,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SAH,1,F,83.506849


In [21]:
adm_pt.to_csv(output_files+'demographics.csv')

# ICU Stays File

In [22]:
icu=pd.read_csv(mimic_path+"ICUSTAYS.csv.gz", header=0, index_col=0) 
icu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61532 entries, 365 to 59810
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SUBJECT_ID      61532 non-null  int64  
 1   HADM_ID         61532 non-null  int64  
 2   ICUSTAY_ID      61532 non-null  int64  
 3   DBSOURCE        61532 non-null  object 
 4   FIRST_CAREUNIT  61532 non-null  object 
 5   LAST_CAREUNIT   61532 non-null  object 
 6   FIRST_WARDID    61532 non-null  int64  
 7   LAST_WARDID     61532 non-null  int64  
 8   INTIME          61532 non-null  object 
 9   OUTTIME         61522 non-null  object 
 10  LOS             61522 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 5.6+ MB


In [23]:
icu.INTIME = pd.to_datetime(icu.INTIME)
icu.OUTTIME = pd.to_datetime(icu.OUTTIME)
icu.ICUSTAY_ID.isnull().sum()

0

In [24]:
icu=icu[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS']]

In [25]:
def adm_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID'], right_on=['SUBJECT_ID','HADM_ID'])

In [26]:
icu_adm_pt=adm_merge(icu, adm_pt)

In [27]:
icu_adm_pt

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,GENDER,AGE
0,268,110404,280836,3.2490,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,SEPARATED,HISPANIC OR LATINO,DYSPNEA,1,F,66.019178
1,269,106296,206613,3.2788,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicaid,NaN,UNOBTAINABLE,SINGLE,WHITE,SEPSIS;PILONIDAL ABSCESS,1,M,40.126027
2,270,188028,220345,2.8939,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,JEHOVAH'S WITNESS,MARRIED,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,1,M,80.128767
3,271,173727,249196,2.0600,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,GALLSTONE PANCREATITIS,1,F,45.715068
4,272,164716,210407,1.6202,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,UNOBTAINABLE,MARRIED,WHITE,PULMONARY EMBOLIS,1,M,67.139726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,94944,143774,201233,2.1894,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,JEWISH,MARRIED,WHITE,CHEST PAIN;RULE OUT CORONARY ARTERY DISEASE\LE...,1,M,77.161644
61528,94950,123750,283653,2.4942,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,SYNCOPE;TELEMETRY,1,F,300.200000
61529,94953,196881,241585,0.9259,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,ENGL,PROTESTANT QUAKER,WIDOWED,WHITE,CEREBRAL ANEURYSM/SDA,1,F,53.128767
61530,94954,118475,202802,2.3346,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,ENGL,NOT SPECIFIED,MARRIED,PATIENT DECLINED TO ANSWER,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT /SDA,1,F,67.884932


In [28]:
icu_adm_pt.to_csv(output_files+'icu_stays_demographics.csv')

# CHARTEVENTS and D_ITEMS Files

In [29]:
charts=pd.read_csv(mimic_path + "CHARTEVENTS.csv.gz", nrows=10000000, header=0, index_col=0) 
charts.info

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<bound method DataFrame.info of          SUBJECT_ID  HADM_ID  ICUSTAY_ID  ITEMID            CHARTTIME  \
ROW_ID                                                                  
788              36   165660    241249.0  223834  2134-05-12 12:00:00   
789              36   165660    241249.0  223835  2134-05-12 12:00:00   
790              36   165660    241249.0  224328  2134-05-12 12:00:00   
791              36   165660    241249.0  224329  2134-05-12 12:00:00   
792              36   165660    241249.0  224330  2134-05-12 12:00:00   
...             ...      ...         ...     ...                  ...   
9931857       48910   143290    246813.0  224695  2186-10-14 03:56:00   
9931858       48910   143290    246813.0  224697  2186-10-14 03:56:00   
9931859       48910   143290    246813.0  220045  2186-10-14 03:57:00   
9931860       48910   143290    246813.0  220050  2186-10-14 03:57:00   
9931861       48910   143290    246813.0  220051  2186-10-14 03:57:00   

                  

In [30]:
charts.sort_values('SUBJECT_ID', ascending=True)
charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
ROW_ID,,,,,,,,,,,,,,
788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.00,15.00,L/min,0,0,NaN,NaN
789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.00,100.00,NaN,0,0,NaN,NaN
790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823.0,0.37,0.37,NaN,0,0,NaN,NaN
791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,6.00,6.00,min,0,0,NaN,NaN
792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,2.50,2.50,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931857,48910,143290,246813.0,224695,2186-10-14 03:56:00,NaN,NaN,11.00,11.00,cmH2O,0,0,NaN,NaN
9931858,48910,143290,246813.0,224697,2186-10-14 03:56:00,NaN,NaN,7.00,7.00,cmH2O,0,0,NaN,NaN
9931859,48910,143290,246813.0,220045,2186-10-14 03:57:00,NaN,NaN,86.00,86.00,bpm,0,0,NaN,NaN


In [31]:
charts.sort_values('SUBJECT_ID', ascending=True)
charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
ROW_ID,,,,,,,,,,,,,,
788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.00,15.00,L/min,0,0,NaN,NaN
789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.00,100.00,NaN,0,0,NaN,NaN
790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823.0,0.37,0.37,NaN,0,0,NaN,NaN
791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,6.00,6.00,min,0,0,NaN,NaN
792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,2.50,2.50,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931857,48910,143290,246813.0,224695,2186-10-14 03:56:00,NaN,NaN,11.00,11.00,cmH2O,0,0,NaN,NaN
9931858,48910,143290,246813.0,224697,2186-10-14 03:56:00,NaN,NaN,7.00,7.00,cmH2O,0,0,NaN,NaN
9931859,48910,143290,246813.0,220045,2186-10-14 03:57:00,NaN,NaN,86.00,86.00,bpm,0,0,NaN,NaN


In [32]:
charts=charts.dropna(how='any', subset=['VALUENUM']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [33]:
d_c=pd.read_csv(mimic_path + 'D_ITEMS.csv.gz', header=0, index_col=0) 
d_c=d_c[['ITEMID', 'LABEL', 'CATEGORY']]
d_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12487 entries, 457 to 14522
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ITEMID    12487 non-null  int64 
 1   LABEL     12483 non-null  object
 2   CATEGORY  6049 non-null   object
dtypes: int64(1), object(2)
memory usage: 390.2+ KB


In [15]:
def item_merge(table1, table2):
    return table1.merge(table2, how='inner', left_on=['ITEMID'], right_on=['ITEMID'])

In [35]:
charts=item_merge(charts, d_c)
charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
0,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
1,23,124321,234044.0,223834,2157-10-21 20:00:00,2157-10-22 00:45:00,19050.0,3.0,3.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
2,23,124321,234044.0,223834,2157-10-21 22:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
3,23,124321,234044.0,223834,2157-10-22 00:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
4,36,165660,241249.0,223834,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,15.0,15.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,45344,195325,276206.0,228379,2172-06-14 20:16:00,2172-06-14 20:18:00,14939.0,5.0,5.0,%,0,0,NaN,NaN,TFCd0 (NICOM),NICOM
9999996,46156,127108,258962.0,220245,2134-08-22 12:00:00,2134-08-22 12:54:00,20389.0,180.0,180.0,ml/min,0,0,NaN,NaN,CO2 production,Respiratory
9999997,49138,141813,296225.0,220245,2111-06-24 16:00:00,2111-06-24 18:50:00,20389.0,290.0,290.0,ml/min,1,0,NaN,NaN,CO2 production,Respiratory
9999998,46732,167513,223514.0,227645,2170-04-08 09:00:00,2170-04-08 09:43:00,16617.0,10.0,10.0,min,0,0,NaN,NaN,Breast Feed - L (Minutes),OB-GYN


In [36]:
charts['LABEL'] = charts['LABEL'].astype('category')

In [37]:
charts['LABEL'].cat.set_categories(['ABI Ankle BP L', 'ABI Ankle BP L (Impella)', 'ABI Ankle BP R',
       'ABI Brachial BP L', 'ABI Brachial BP R',
       'ABI Brachial BP R (Impella)', 'ALT', 'APACHE II',
       'APACHE II PDR - Adjusted', 'APACHE II Predecited Death Rate',
       'APACHEIII', 'APS', 'ART BP Diastolic', 'ART BP Systolic',
       'ART BP mean', 'ART Blood Pressure Alarm - High',
       'ART Blood Pressure Alarm - Low', 'ART Lumen Volume', 'AST',
       'ATC %', 'AaDO2ApacheIIValue', 'Access Pressure',
       'Activated Clotting Time',
       'Activity HR - Aerobic Activity Response',
       'Activity HR - Aerobic Capacity',
       'Activity O2 Sat - Aerobic Capacity',
       'Activity O2 sat - Aerobic Activity Response',
       'Activity RR - Aerobic Activity Response',
       'Activity RR - Aerobic Capacity', 'Admission Weight (Kg)',
       'Admission Weight (lbs.)', 'AgeApacheIIScore', 'AgeApacheIIValue',
       'AgeScore_ApacheIV', 'Albumin', 'AlbuminScore_ApacheIV',
       'Albumin_ApacheIV', 'Alkaline Phosphate', 'Ammonia', 'Amylase',
       'Anion gap', 'Aortic Pressure Signal - Diastolic',
       'Aortic Pressure Signal - Systolic', 'Apache IV Age',
       'Apache IV PaFiRatio', 'ApacheIV_LOS',
       'ApacheIV_Mortality prediction', 'ApacheIV_Natural antilog',
       'Apnea Interval', 'Arctic Sun Water Temp',
       'Arctic Sun/Alsius Set Temp', 'Arctic Sun/Alsius Temp #1 C',
       'Arctic Sun/Alsius Temp #2 C', 'Arterial Base Excess',
       'Arterial Blood Pressure Alarm - High',
       'Arterial Blood Pressure Alarm - Low',
       'Arterial Blood Pressure diastolic',
       'Arterial Blood Pressure mean', 'Arterial Blood Pressure systolic',
       'Arterial CO2 Pressure', 'Arterial O2 Saturation',
       'Arterial O2 pressure', 'Assisted Systole', 'Augmented Diastole',
       'BAEDP', 'BIS Index Range', 'BUN', 'BUN_ApacheIV',
       'Baseline Current/mA', 'BiPap EPAP', 'BiPap IPAP', 'BiPap O2 Flow',
       'BiPap bpm (S/T -Back up)', 'BiliScore_ApacheIV',
       'Bilirubin_ApacheIV', 'Bladder Pressure', 'Blood Flow (ml/min)',
       'Blood Temperature CCO (C)', 'Brain Natiuretic Peptide (BNP)',
       'Breast Feed - L (Minutes)', 'Breast Feed - R (Minutes)',
       'BunScore_ApacheIV', 'C Reactive Protein (CRP)', 'CFI (PiCCO)',
       'CI (PiCCO)', 'CK (CPK)', 'CK-MB', 'CK-MB fraction (%)',
       'CO (Arterial)', 'CO (PiCCO)', 'CO / CI Change',
       'CO-Tandem Heart Flow', 'CO2 production', 'Calcium non-ionized',
       'Cardiac Index (CI NICOM)', 'Cardiac Output (CCO)',
       'Cardiac Output (CO NICOM)', 'Cardiac Output (thermodilution)',
       'Cash amount', 'Central Venous O2% Sat', 'Central Venous Pressure',
       'Central Venous Pressure  Alarm - Low',
       'Central Venous Pressure Alarm - High',
       'Cerebral Perfusion Pressure',
       'Cerebral Perfusion Pressure Alarm - High',
       'Cerebral Perfusion Pressure Alarm - Low', 'Chloride (serum)',
       'Chloride (whole blood)', 'Cholesterol', 'ChpApacheIIScore',
       'ChronicScore_ApacheIV', 'Citrate (ACD-A)', 'Clot Size (cm)',
       'Cortisol', 'CreatScore_ApacheIV', 'Creatinine',
       'CreatinineApacheIIScore', 'CreatinineApacheIIValue',
       'Creatinine_ApacheIV', 'Cuff Pressure', 'Cuff Volume (mL)',
       'Current Goal', 'Current Used/mA', 'D-Dimer', 'Daily Weight',
       'Dialysate Rate', 'Differential-Atyps', 'Differential-Bands',
       'Differential-Basos', 'Differential-Eos', 'Differential-Lymphs',
       'Differential-Monos', 'Differential-Neuts', 'Digoxin',
       'Direct Bilirubin', 'Doppler BP', 'DswfApacheScore',
       'Dwell Time (Peritoneal Dialysis)', 'EF (CCO)', 'ELWI (PiCCO)',
       'Effluent Pressure', 'Ejection Fraction', 'Epidural Bolus (mL)',
       'Epidural Infusion Rate (mL/hr)', 'Epidural Total Dose (mL)',
       'Estimated Energy Needs/Kg', 'Estimated Protein Needs/Kg',
       'Expiratory Ratio', 'FK506', 'Feeding Weight', 'FiO2ApacheIIValue',
       'FiO2_ApacheIV', 'Fibrinogen', 'Filter Pressure',
       'Flow Rate (L/min)', 'Freeform Chemistry', 'Fspn High',
       'GEDI (PiCCO)', 'GI pH', 'GcsApacheIIScore', 'GcsScore_ApacheIV',
       'Gentamicin (Peak)', 'Gentamicin (Random)', 'Gentamicin (Trough)',
       'Glucose (serum)', 'Glucose (whole blood)', 'Glucose finger stick',
       'GlucoseScore_ApacheIV', 'Glucose_ApacheIV', 'HCO3 (serum)',
       'HCO3ApacheIIValue', 'HCO3Score', 'HDL', 'HR_ApacheIV',
       'Head of Bead Measurement', 'Heart Rate', 'Heart Rate Alarm - Low',
       'Heart rate Alarm - High', 'Height', 'Height (cm)',
       'Hematocrit (serum)', 'Hematocrit (whole blood - calc)',
       'HematocritApacheIIScore', 'HematocritApacheIIValue',
       'Hematocrit_ApacheIV', 'Hemodialysis Output', 'Hemoglobin',
       'Heparin Dose (per hour)', 'Hourly Patient Fluid Removal',
       'HrApacheIIScore', 'HrApacheIIValue', 'HrScore_ApacheIV',
       'HtScore_ApacheIV', 'IABP Mean', 'IABP Volume', 'INR',
       'Impaired Skin Depth #1', 'Impaired Skin Depth #10',
       'Impaired Skin Depth #2', 'Impaired Skin Depth #3',
       'Impaired Skin Depth #4', 'Impaired Skin Depth #5',
       'Impaired Skin Depth #6', 'Impaired Skin Depth #7',
       'Impaired Skin Depth #8', 'Impaired Skin Depth #9',
       'Impaired Skin Length #1', 'Impaired Skin Length #10',
       'Impaired Skin Length #2', 'Impaired Skin Length #3',
       'Impaired Skin Length #4', 'Impaired Skin Length #5',
       'Impaired Skin Length #6', 'Impaired Skin Length #7',
       'Impaired Skin Length #8', 'Impaired Skin Length #9',
       'Impaired Skin Width #1', 'Impaired Skin Width #10',
       'Impaired Skin Width #2', 'Impaired Skin Width #3',
       'Impaired Skin Width #4', 'Impaired Skin Width #5',
       'Impaired Skin Width #6', 'Impaired Skin Width #7',
       'Impaired Skin Width #8', 'Impaired Skin Width #9',
       'Impella 2.5 Flow Rate', 'Impella Catheter Position',
       'Infusate Drops/Minute', 'Infusion Pressure of Line',
       'Inspiratory Ratio', 'Inspiratory Time', 'Inspired Gas Temp.',
       'Inspired O2 Fraction', 'Intra Cranial Pressure',
       'Intra Cranial Pressure #2', 'Intra Cranial Pressure Alarm - High',
       'Intra Cranial Pressure Alarm - Low', 'Ionized Calcium', 'LDH',
       'LDL calculated', 'LDL measured', 'LOS pre-ICU admission',
       'Lactic Acid', 'Lipase', 'MAP_ApacheIV', 'MDI #1 Puff',
       'MDI #2 Puff', 'MDI #3 Puff', 'Magnesium',
       'Manual Blood Pressure Diastolic Left',
       'Manual Blood Pressure Diastolic Right',
       'Manual Blood Pressure Systolic Left',
       'Manual Blood Pressure Systolic Right', 'MapApacheIIScore',
       'MapApacheIIValue', 'MapScore_ApacheIV', 'Mean Airway Pressure',
       'Medication Added Amount  #1 (Peritoneal Dialysis)',
       'Medication Added Amount  #2 (Peritoneal Dialysis)',
       'Medication Infusion Rate - Adjunctive Pain Management',
       'Minute Volume', 'Minute Volume Alarm - High',
       'Minute Volume Alarm - Low', 'Mixed Venous O2% Sat',
       'Negative Insp. Force', 'Nitric Oxide',
       'Nitric Oxide Tank Pressure',
       'Non Invasive Blood Pressure diastolic',
       'Non Invasive Blood Pressure mean',
       'Non Invasive Blood Pressure systolic',
       'Non-Invasive Blood Pressure Alarm - High',
       'Non-Invasive Blood Pressure Alarm - Low', 'O2 Flow',
       'O2 Flow (additional cannula)',
       'O2 Saturation Pulseoxymetry Alarm - High',
       'O2 Saturation Pulseoxymetry Alarm - Low',
       'O2 saturation pulseoxymetry', 'Orthostatic BPd lying',
       'Orthostatic BPd sitting', 'Orthostatic BPd standing',
       'Orthostatic BPs lying', 'Orthostatic BPs sitting',
       'Orthostatic BPs standing', 'Orthostatic HR lying',
       'Orthostatic HR sitting', 'Orthostatic HR standing',
       'OxygenApacheIIScore', 'OxygenScore_ApacheIV',
       'Oxygenator Sweep Rate', 'P High (APRV)', 'P Low (APRV)',
       'PA Line cm Mark', 'PAEDP', 'PBP (Prefilter) Replacement Rate',
       'PCA 1 hour limit', 'PCA attempt', 'PCA basal rate (mL/hour)',
       'PCA bolus', 'PCA dose', 'PCA inject', 'PCA lockout (min)',
       'PCA total dose', 'PCO2_ApacheIV', 'PCV Level (Avea)', 'PCWP',
       'PEEP set', 'PH (Arterial)', 'PH (Venous)', 'PH (dipstick)',
       'PHApacheIIValue', 'PHPaCO2Score_ApacheIV', 'PH_ApacheIV',
       'PO2 (Mixed Venous)', 'PO2_ApacheIV', 'PSV Level', 'PTT',
       'Paw High', 'Peak Exp Flow Rate', 'Peak Insp. Pressure',
       'Permanent Pacemaker Rate', 'PhApacheIIScore', 'Phenobarbital',
       'Phenytoin (Dilantin)', 'Phenytoin (Free)', 'Phosphorous',
       'Pinsp (Draeger only)', 'Plateau Pressure', 'Platelet Count',
       'Post Filter Replacement Rate', 'Potassium (serum)',
       'Potassium (whole blood)', 'PotassiumApacheIIScore',
       'PotassiumApacheIIValue', 'Procan', 'Procan Napa',
       'Prothrombin time', 'Pulmonary Artery Pressure Alarm - High',
       'Pulmonary Artery Pressure Alarm - Low',
       'Pulmonary Artery Pressure diastolic',
       'Pulmonary Artery Pressure mean',
       'Pulmonary Artery Pressure systolic', 'Pulsus Paradoxus',
       'Pump Speed', 'Purge Pressure', 'Purge Solution Flow Rate', 'QTc',
       'RRApacheIIValue', 'RRScore_ApacheIV', 'RR_ApacheIV',
       'Recovery HR - Aerobic Activity Response',
       'Recovery HR - Aerobic Capacity',
       'Recovery O2 Sat - Aerobic Capacity',
       'Recovery O2 sat - Aerobic Activity Response',
       'Recovery RR - Aerobic Activity Response',
       'Recovery RR - Aerobic Capacity', 'Recruitment Duration',
       'Recruitment Press', 'Replacement Rate', 'Resistance',
       'Resp Alarm - High', 'Resp Alarm - Low', 
       'Respiratory Rate',
       'Respiratory Rate (Set)', 'Respiratory Rate (Total)',
       'Respiratory Rate (spontaneous)',
       'Rest HR -  Aerobic Activity Response',
       'Rest HR - Aerobic Capacity',
       'Rest O2 Sat - Aerobic Activity Response',
       'Rest O2 Sat - Aerobic Capacity',
       'Rest RR - Aerobic Activity Response',
       'Rest RR - Aerobic Capacity', 'Return Pressure', 'RrApacheIIScore',
       'SOFA Score', 'SV (Arterial)', 'SVI (PiCCO)', 'SVI Change',
       'SVRI (PiCCO)', 'SVV (Arterial)', 'SVV (PiCCO)', 'ScvO2 (Presep)',
       'ScvO2 Central Venous O2% Sat', 'Sed Rate', 'Serum Osmolality',
       'Skeletal Traction #1 - Pounds', 'Skeletal Traction #2 - Pounds',
       'Small Volume Neb Dose #2', 'Sodium (serum)',
       'Sodium (whole blood)', 'SodiumApacheIIScore',
       'SodiumApacheIIValue', 'SodiumScore_ApacheIV', 'Sodium_ApacheIV',
       'SpO2 Desat Limit', 'Specific Gravity (urine)', 'Spont RR',
       'Spont Vt', 'Stroke Volume (SV NICOM)',
       'Stroke Volume Index (SVI NICOM)',
       'Stroke Volume Variation (SVV NICOM)', 'SvO2', 'T High (APRV)',
       'T Low (APRV)', 'TCO2 (calc) Arterial', 'TCO2 (calc) Venous',
       'TFCd (NICOM)', 'TFCd0 (NICOM)',
       'Tandem Heart Access Line Catheter Length', 'Tandem Heart Flow',
       'TempApacheIIScore', 'TempApacheIIValue', 'TempScore_ApacheIV',
       'Temperature Celsius', 'Temperature Fahrenheit',
       'TemperatureF_ApacheIV', 'Temporary AV interval',
       'Temporary Atrial Sens Setting mV',
       'Temporary Atrial Sens Threshold mV',
       'Temporary Atrial Stim Setting mA',
       'Temporary Atrial Stim Threshold mA', 'Temporary Pacemaker Rate',
       'Temporary Pacemaker Wires Atrial',
       'Temporary Pacemaker Wires Ground',
       'Temporary Pacemaker Wires Venticular',
       'Temporary Venticular Sens Threshold mV',
       'Temporary Venticular Stim Threshold mA',
       'Temporary Ventricular Sens Setting mV',
       'Temporary Ventricular Stim Setting mA',
       'Thoracic Fluid Content (TFC) (NICOM)', 'Thrombin',
       'Tidal Volume (observed)', 'Tidal Volume (set)',
       'Tidal Volume (spontaneous)', 'Tobramycin (Peak)',
       'Tobramycin (Random)', 'Tobramycin (Trough)', 'Total Bilirubin',
       'Total Granulocyte Count (TGC)', 'Total PEEP Level',
       'Total Peripheral Resistance (TPR) (NICOM)',
       'Total Peripheral Resistance Index (TPRI) (NICOM)',
       'Total Protein', 'Transpulmonary Pressure (Exp. Hold)',
       'Transpulmonary Pressure (Insp. Hold)', 'Triglyceride',
       'Troponin-T', 'Ultrafiltrate Output', 'Unassisted Systole',
       'Uric Acid', 'Urine output_ApacheIV', 'UrineScore_ApacheIV',
       'VEN Lumen Volume', 'Vancomycin (Peak)', 'Vancomycin (Random)',
       'Vancomycin (Trough)', 'Vd/Vt Ratio', 'Venous CO2 Pressure',
       'Venous O2 Pressure', 'Ventilator Tank #1', 'Ventilator Tank #2',
       'Vital Cap', 'Volume In (PD)', 'Volume Out (PD)', 'Vti High',
       'WBC', 'WBCApacheIIValue', 'WBCScore_ApacheIV', 'WBC_ApacheIV',
       'WbcApacheIIScore', ], inplace=True) 

In [38]:
charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
0,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
1,23,124321,234044.0,223834,2157-10-21 20:00:00,2157-10-22 00:45:00,19050.0,3.0,3.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
2,23,124321,234044.0,223834,2157-10-21 22:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
3,23,124321,234044.0,223834,2157-10-22 00:00:00,2157-10-22 00:45:00,19050.0,2.0,2.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
4,36,165660,241249.0,223834,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,15.0,15.0,L/min,0,0,NaN,NaN,O2 Flow,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,45344,195325,276206.0,228379,2172-06-14 20:16:00,2172-06-14 20:18:00,14939.0,5.0,5.0,%,0,0,NaN,NaN,TFCd0 (NICOM),NICOM
9999996,46156,127108,258962.0,220245,2134-08-22 12:00:00,2134-08-22 12:54:00,20389.0,180.0,180.0,ml/min,0,0,NaN,NaN,CO2 production,Respiratory
9999997,49138,141813,296225.0,220245,2111-06-24 16:00:00,2111-06-24 18:50:00,20389.0,290.0,290.0,ml/min,1,0,NaN,NaN,CO2 production,Respiratory
9999998,46732,167513,223514.0,227645,2170-04-08 09:00:00,2170-04-08 09:43:00,16617.0,10.0,10.0,min,0,0,NaN,NaN,Breast Feed - L (Minutes),OB-GYN


In [39]:
charts[charts.LABEL.isin([ 'ABI Ankle BP L', 'ABI Ankle BP L (Impella)', 'ABI Ankle BP R','ABI Ankle BP L'])]
charts['LABEL'].mask(charts['LABEL']== 'ABI Ankle BP L (Impella)', 'ABI Ankle BP L', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ABI Ankle BP R', 'ABI Ankle BP L', inplace=True)
charts[charts.LABEL.isin(['ABI Ankle BP L'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9999659,31112,146735,213400.0,224317,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,75.0,75.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999660,31112,146735,213400.0,224317,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,34.0,34.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999661,31112,146735,213400.0,224317,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,70.0,70.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999662,40897,184495,206064.0,224317,2132-01-30 21:17:00,2132-01-30 21:34:00,20675.0,90.0,90.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999663,43089,114974,202624.0,224317,2120-02-22 15:57:00,2120-02-22 15:58:00,18730.0,158.0,158.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999664,46041,176979,208784.0,224317,2160-05-24 12:50:00,2160-05-24 12:54:00,18424.0,88.0,88.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999665,46041,176979,208784.0,224317,2160-05-25 08:21:00,2160-05-25 08:22:00,18424.0,84.0,84.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999666,31112,146735,213400.0,224318,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,130.0,130.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella
9999667,31112,146735,213400.0,224318,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,60.0,60.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella
9999668,31112,146735,213400.0,224318,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,75.0,75.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella


In [40]:
charts[charts.LABEL.isin(['Impaired Skin Length #1','Impaired Skin Length #2','Impaired Skin Length #3','Impaired Skin Length #4','Impaired Skin Length #5','Impaired Skin Length #6','Impaired Skin Length #7','Impaired Skin Length #8','Impaired Skin Length #9','Impaired Skin Length #10',])]


charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #2', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #3', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #4', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #5', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #6', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #7', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #8', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #9', 'Impaired Skin Length #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Length #10', 'Impaired Skin Length #1', inplace=True)


charts[charts.LABEL.isin(['Impaired Skin Length #1'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9376098,109,176760,256504.0,224562,2142-01-22 09:00:00,2142-01-22 09:57:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9376099,109,176760,256504.0,224562,2142-01-22 11:00:00,2142-01-22 11:48:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9376100,109,176760,256504.0,224562,2142-01-22 12:28:00,2142-01-22 16:29:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9376101,124,138376,256064.0,224562,2166-01-12 00:59:00,2166-01-12 00:59:00,14986.0,18.0,18.0,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9376102,124,138376,256064.0,224562,2166-01-09 23:37:00,2166-01-09 23:37:00,19169.0,18.0,18.0,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998052,46836,199740,262164.0,227603,2165-04-07 20:00:00,2165-04-07 21:28:00,14332.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9998053,46836,199740,262164.0,227603,2165-04-05 20:50:00,2165-04-05 20:50:00,18928.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9998054,46836,199740,262164.0,227603,2165-04-08 04:00:00,2165-04-08 04:35:00,14332.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment
9998055,48706,135096,271254.0,227603,2179-11-20 14:30:00,2179-11-20 20:28:00,15659.0,3.0,3.0,cm,0,0,NaN,NaN,Impaired Skin Length #1,Skin - Impairment


In [41]:
charts[charts.LABEL.isin(['Impaired Skin Depth #1','Impaired Skin Depth #2','Impaired Skin Depth #3','Impaired Skin Depth #4','Impaired Skin Depth #5','Impaired Skin Depth #6','Impaired Skin Depth #7','Impaired Skin Depth #8','Impaired Skin Depth #9','Impaired Skin Depth #10',])]


charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #2', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #3', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #4', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #5', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #6', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #7', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #8', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #9', 'Impaired Skin Depth #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Depth #10', 'Impaired Skin Depth #1', inplace=True)

charts[charts.LABEL.isin(['Impaired Skin Width #1'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9393107,109,176760,256504.0,224846,2142-01-22 09:00:00,2142-01-22 09:57:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393108,109,176760,256504.0,224846,2142-01-22 11:00:00,2142-01-22 11:48:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393109,109,176760,256504.0,224846,2142-01-22 12:28:00,2142-01-22 16:29:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393110,124,138376,256064.0,224846,2166-01-12 00:59:00,2166-01-12 00:59:00,14986.0,22.0,22.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393111,124,138376,256064.0,224846,2166-01-09 23:37:00,2166-01-09 23:37:00,19169.0,22.0,22.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9409684,49379,129372,201098.0,224846,2124-12-28 08:00:00,2124-12-28 13:39:00,14431.0,7.0,7.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9409685,48925,127507,260855.0,224846,2129-01-27 08:04:00,2129-01-27 08:04:00,15666.0,11.0,11.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9409686,49379,129372,201098.0,224846,2124-12-31 08:32:00,2124-12-31 11:32:00,14431.0,7.0,7.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9409687,49379,129372,201098.0,224846,2125-01-04 08:00:00,2125-01-04 11:32:00,14431.0,7.0,7.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment


In [42]:
charts[charts.LABEL.isin(['Impaired Skin Width #1','Impaired Skin Width #2','Impaired Skin Width #3','Impaired Skin Width #4','Impaired Skin Width #5','Impaired Skin Width #6','Impaired Skin Width #7','Impaired Skin Width #8','Impaired Skin Width #9','Impaired Skin Width #10',])]

charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #2', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #3', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #4', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #5', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #6', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #7', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #8', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #9', 'Impaired Skin Width #1', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Impaired Skin Width #10', 'Impaired Skin Width #1', inplace=True)

charts[charts.LABEL.isin(['Impaired Skin Width #1'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9393107,109,176760,256504.0,224846,2142-01-22 09:00:00,2142-01-22 09:57:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393108,109,176760,256504.0,224846,2142-01-22 11:00:00,2142-01-22 11:48:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393109,109,176760,256504.0,224846,2142-01-22 12:28:00,2142-01-22 16:29:00,17807.0,0.5,0.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393110,124,138376,256064.0,224846,2166-01-12 00:59:00,2166-01-12 00:59:00,14986.0,22.0,22.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9393111,124,138376,256064.0,224846,2166-01-09 23:37:00,2166-01-09 23:37:00,19169.0,22.0,22.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998266,46836,199740,262164.0,227615,2165-04-07 20:00:00,2165-04-07 21:28:00,14332.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9998267,46836,199740,262164.0,227615,2165-04-05 20:50:00,2165-04-05 20:50:00,18928.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9998268,46836,199740,262164.0,227615,2165-04-08 04:00:00,2165-04-08 04:35:00,14332.0,1.5,1.5,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment
9998269,48706,135096,271254.0,227615,2179-11-20 14:30:00,2179-11-20 20:28:00,15659.0,1.0,1.0,cm,0,0,NaN,NaN,Impaired Skin Width #1,Skin - Impairment


In [43]:

#FEATURE SET 3
charts[charts.LABEL.isin(['Manual Blood Pressure Diastolic Right','Manual Blood Pressure Diastolic Left',])]

charts['LABEL'].mask(charts['LABEL']== 'Manual Blood Pressure Diastolic Left', 'Manual Blood Pressure Diastolic Right', inplace=True)
charts[charts.LABEL.isin(['Manual Blood Pressure Diastolic Right'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9304992,107,174162,264253.0,224643,2122-05-15 15:15:00,2122-05-15 15:58:00,14706.0,60.0,60.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9304993,107,174162,264253.0,224643,2122-05-15 17:00:00,2122-05-15 17:46:00,14706.0,60.0,60.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9304994,109,126055,236124.0,224643,2141-10-23 00:22:00,2141-10-23 00:23:00,14626.0,138.0,138.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9304995,291,125726,275109.0,224643,2106-04-17 13:00:00,2106-04-17 13:13:00,15564.0,80.0,80.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9304996,975,175734,286445.0,224643,2142-05-17 20:00:00,2142-05-17 19:55:00,14542.0,62.0,62.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9986928,48697,117793,217153.0,227242,2173-12-09 12:00:00,2173-12-09 12:54:00,16948.0,51.0,51.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9986929,48672,162488,200665.0,227242,2166-03-14 20:14:00,2166-03-14 20:15:00,16948.0,57.0,57.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9986930,49485,119002,251750.0,227242,2181-04-25 03:00:00,2181-04-25 03:36:00,16948.0,66.0,66.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs
9986931,49291,104004,213105.0,227242,2104-02-14 05:30:00,2104-02-14 05:30:00,16948.0,72.0,72.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Diastolic Right,Routine Vital Signs


In [44]:
#FEATURE SET 3
 
charts[charts.LABEL.isin([ 'Manual Blood Pressure Systolic Left', 'Manual Blood Pressure Systolic Right',])]

charts['LABEL'].mask(charts['LABEL']==  'Manual Blood Pressure Systolic Left', 'Manual Blood Pressure Systolic Right', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Manual Blood Pressure Systolic Left', 'Manual Blood Pressure Systolic Right', inplace=True)
charts[charts.LABEL.isin(['Manual Blood Pressure Systolic Right'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9244582,107,174162,264253.0,224167,2122-05-15 09:00:00,2122-05-15 09:51:00,14706.0,102.0,102.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9244583,107,174162,264253.0,224167,2122-05-15 10:00:00,2122-05-15 10:25:00,14706.0,86.0,86.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9244584,107,174162,264253.0,224167,2122-05-15 12:00:00,2122-05-15 13:14:00,14706.0,122.0,122.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9244585,107,174162,264253.0,224167,2122-05-15 14:00:00,2122-05-15 14:01:00,14706.0,120.0,120.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9244586,107,174162,264253.0,224167,2122-05-15 15:00:00,2122-05-15 15:57:00,14706.0,102.0,102.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9930800,48672,162488,200665.0,227243,2166-03-14 20:14:00,2166-03-14 20:15:00,16948.0,128.0,128.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9930801,49358,158550,209348.0,227243,2100-09-10 20:00:00,2100-09-10 20:03:00,19710.0,120.0,120.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9930802,49485,119002,251750.0,227243,2181-04-25 03:00:00,2181-04-25 03:36:00,16948.0,116.0,116.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs
9930803,49291,104004,213105.0,227243,2104-02-14 05:30:00,2104-02-14 05:30:00,16948.0,138.0,138.0,mmHg,0,0,NaN,NaN,Manual Blood Pressure Systolic Right,Routine Vital Signs


In [45]:

charts[charts.LABEL.isin(['ABI Brachial BP L', 'ABI Brachial BP R',
       'ABI Brachial BP R (Impella)'])]
charts['LABEL'].mask(charts['LABEL']== 'ABI Brachial BP R', 'ABI Brachial BP L', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ABI Brachial BP R (Impella)', 'ABI Brachial BP L', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ABI Brachial BP R ', 'ABI Brachial BP L', inplace=True)
charts[charts.LABEL.isin(['ABI Brachial BP L'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9999256,21318,186933,233039.0,224314,2188-10-08 10:29:00,2188-10-08 10:52:00,17122.0,-2.0,-2.0,mmHg,0,1,NaN,NaN,ABI Brachial BP L,Impella
9999257,31112,146735,213400.0,224314,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,110.0,110.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999258,31112,146735,213400.0,224314,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,100.0,100.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999259,31112,146735,213400.0,224314,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,75.0,75.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999260,40897,184495,206064.0,224314,2132-01-30 21:17:00,2132-01-30 21:34:00,20675.0,110.0,110.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999261,46041,176979,208784.0,224314,2160-05-24 12:50:00,2160-05-24 12:54:00,18424.0,100.0,100.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999262,46041,176979,208784.0,224314,2160-05-25 08:21:00,2160-05-25 08:22:00,18424.0,110.0,110.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,Impella
9999652,31112,146735,213400.0,224315,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,120.0,120.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,IABP
9999653,31112,146735,213400.0,224315,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,100.0,100.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,IABP
9999654,31112,146735,213400.0,224315,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,80.0,80.0,mmHg,0,0,NaN,NaN,ABI Brachial BP L,IABP


In [46]:
charts[charts.LABEL.isin([ 'ABI Ankle BP L', 'ABI Ankle BP L (Impella)', 'ABI Ankle BP R','ABI Ankle BP L'])]
charts['LABEL'].mask(charts['LABEL']== 'ABI Ankle BP L (Impella)', 'ABI Ankle BP L', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ABI Ankle BP R', 'ABI Ankle BP L', inplace=True)
charts[charts.LABEL.isin(['ABI Ankle BP L'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9999659,31112,146735,213400.0,224317,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,75.0,75.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999660,31112,146735,213400.0,224317,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,34.0,34.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999661,31112,146735,213400.0,224317,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,70.0,70.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999662,40897,184495,206064.0,224317,2132-01-30 21:17:00,2132-01-30 21:34:00,20675.0,90.0,90.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999663,43089,114974,202624.0,224317,2120-02-22 15:57:00,2120-02-22 15:58:00,18730.0,158.0,158.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999664,46041,176979,208784.0,224317,2160-05-24 12:50:00,2160-05-24 12:54:00,18424.0,88.0,88.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999665,46041,176979,208784.0,224317,2160-05-25 08:21:00,2160-05-25 08:22:00,18424.0,84.0,84.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,IABP
9999666,31112,146735,213400.0,224318,2176-11-01 16:30:00,2176-11-01 16:38:00,14214.0,130.0,130.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella
9999667,31112,146735,213400.0,224318,2176-10-31 22:00:00,2176-10-31 21:56:00,19062.0,60.0,60.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella
9999668,31112,146735,213400.0,224318,2176-11-01 11:36:00,2176-11-01 11:37:00,14214.0,75.0,75.0,mmHg,0,0,NaN,NaN,ABI Ankle BP L,Impella


In [47]:
charts[charts.LABEL.isin(['APACHE II', 'APACHE II PDR - Adjusted', 'APACHE II Predecited Death Rate',])]
charts['LABEL'].mask(charts['LABEL']== 'APACHE II PDR - Adjusted', 'APACHE II', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'APACHE II Predecited Death Rate', 'APACHE II', inplace=True)
charts[charts.LABEL.isin(['APACHE II'])]
 

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9950344,711,158767,270525.0,226743,2185-03-23 14:26:00,2185-03-23 14:26:00,19006.0,21.0,21.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950345,711,158767,270525.0,226743,2185-03-23 15:07:00,2185-03-23 15:07:00,19006.0,21.0,21.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950346,10757,153982,256332.0,226743,2180-11-14 00:26:00,2180-11-14 00:26:00,14770.0,11.0,11.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950347,22792,192087,271490.0,226743,2173-10-23 07:07:00,2173-10-23 07:07:00,15511.0,14.0,14.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950348,27366,174736,248077.0,226743,2143-03-01 09:16:00,2143-03-01 09:16:00,16671.0,18.0,18.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950349,27366,174736,248077.0,226743,2143-03-01 11:24:00,2143-03-01 11:24:00,15237.0,0.0,0.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950350,27554,186902,229620.0,226743,2196-11-07 14:27:00,2196-11-07 14:27:00,15103.0,44.0,44.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950351,28819,135454,212633.0,226743,2116-02-11 02:18:00,2116-02-11 02:18:00,19006.0,24.0,24.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950352,29070,115071,232563.0,226743,2145-03-13 17:50:00,2145-03-13 17:50:00,15052.0,24.0,24.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II
9950353,29070,115071,232563.0,226743,2145-03-13 17:39:00,2145-03-13 17:39:00,15052.0,24.0,24.0,NaN,0,0,NaN,NaN,APACHE II,Scores - APACHE II


In [48]:
charts[charts.LABEL.isin(['Arterial Blood Pressure Alarm - High', 'Arterial Blood Pressure Alarm - Low', 'Arterial Blood Pressure diastolic','Arterial Blood Pressure mean',])]
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure Alarm - Low', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure Alarm - High', 'Arterial Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure diastolic', 'Arterial Blood Pressure mean', inplace=True)
charts[charts.LABEL.isin(['Arterial Blood Pressure mean'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
6848888,23,124321,234044.0,220051,2157-10-22 11:00:00,2157-10-22 11:05:00,19907.0,67.0,67.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
6848889,23,124321,234044.0,220051,2157-10-22 12:00:00,2157-10-22 12:49:00,19907.0,72.0,72.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
6848890,23,124321,234044.0,220051,2157-10-22 13:00:00,2157-10-22 13:35:00,19907.0,55.0,55.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
6848891,23,124321,234044.0,220051,2157-10-22 14:00:00,2157-10-22 15:03:00,16890.0,55.0,55.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
6848892,23,124321,234044.0,220051,2157-10-22 15:00:00,2157-10-22 15:03:00,16890.0,56.0,56.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7660127,48925,127507,260855.0,220058,2129-01-31 20:00:00,2129-01-31 21:41:00,20115.0,110.0,110.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Alarms
7660128,48925,127507,260855.0,220058,2129-01-22 20:00:00,2129-01-23 02:01:00,16643.0,120.0,120.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Alarms
7660129,49379,129372,201098.0,220058,2124-12-31 08:00:00,2124-12-31 07:23:00,14431.0,110.0,110.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Alarms
7660130,48925,127507,260855.0,220058,2129-02-09 08:00:00,2129-02-09 08:25:00,19593.0,180.0,180.0,mmHg,0,0,NaN,NaN,Arterial Blood Pressure mean,Alarms


In [49]:
 charts[charts.LABEL.isin([ 'ART BP Diastolic', 'ART BP mean', 'ART Blood Pressure Alarm - High','ART Blood Pressure Alarm - Low',])]
charts['LABEL'].mask(charts['LABEL']== 'ART BP Diastolic', 'ART BP mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ART Blood Pressure Alarm - High', 'ART BP mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'ART Blood Pressure Alarm - Low', 'ART BP mean', inplace=True)
charts[charts.LABEL.isin(['ART BP mean'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9318285,109,139061,257358.0,227537,2141-09-11 11:14:00,2141-09-11 11:20:00,21297.0,250.0,250.0,mmHg,0,0,NaN,NaN,ART BP mean,Alarms
9318286,109,139061,257358.0,227537,2141-09-11 16:00:00,2141-09-11 16:03:00,14411.0,205.0,205.0,mmHg,0,0,NaN,NaN,ART BP mean,Alarms
9318287,109,139061,257358.0,227537,2141-09-11 20:00:00,2141-09-11 19:34:00,17248.0,200.0,200.0,mmHg,0,0,NaN,NaN,ART BP mean,Alarms
9318288,109,139061,257358.0,227537,2141-09-12 00:00:00,2141-09-12 01:19:00,17248.0,180.0,180.0,mmHg,0,0,NaN,NaN,ART BP mean,Alarms
9318289,109,139061,257358.0,227537,2141-09-12 04:00:00,2141-09-12 06:20:00,17248.0,180.0,180.0,mmHg,0,0,NaN,NaN,ART BP mean,Alarms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491570,49358,158550,209348.0,225312,2100-09-24 17:00:00,2100-09-24 17:04:00,14542.0,106.0,106.0,mmHg,0,0,NaN,NaN,ART BP mean,Routine Vital Signs
9491571,49358,158550,209348.0,225312,2100-09-24 18:00:00,2100-09-24 18:15:00,14542.0,101.0,101.0,mmHg,0,0,NaN,NaN,ART BP mean,Routine Vital Signs
9491572,49358,158550,209348.0,225312,2100-09-24 19:00:00,2100-09-24 19:48:00,21202.0,110.0,110.0,mmHg,0,0,NaN,NaN,ART BP mean,Routine Vital Signs
9491573,49358,158550,209348.0,225312,2100-09-24 20:00:00,2100-09-24 19:53:00,21202.0,104.0,104.0,mmHg,0,0,NaN,NaN,ART BP mean,Routine Vital Signs


In [50]:
charts[charts.LABEL.isin(['Ventilator Tank #2', 'Ventilator Tank #1',])]
charts['LABEL'].mask(charts['LABEL']==  'Ventilator Tank #2', 'Ventilator Tank #1', inplace=True)
charts[charts.LABEL.isin(['Ventilator Tank #1'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9104689,36,165660,241249.0,227565,2134-05-13 20:00:00,2134-05-13 20:29:00,20111.0,2800.0,2800.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9104690,36,165660,241249.0,227565,2134-05-12 16:00:00,2134-05-12 16:29:00,18428.0,1200.0,1200.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9104691,36,165660,241249.0,227565,2134-05-12 21:00:00,2134-05-13 01:10:00,17698.0,2500.0,2500.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9104692,36,165660,241249.0,227565,2134-05-13 01:00:00,2134-05-13 01:12:00,17698.0,2500.0,2500.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9104693,36,165660,241249.0,227565,2134-05-13 04:00:00,2134-05-13 04:38:00,17698.0,2500.0,2500.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9146173,48925,127507,260855.0,227566,2129-01-22 20:00:00,2129-01-22 20:48:00,14612.0,1800.0,1800.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9146174,49379,129372,201098.0,227566,2124-12-28 08:00:00,2124-12-28 08:21:00,18843.0,1600.0,1600.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9146175,48925,127507,260855.0,227566,2129-02-03 04:00:00,2129-02-03 05:06:00,18843.0,2500.0,2500.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory
9146176,48939,139380,270705.0,227566,2187-08-14 20:00:00,2187-08-14 20:14:00,16924.0,2200.0,2200.0,NaN,0,0,NaN,NaN,Ventilator Tank #1,Respiratory


In [51]:
charts[charts.LABEL.isin(['MDI #1 Puff', 'MDI #2 Puff', 'MDI #3 Puff'])]
charts['LABEL'].mask(charts['LABEL']==   'MDI #2 Puff', 'MDI #1 Puff', inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'MDI #3 Puff', 'MDI #1 Puff', inplace=True)
charts[charts.LABEL.isin(['MDI #1 Puff'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9083567,36,165660,241249.0,224169,2134-05-13 16:00:00,2134-05-13 16:29:00,20290.0,6.0,6.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9083568,36,165660,241249.0,224169,2134-05-12 17:52:00,2134-05-12 17:52:00,18428.0,6.0,6.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9083569,36,165660,241249.0,224169,2134-05-12 22:00:00,2134-05-13 01:11:00,17698.0,6.0,6.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9083570,36,165660,241249.0,224169,2134-05-13 01:00:00,2134-05-13 01:14:00,17698.0,6.0,6.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9083571,36,165660,241249.0,224169,2134-05-13 04:00:00,2134-05-13 04:45:00,17698.0,6.0,6.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9628562,47483,169324,202941.0,224175,2107-05-16 20:00:00,2107-05-16 20:55:00,14612.0,1.0,1.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9628563,47511,132223,210266.0,224175,2139-04-17 08:00:00,2139-04-17 08:03:00,17857.0,2.0,2.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9628564,48059,101819,216027.0,224175,2161-12-01 07:00:00,2161-12-01 07:42:00,17443.0,4.0,4.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory
9628565,48059,101819,216027.0,224175,2161-11-30 20:00:00,2161-11-30 20:05:00,14612.0,4.0,4.0,NaN,0,0,NaN,NaN,MDI #1 Puff,Respiratory


In [52]:
charts[charts.LABEL.isin(['LDL measured','LDL calculated'])]
charts['LABEL'].mask(charts['LABEL']==   'LDL calculated', 'LDL measured', inplace=True)
charts[charts.LABEL.isin(['LDL measured'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9039023,34,144319,290505.0,225671,2191-02-24 04:32:00,2191-02-24 13:17:00,20889.0,105.0,105.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9039024,188,132401,246866.0,225671,2161-12-11 01:18:00,2161-12-11 06:14:00,20889.0,74.0,74.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9039025,249,149546,269035.0,225671,2155-02-04 05:29:00,2155-02-04 13:51:00,20889.0,56.0,56.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9039026,249,149546,263055.0,225671,2155-02-08 03:34:00,2155-02-08 13:26:00,20889.0,33.0,33.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9039027,236,191151,NaN,225671,2139-03-20 06:19:00,2139-03-20 22:21:00,20889.0,48.0,48.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991023,47906,128544,273396.0,227441,2186-10-22 02:32:00,2186-10-22 13:40:00,20889.0,50.0,50.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9991024,47684,148216,226834.0,227441,2150-11-09 10:13:00,2150-11-10 11:05:00,20889.0,55.0,55.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs
9991025,49094,129618,202878.0,227441,2177-05-01 22:58:00,2177-05-02 10:24:00,20889.0,130.0,130.0,mg/dL,1,0,NaN,NaN,LDL measured,Labs
9991026,48755,111320,212363.0,227441,2160-07-27 01:40:00,2160-07-27 10:29:00,20889.0,77.0,77.0,mg/dL,0,0,NaN,NaN,LDL measured,Labs


In [53]:
charts[charts.LABEL.isin(['Intra Cranial Pressure', 'Intra Cranial Pressure #2', 'Intra Cranial Pressure Alarm - High', 'Intra Cranial Pressure Alarm - Low',])]
charts['LABEL'].mask(charts['LABEL']==   'Intra Cranial Pressure #2', 'Intra Cranial Pressure', inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Intra Cranial Pressure Alarm - High', 'Intra Cranial Pressure', inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Intra Cranial Pressure Alarm - Low', 'Intra Cranial Pressure', inplace=True)
charts[charts.LABEL.isin(['Intra Cranial Pressure'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9966075,890,195962,274222.0,220765,2180-12-08 22:15:00,2180-12-08 22:37:00,14213.0,17.0,17.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9966076,890,195962,274222.0,220765,2180-12-08 22:30:00,2180-12-08 22:37:00,14213.0,16.0,16.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9966077,890,195962,274222.0,220765,2180-12-08 23:00:00,2180-12-08 23:01:00,14213.0,21.0,21.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9966078,890,195962,274222.0,220765,2180-12-08 23:15:00,2180-12-08 23:20:00,14213.0,20.0,20.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9966079,890,195962,274222.0,220765,2180-12-09 00:00:00,2180-12-09 00:09:00,14213.0,19.0,19.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999800,44459,101062,232100.0,227989,2171-02-13 18:00:00,2171-02-13 18:50:00,20074.0,78.0,78.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9999801,44459,101062,232100.0,227989,2171-02-13 08:00:00,2171-02-13 12:21:00,20074.0,80.0,80.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9999802,44459,101062,232100.0,227989,2171-02-13 09:00:00,2171-02-13 12:21:00,20074.0,82.0,82.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics
9999803,44459,101062,232100.0,227989,2171-02-13 10:00:00,2171-02-13 12:21:00,20074.0,80.0,80.0,mmHg,0,0,NaN,NaN,Intra Cranial Pressure,Hemodynamics


In [54]:


charts[charts.LABEL.isin(['Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean', 'Non Invasive Blood Pressure Alarm - High', 'Non Invasive Blood Pressure Alarm - Low',])]
charts['LABEL'].mask(charts['LABEL']==   'Non Invasive Blood Pressure diastolic', 'Non Invasive Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Non Invasive Blood Pressure Alarm - High', 'Non Invasive Blood Pressure mean', inplace=True)
charts['LABEL'].mask(charts['LABEL']==  'Non Invasive Blood Pressure Alarm - Low', 'Non Invasive Blood Pressure mean', inplace=True)
charts[charts.LABEL.isin(['Non Invasive Blood Pressure mean'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
1682761,36,165660,241249.0,220180,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,72.0,72.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
1682762,36,165660,241249.0,220180,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,53.0,53.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
1682763,36,165660,241249.0,220180,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,61.0,61.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
1682764,36,165660,241249.0,220180,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,75.0,75.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
1682765,34,144319,290505.0,220180,2191-02-23 05:30:00,2191-02-23 07:45:00,17741.0,66.0,66.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421802,49382,174526,206258.0,220181,2194-12-13 00:01:00,2194-12-13 00:21:00,14719.0,70.0,70.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
2421803,49382,174526,206258.0,220181,2194-12-13 01:00:00,2194-12-13 01:43:00,14719.0,64.0,64.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
2421804,49382,174526,206258.0,220181,2194-12-13 02:00:00,2194-12-13 02:12:00,14719.0,76.0,76.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs
2421805,49407,183964,291495.0,220181,2119-04-12 20:26:00,2119-04-12 21:12:00,14939.0,107.0,107.0,mmHg,0,0,NaN,NaN,Non Invasive Blood Pressure mean,Routine Vital Signs


In [55]:

charts[charts.LABEL.isin(['O2 saturation pulseoxymetry','O2 Saturation Pulseoxymetry Alarm - Low', 'O2 Saturation Pulseoxymetry Alarm - High'])]
charts['LABEL'].mask(charts['LABEL']==   'O2 Saturation Pulseoxymetry Alarm - Low','O2 saturation pulseoxymetry', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'O2 Saturation Pulseoxymetry Alarm - High','O2 saturation pulseoxymetry', inplace=True)
charts[charts.LABEL.isin(['O2 saturation pulseoxymetry'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
3190760,36,165660,241249.0,220277,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,93.0,93.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
3190761,36,165660,241249.0,220277,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,91.0,91.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
3190762,36,165660,241249.0,220277,2134-05-12 15:00:00,2134-05-12 16:51:00,17525.0,91.0,91.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
3190763,36,165660,241249.0,220277,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,92.0,92.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
3190764,34,144319,290505.0,220277,2191-02-23 07:44:00,2191-02-23 07:45:00,17741.0,100.0,100.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786808,48915,140003,291845.0,223770,2159-03-24 20:00:00,2159-03-24 20:21:00,18804.0,92.0,92.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Alarms
7786809,48925,127507,260855.0,223770,2129-01-22 10:13:00,2129-01-22 10:14:00,19562.0,90.0,90.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Alarms
7786810,48939,139380,270705.0,223770,2187-08-13 08:00:00,2187-08-13 07:42:00,18928.0,90.0,90.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Alarms
7786811,49382,174526,206258.0,223770,2194-12-12 22:05:00,2194-12-12 22:05:00,14719.0,92.0,92.0,%,0,0,NaN,NaN,O2 saturation pulseoxymetry,Alarms


In [56]:
charts[charts.LABEL.isin(['Gentamicin (Random)','Gentamicin (Trough)','Gentamicin (Peak)'])]
charts['LABEL'].mask(charts['LABEL']==  'Gentamicin (Trough)', 'Gentamicin (Random)' , inplace=True)
charts['LABEL'].mask(charts['LABEL']==  'Gentamicin (Peak)', 'Gentamicin (Random)' , inplace=True)

charts[charts.LABEL.isin(['Gentamicin (Random)'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9948926,605,115545,248569.0,227447,2197-11-22 16:47:00,2197-11-22 19:43:00,20889.0,3.6,3.6,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9948927,605,115545,248569.0,227447,2197-11-22 07:49:00,2197-11-22 11:59:00,20889.0,1.2,1.2,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9948928,773,170261,213086.0,227447,2109-03-04 03:25:00,2109-03-04 11:36:00,20889.0,1.7,1.7,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9948929,773,170261,213086.0,227447,2109-03-12 06:47:00,2109-03-12 14:57:00,20889.0,1.9,1.9,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9948930,773,170261,213086.0,227447,2109-03-14 04:18:00,2109-03-14 11:42:00,20889.0,2.3,2.3,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9952712,44500,101872,219297.0,227448,2164-06-10 00:19:00,2164-06-10 02:25:00,20889.0,2.4,2.4,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9952713,44500,101872,219297.0,227448,2164-06-13 23:07:00,2164-06-14 01:32:00,20889.0,6.2,6.2,ug/mL,0,0,NaN,NaN,Gentamicin (Random),Labs
9952714,44500,101872,219297.0,227448,2164-06-08 22:59:00,2164-06-09 01:09:00,20889.0,3.1,3.1,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs
9952715,47912,182705,261684.0,227448,2179-10-29 15:50:00,2179-10-29 16:59:00,20889.0,11.8,11.8,ug/mL,1,0,NaN,NaN,Gentamicin (Random),Labs


In [57]:
charts[charts.LABEL.isin(['Temp Axillary [F]','Temp Rectal [F]',
                          'Blood Temperature CCO (C)', 'Temperature Fahrenheit', 'Temperature Celsius', 'Skin [Temperature]', 
                          'Temperature C', 'Temperature C (calc)', 'Temperature F', 'Temperature F (calc)',])]
charts.loc[charts.LABEL== 'Temperature Fahrenheit', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temperature F', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temperature F (calc)', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temp Axillary [F]', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)
charts.loc[charts.LABEL== 'Temp Rectal [F]', 'VALUENUM'] = ((charts.VALUENUM -32) * 5./9)

In [58]:
charts['LABEL'].mask(charts['LABEL']== 'Temperature Fahrenheit', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature F', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature F (calc)', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Blood Temperature CCO (C)', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Skin [Temperature]', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature C', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temperature C (calc)', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temp Axillary [F]', 'Temperature Celsius', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Temp Rectal [F]', 'Temperature Celsius', inplace=True)
charts[charts.LABEL.isin(['Temperature Celsius'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
4086912,36,165660,241249.0,223761,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,97.3,36.277778,?F,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
4086913,34,144319,290505.0,223761,2191-02-23 08:00:00,2191-02-23 08:23:00,17741.0,97.0,36.111111,?F,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
4086914,23,124321,234044.0,223761,2157-10-22 16:00:00,2157-10-22 15:53:00,19907.0,98.4,36.888889,?F,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
4086915,23,124321,234044.0,223761,2157-10-21 12:05:00,2157-10-21 12:05:00,16978.0,95.8,35.444444,?F,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
4086916,23,124321,234044.0,223761,2157-10-21 14:00:00,2157-10-21 13:56:00,16978.0,97.2,36.222222,?F,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9623943,49311,136786,272939.0,226329,2121-06-17 18:00:00,2121-06-17 18:04:00,16960.0,37.9,37.900000,?C,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
9623944,49311,136786,272939.0,226329,2121-06-17 19:00:00,2121-06-17 19:01:00,16960.0,38.0,38.000000,?C,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
9623945,49311,136786,272939.0,226329,2121-06-17 05:17:00,2121-06-17 05:26:00,20335.0,37.3,37.300000,?C,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs
9623946,49311,136786,272939.0,226329,2121-06-17 06:00:00,2121-06-17 06:14:00,20335.0,37.4,37.400000,?C,0,0,NaN,NaN,Temperature Celsius,Routine Vital Signs


In [59]:
charts[charts.LABEL.isin(['ART BP Systolic', 'NBP [Systolic]', 'Arterial BP [Systolic]', 
        'BP Cuff [Systolic]', 'Manual BP [Systolic]', 'BP UAC [Systolic]', 'BP Right Leg [Systolic]',
        'Pulmonary Artery Pressure systolic', 'Non Invasive Blood Pressure systolic', 'Arterial Blood Pressure systolic'])]
charts['LABEL'].mask(charts['LABEL']== 'NBP [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial BP [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'BP Cuff [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Manual BP [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'BP UAC [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'BP Right Leg [Systolic]', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Pulmonary Artery Pressure systolic', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Non Invasive Blood Pressure systolic', 'ART BP Systolic', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial Blood Pressure systolic', 'ART BP Systolic', inplace=True)
charts[charts.LABEL.isin(['ART BP Systolic'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
1313517,36,165660,241249.0,220179,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,137.0,137.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Routine Vital Signs
1313518,36,165660,241249.0,220179,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,118.0,118.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Routine Vital Signs
1313519,36,165660,241249.0,220179,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,130.0,130.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Routine Vital Signs
1313520,36,165660,241249.0,220179,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,117.0,117.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Routine Vital Signs
1313521,34,144319,290505.0,220179,2191-02-23 07:34:00,2191-02-23 07:45:00,17741.0,135.0,135.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9809328,48939,139380,270705.0,220059,2187-08-13 04:00:00,2187-08-13 04:02:00,18928.0,33.0,33.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Hemodynamics
9809329,48939,139380,270705.0,220059,2187-08-13 04:15:00,2187-08-13 04:16:00,18928.0,35.0,35.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Hemodynamics
9809330,48939,139380,270705.0,220059,2187-08-13 04:30:00,2187-08-13 04:33:00,18928.0,34.0,34.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Hemodynamics
9809331,48939,139380,270705.0,220059,2187-08-13 04:45:00,2187-08-13 04:46:00,18928.0,26.0,26.0,mmHg,0,0,NaN,NaN,ART BP Systolic,Hemodynamics


In [60]:
charts[charts.LABEL.isin(['Heart Rate', 'Heart Rate Alarm - Low','Heart rate Alarm - High'])]

charts['LABEL'].mask(charts['LABEL']== 'Heart Rate Alarm - Low', 'Heart Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Heart rate Alarm - High', 'Heart Rate', inplace=True)
charts[charts.LABEL.isin(['Heart Rate'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
534651,36,165660,241249.0,220045,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,86.0,86.0,bpm,0,0,NaN,NaN,Heart Rate,Routine Vital Signs
534652,36,165660,241249.0,220045,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,85.0,85.0,bpm,0,0,NaN,NaN,Heart Rate,Routine Vital Signs
534653,36,165660,241249.0,220045,2134-05-12 15:00:00,2134-05-12 16:50:00,17525.0,87.0,87.0,bpm,0,0,NaN,NaN,Heart Rate,Routine Vital Signs
534654,36,165660,241249.0,220045,2134-05-12 16:00:00,2134-05-12 16:50:00,17525.0,91.0,91.0,bpm,0,0,NaN,NaN,Heart Rate,Routine Vital Signs
534655,34,144319,290505.0,220045,2191-02-23 07:34:00,2191-02-23 10:53:00,17741.0,44.0,44.0,bpm,0,0,NaN,NaN,Heart Rate,Routine Vital Signs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6371761,48915,140003,291845.0,220047,2159-03-24 20:00:00,2159-03-24 20:21:00,18804.0,50.0,50.0,bpm,0,0,NaN,NaN,Heart Rate,Alarms
6371762,48925,127507,260855.0,220047,2129-01-22 10:13:00,2129-01-22 10:14:00,19562.0,60.0,60.0,bpm,0,0,NaN,NaN,Heart Rate,Alarms
6371763,48939,139380,270705.0,220047,2187-08-13 08:00:00,2187-08-13 07:42:00,18928.0,50.0,50.0,bpm,0,0,NaN,NaN,Heart Rate,Alarms
6371764,49382,174526,206258.0,220047,2194-12-12 22:05:00,2194-12-12 22:05:00,14719.0,60.0,60.0,bpm,0,0,NaN,NaN,Heart Rate,Alarms


In [61]:
charts[charts.LABEL.isin(['Potassium (whole blood)', 'Potassium  (3.5-5.3)', 'Potassium (3.5-5.3)', 'Potassium (serum)'])]
charts['LABEL'].mask(charts['LABEL']== 'Potassium  (3.5-5.3)', 'Potassium (whole blood)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Potassium (3.5-5.3)', 'Potassium (whole blood)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Potassium (serum)', 'Potassium (whole blood)', inplace=True)
#cd.VALUEUOM.fillna('mmHg', inplace=True) # this is not necessary as all have the same unit (mmHg)
charts[charts.LABEL.isin(['Potassium (whole blood)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
8592302,23,124321,234044.0,227442,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,3.9,3.9,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
8592303,23,124321,234044.0,227442,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,3.4,3.4,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
8592304,36,165660,241249.0,227442,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,4.2,4.2,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
8592305,34,144319,290505.0,227442,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,4.6,4.6,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
8592306,34,144319,290505.0,227442,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,3.9,3.9,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261679,49311,136786,272939.0,227464,2121-06-17 06:03:00,2121-06-17 06:07:00,20889.0,4.6,4.6,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
9261680,49431,194042,206704.0,227464,2161-01-13 23:17:00,2161-01-13 23:20:00,20889.0,3.8,3.8,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
9261681,49304,154856,246427.0,227464,2148-09-12 12:22:00,2148-09-12 12:26:00,20889.0,4.2,4.2,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs
9261682,48939,139380,270705.0,227464,2187-08-14 08:56:00,2187-08-14 08:58:00,20889.0,3.5,3.5,mEq/L,0,0,NaN,NaN,Potassium (whole blood),Labs


In [62]:
charts[charts.LABEL.isin(['Sodium (serum)', 'Sodium', 'Sodium  (135-148)', 'Sodium (135-148)', 'Sodium (whole blood)'])]
charts['LABEL'].mask(charts['LABEL']== 'Sodium', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium  (135-148)', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium (135-148)', 'Sodium (serum)', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Sodium (whole blood)', 'Sodium (serum)', inplace=True)
charts[charts.LABEL.isin(['Sodium (serum)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
8335898,23,124321,234044.0,220645,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,144.0,144.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs
8335899,23,124321,234044.0,220645,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,144.0,144.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs
8335900,36,165660,241249.0,220645,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,139.0,139.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs
8335901,34,144319,290505.0,220645,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,139.0,139.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs
8335902,34,144319,290505.0,220645,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,142.0,142.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9356145,49311,136786,272939.0,226534,2121-06-16 12:56:00,2121-06-16 12:59:00,20889.0,129.0,129.0,mEq/L,1,0,NaN,NaN,Sodium (serum),Labs
9356146,49471,184788,227744.0,226534,2143-05-10 10:58:00,2143-05-10 11:01:00,20889.0,132.0,132.0,mEq/L,1,0,NaN,NaN,Sodium (serum),Labs
9356147,49370,111547,249824.0,226534,2137-12-03 05:26:00,2137-12-03 05:29:00,20889.0,132.0,132.0,mEq/L,1,0,NaN,NaN,Sodium (serum),Labs
9356148,49315,122228,217459.0,226534,2200-09-17 13:59:00,2200-09-17 14:01:00,20889.0,137.0,137.0,mEq/L,0,0,NaN,NaN,Sodium (serum),Labs


In [63]:
charts[charts.LABEL.isin([ 'WBC   (4-11,000)', 'WBC (4-11,000)', 'WBC 4.0-11.0', 'WBC'])]
charts['LABEL'].mask(charts['LABEL']== 'WBC   (4-11,000)', 'WBC', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'WBC (4-11,000)', 'WBC', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'WBC 4.0-11.0', 'WBC', inplace=True)
charts[charts.LABEL.isin(['WBC'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
8075170,23,124321,234044.0,220546,2157-10-21 14:40:00,2157-10-21 15:06:00,20889.0,14.8,14.8,K/uL,1,0,NaN,NaN,WBC,Labs
8075171,23,124321,234044.0,220546,2157-10-22 03:21:00,2157-10-22 04:02:00,20889.0,16.4,16.4,K/uL,1,0,NaN,NaN,WBC,Labs
8075172,36,165660,241249.0,220546,2134-05-12 08:33:00,2134-05-12 09:36:00,20889.0,15.3,15.3,K/uL,1,0,NaN,NaN,WBC,Labs
8075173,34,144319,290505.0,220546,2191-02-23 10:48:00,2191-02-23 11:12:00,20889.0,7.1,7.1,K/uL,0,0,NaN,NaN,WBC,Labs
8075174,34,144319,290505.0,220546,2191-02-24 04:32:00,2191-02-24 05:07:00,20889.0,7.9,7.9,K/uL,0,0,NaN,NaN,WBC,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8121397,48910,143290,246813.0,220546,2186-10-14 06:04:00,2186-10-14 06:28:00,20889.0,8.8,8.8,K/uL,0,0,NaN,NaN,WBC,Labs
8121398,49379,129372,201098.0,220546,2124-12-23 13:28:00,2124-12-23 14:13:00,20889.0,17.6,17.6,K/uL,1,0,NaN,NaN,WBC,Labs
8121399,48935,177563,276120.0,220546,2133-10-24 03:52:00,2133-10-24 04:50:00,20889.0,4.1,4.1,K/uL,0,0,NaN,NaN,WBC,Labs
8121400,48925,127507,260855.0,220546,2129-01-30 02:41:00,2129-01-30 03:26:00,20889.0,10.9,10.9,K/uL,0,0,NaN,NaN,WBC,Labs


In [64]:
charts[charts.LABEL.isin(['BUN', 'BUN    (6-20)', 'BUN (6-20)'])]
charts['LABEL'].mask(charts['LABEL']== 'BUN    (6-20)', 'BUN', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'BUN (6-20)', 'BUN', inplace=True)
charts[charts.LABEL.isin(['BUN'])]
     

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
8391867,23,124321,234044.0,225624,2157-10-21 14:40:00,2157-10-21 15:53:00,20889.0,14.0,14.0,mg/dL,0,0,NaN,NaN,BUN,Labs
8391868,23,124321,234044.0,225624,2157-10-22 03:21:00,2157-10-22 04:12:00,20889.0,14.0,14.0,mg/dL,0,0,NaN,NaN,BUN,Labs
8391869,36,165660,241249.0,225624,2134-05-12 08:33:00,2134-05-12 10:01:00,20889.0,19.0,19.0,mg/dL,0,0,NaN,NaN,BUN,Labs
8391870,34,144319,290505.0,225624,2191-02-23 10:48:00,2191-02-23 12:23:00,20889.0,36.0,36.0,mg/dL,1,0,NaN,NaN,BUN,Labs
8391871,34,144319,290505.0,225624,2191-02-24 04:32:00,2191-02-24 05:41:00,20889.0,30.0,30.0,mg/dL,1,0,NaN,NaN,BUN,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445234,49379,129372,201098.0,225624,2124-12-23 13:28:00,2124-12-23 15:10:00,20889.0,34.0,34.0,mg/dL,1,0,NaN,NaN,BUN,Labs
8445235,48935,177563,276120.0,225624,2133-10-24 03:52:00,2133-10-24 05:20:00,20889.0,21.0,21.0,mg/dL,1,0,NaN,NaN,BUN,Labs
8445236,48939,139380,270705.0,225624,2187-08-19 14:51:00,2187-08-19 15:32:00,20889.0,110.0,110.0,mg/dL,1,0,NaN,NaN,BUN,Labs
8445237,48939,139380,270705.0,225624,2187-08-24 03:50:00,2187-08-24 04:34:00,20889.0,58.0,58.0,mg/dL,1,0,NaN,NaN,BUN,Labs


In [65]:
charts.loc[charts.LABEL== 'Admission Weight (lbs.)', 'VALUENUM'] = ((charts.VALUENUM) * 0.45359237)
charts[charts.LABEL.isin(['Admission Weight (lbs.)','Admission Weight (Kg)'])]
charts['LABEL'].mask(charts['LABEL']== 'Admission Weight (lbs.)','Admission Weight (Kg)', inplace=True)

charts[charts.LABEL.isin(['Admission Weight (Kg)'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
6223867,34,144319,290505.0,226512,2191-02-23 07:44:00,2191-02-23 07:44:00,17741.0,74.5,74.500000,kg,0,0,NaN,NaN,Admission Weight (Kg),General
6223868,23,124321,234044.0,226512,2157-10-21 12:15:00,2157-10-21 12:15:00,16978.0,66.8,66.800000,kg,0,0,NaN,NaN,Admission Weight (Kg),General
6223869,36,165660,241249.0,226512,2134-05-16 10:58:00,2134-05-16 10:58:00,16223.0,106.2,106.200000,kg,0,0,NaN,NaN,Admission Weight (Kg),General
6223870,85,112077,291697.0,226512,2167-07-25 21:31:00,2167-07-25 21:31:00,21050.0,98.0,98.000000,kg,0,0,NaN,NaN,Admission Weight (Kg),General
6223871,107,182383,252542.0,226512,2121-12-01 05:54:00,2121-12-01 05:54:00,16526.0,88.6,88.600000,kg,0,0,NaN,NaN,Admission Weight (Kg),General
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6243367,49380,110837,241303.0,226531,2105-04-06 12:50:00,2105-04-06 12:50:00,15667.0,256.7,116.437161,NaN,0,0,NaN,NaN,Admission Weight (Kg),General
6243368,49380,110837,241303.0,226531,2105-04-06 12:52:00,2105-04-06 12:52:00,15667.0,256.7,116.437161,NaN,0,0,NaN,NaN,Admission Weight (Kg),General
6243369,49395,169700,278148.0,226531,2197-01-09 02:34:00,2197-01-09 02:34:00,16037.0,198.7,90.128804,NaN,0,0,NaN,NaN,Admission Weight (Kg),General
6243370,49453,161038,287731.0,226531,2105-01-30 19:49:00,2105-01-30 19:49:00,19937.0,374.0,169.643546,NaN,0,0,NaN,NaN,Admission Weight (Kg),General


In [66]:


charts[charts.LABEL.isin(['Respiratory Rate (Set)',  'Respiratory Rate',
 'Respiratory Rate (Total)', 'Respiratory Rate (spontaneous)'])]
charts['LABEL'].mask(charts['LABEL']==  'Respiratory Rate (Set)', 'Respiratory Rate',   inplace=True)
charts['LABEL'].mask(charts['LABEL']==   'Respiratory Rate (Total)', 'Respiratory Rate', inplace=True)
charts['LABEL'].mask(charts['LABEL']==    'Respiratory Rate (spontaneous)',  'Respiratory Rate', inplace=True)
charts[charts.LABEL.isin(['Respiratory Rate'])]
    

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
2421807,36,165660,241249.0,220210,2134-05-12 13:00:00,2134-05-12 13:55:00,17525.0,21.0,21.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
2421808,36,165660,241249.0,220210,2134-05-12 14:00:00,2134-05-12 14:39:00,17525.0,19.0,19.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
2421809,36,165660,241249.0,220210,2134-05-12 15:00:00,2134-05-12 16:51:00,17525.0,20.0,20.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
2421810,36,165660,241249.0,220210,2134-05-12 16:00:00,2134-05-12 16:51:00,17525.0,20.0,20.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
2421811,34,144319,290505.0,220210,2191-02-23 07:33:00,2191-02-23 07:45:00,17741.0,26.0,26.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5732971,48910,143290,246813.0,224689,2186-10-14 10:48:00,NaN,NaN,0.0,0.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
5732972,49407,183964,291495.0,224689,2119-04-12 21:00:00,2119-04-12 20:44:00,21322.0,0.0,0.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
5732973,48910,143290,246813.0,224689,2186-10-14 03:03:00,NaN,NaN,16.0,16.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory
5732974,48910,143290,246813.0,224689,2186-10-14 03:04:00,NaN,NaN,17.0,17.0,insp/min,0,0,NaN,NaN,Respiratory Rate,Respiratory


In [67]:
charts[charts.LABEL.isin(['Arterial O2 pressure', 'Arterial PaO2', 'Inspired O2 Fraction', 'FIO2', 'FIO2 Alarm [High]',
                          'FIO2 Alarm [Low]', 'FIO2 Alarm-High', 'FIO2 [Meas]'])]

charts['LABEL'].mask(charts['LABEL']== 'Arterial O2 pressure', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Arterial PaO2', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm [High]', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm [Low]', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 Alarm-High', 'Inspired O2 Fraction', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'FIO2 [Meas]', 'Inspired O2 Fraction', inplace=True)
charts[charts.LABEL.isin(['Inspired O2 Fraction'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
56174,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.0,100.0,NaN,0,0,NaN,NaN,Inspired O2 Fraction,Respiratory
56175,34,144319,290505.0,223835,2191-02-23 07:31:00,2191-02-23 07:35:00,16924.0,60.0,60.0,NaN,0,0,NaN,NaN,Inspired O2 Fraction,Respiratory
56176,36,165660,241249.0,223835,2134-05-12 07:09:00,2134-05-12 07:09:00,17525.0,100.0,100.0,NaN,0,0,NaN,NaN,Inspired O2 Fraction,Respiratory
56177,34,144319,290505.0,223835,2191-02-23 11:00:00,2191-02-23 11:04:00,14913.0,60.0,60.0,NaN,0,0,NaN,NaN,Inspired O2 Fraction,Respiratory
56178,36,165660,241249.0,223835,2134-05-13 16:00:00,2134-05-13 16:27:00,20290.0,50.0,50.0,NaN,0,0,NaN,NaN,Inspired O2 Fraction,Respiratory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313090,48925,127507,260855.0,220224,2129-02-09 06:11:00,2129-02-09 06:12:00,20889.0,75.0,75.0,mmHg,1,0,NaN,NaN,Inspired O2 Fraction,Labs
313091,48925,127507,260855.0,220224,2129-01-22 10:39:00,2129-01-22 10:41:00,20889.0,50.0,50.0,mmHg,1,0,NaN,NaN,Inspired O2 Fraction,Labs
313092,48939,139380,270705.0,220224,2187-08-13 08:12:00,2187-08-13 08:14:00,20889.0,104.0,104.0,mmHg,0,0,NaN,NaN,Inspired O2 Fraction,Labs
313093,48939,139380,270705.0,220224,2187-08-14 20:18:00,2187-08-14 20:19:00,20889.0,103.0,103.0,mmHg,0,0,NaN,NaN,Inspired O2 Fraction,Labs


In [68]:
charts[charts.LABEL.isin(['Total Bilirubin', 'Total Bili', 'Direct Bili (0-0.3)',])]
charts['LABEL'].mask(charts['LABEL']== 'Total Bili', 'Total Bilirubin', inplace=True)
charts['LABEL'].mask(charts['LABEL']== 'Direct Bili (0-0.3)', 'Total Bilirubin', inplace=True)
charts[charts.LABEL.isin(['Total Bilirubin'])]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,LABEL,CATEGORY
9230954,36,165660,241249.0,225690,2134-05-14 03:20:00,2134-05-14 04:08:00,20889.0,0.6,0.6,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9230955,107,174162,264253.0,225690,2122-05-14 21:48:00,2122-05-14 22:37:00,20889.0,0.1,0.1,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9230956,109,170149,266497.0,225690,2141-05-24 20:01:00,2141-05-25 01:17:00,20889.0,0.3,0.3,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9230957,109,147469,253139.0,225690,2141-06-11 13:07:00,2141-06-11 18:50:00,20889.0,0.5,0.5,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9230958,109,147469,253139.0,225690,2141-06-12 14:50:00,2141-06-12 17:47:00,20889.0,0.4,0.4,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9243480,49281,168911,228407.0,225690,2110-03-03 03:15:00,2110-03-03 04:23:00,20889.0,0.3,0.3,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9243481,49370,111547,249824.0,225690,2137-12-04 08:17:00,2137-12-04 10:17:00,20889.0,0.8,0.8,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9243482,48925,127507,260855.0,225690,2129-02-10 01:15:00,2129-02-10 14:59:00,20889.0,0.5,0.5,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs
9243483,49379,129372,201098.0,225690,2124-12-23 13:28:00,2124-12-23 15:10:00,20889.0,1.0,1.0,mg/dL,0,0,NaN,NaN,Total Bilirubin,Labs


In [69]:
charts['LABEL'] = charts['LABEL'].map({
     
'ABI Ankle BP L':'ABI Ankle BP L', 
'ABI Ankle BP L (Impella)':'ABI Ankle BP L', 
'ABI Ankle BP R':'ABI Ankle BP L',
'Impaired Skin Width #1':'Impaired Skin Width #1', 'Impaired Skin Width #10':'Impaired Skin Width #1',
'Impaired Skin Width #2':'Impaired Skin Width #1', 'Impaired Skin Width #3':'Impaired Skin Width #1',
'Impaired Skin Width #4':'Impaired Skin Width #1', 'Impaired Skin Width #5':'Impaired Skin Width #1',
'Impaired Skin Width #6':'Impaired Skin Width #1', 'Impaired Skin Width #7':'Impaired Skin Width #1',
'Impaired Skin Width #8':'Impaired Skin Width #1', 'Impaired Skin Width #9':'Impaired Skin Width #1',                                     
'Impaired Skin Length #1':'Impaired Skin Length #1', 'Impaired Skin Length #10':'Impaired Skin Length #1',
'Impaired Skin Length #2':'Impaired Skin Length #1', 'Impaired Skin Length #3':'Impaired Skin Length #1',
'Impaired Skin Length #4':'Impaired Skin Length #1', 'Impaired Skin Length #5':'Impaired Skin Length #1',
'Impaired Skin Length #6':'Impaired Skin Length #1', 'Impaired Skin Length #7':'Impaired Skin Length #1',
'Impaired Skin Length #8':'Impaired Skin Length #1', 'Impaired Skin Length #9':'Impaired Skin Length #1',                                    
'Impaired Skin Depth #1':'Impaired Skin Depth #1', 'Impaired Skin Depth #10':'Impaired Skin Depth #1',
'Impaired Skin Depth #2':'Impaired Skin Depth #1', 'Impaired Skin Depth #3':'Impaired Skin Depth #1',
'Impaired Skin Depth #4':'Impaired Skin Depth #1', 'Impaired Skin Depth #5':'Impaired Skin Depth #1',
'Impaired Skin Depth #6':'Impaired Skin Depth #1', 'Impaired Skin Depth #7':'Impaired Skin Depth #1',
'Impaired Skin Depth #8':'Impaired Skin Depth #1', 'Impaired Skin Depth #9':'Impaired Skin Depth #1',
'Skeletal Traction #1 - Pounds':'Skeletal Traction #1 - Pounds', 'Skeletal Traction #2 - Pounds':'Skeletal Traction #1 - Pounds',                          
'ABI Brachial BP L':'ABI Brachial BP L', 'ABI Brachial BP R':'ABI Brachial BP L',
'ABI Brachial BP R (Impella)':'ABI Brachial BP L',
'ART BP Diastolic': 'ART BP mean',
'ART BP mean': 'ART BP mean',
'ART Blood Pressure Alarm - High': 'ART BP mean','ART Blood Pressure Alarm - Low': 'ART BP mean',                                       
'APACHE II': 'APACHE II',
'APACHE II PDR - Adjusted': 'APACHE II',
'APACHE II Predecited Death Rate': 'APACHE II',
'APS': 'APS',
'ALT': 'ALT',
'ART Lumen Volume': 'ART Lumen Volume',
'AST': 'AST',
'Albumin': 'Albumin',
'AlbuminScore_ApacheIV': 'AlbuminScore_ApacheIV',
'Albumin_ApacheIV': 'Albumin_ApacheIV',
'Alkaline Phosphate': 'Alkaline Phosphate',
'Anion gap': 'Anion gap',
'Aortic Pressure Signal - Diastolic': 'Aortic Pressure Signal - Diastolic',
'Aortic Pressure Signal - Systolic': 'Aortic Pressure Signal - Systolic',
'Arterial Blood Pressure mean':'Arterial Blood Pressure mean',
'Arterial Blood Pressure Alarm - High':'Arterial Blood Pressure mean', 
'Arterial Blood Pressure Alarm - Low':'Arterial Blood Pressure mean',
'Arterial Blood Pressure diastolic':'Arterial Blood Pressure mean',
'Arterial Base Excess': 'Arterial Base Excess',
'Arterial CO2 Pressure': 'Arterial CO2 Pressure',
'Arterial O2 Saturation': 'Arterial O2 Saturation',
'Assisted Systole': 'Assisted Systole',
'Augmented Diastole': 'Augmented Diastole',
'Ventilator Tank #2': 'Ventilator Tank #1',
'Ventilator Tank #1':'Ventilator Tank #1',
'Dialysate Rate': 'Dialysate Rate',
'C Reactive Protein (CRP)': 'C Reactive Protein (CRP)',
'CFI (PiCCO)': 'CFI (PiCCO)',
'CI (PiCCO)': 'CI (PiCCO)',
'CK (CPK)': 'CK (CPK)',
'CK-MB': 'CK-MB',
'CO (Arterial)': 'CO (Arterial)',
'CO (PiCCO)': 'CO (PiCCO)',
'CO2 production': 'CO2 production',
'Brain Natiuretic Peptide (BNP)': 'Brain Natiuretic Peptide (BNP)',
'Ammonia': 'Ammonia',
'Admission Weight (Kg)': 'Admission Weight (Kg)',
'Admission Weight (lbs.)': 'Admission Weight (Kg)', 
'Calcium non-ionized': 'Calcium non-ionized',
'MDI #1 Puff': 'MDI #1 Puff',
'MDI #2 Puff': 'MDI #1 Puff',
'MDI #3 Puff': 'MDI #1 Puff',
'Venous CO2 Pressure': 'Venous CO2 Pressure',
'Venous O2 Pressure': 'Venous O2 Pressure',
'LDL measured': 'LDL measured',
'LDL calculated': 'LDL measured',
'Intra Cranial Pressure': 'Intra Cranial Pressure',
'Intra Cranial Pressure #2': 'Intra Cranial Pressure',
'Intra Cranial Pressure Alarm - High': 'Intra Cranial Pressure',
'Intra Cranial Pressure Alarm - Low': 'Intra Cranial Pressure',                                  
'Non Invasive Blood Pressure diastolic': 'Non Invasive Blood Pressure mean',
'Non Invasive Blood Pressure mean': 'Non Invasive Blood Pressure mean',
'Non-Invasive Blood Pressure Alarm - High': 'Non-Invasive Blood Pressure mean',
'Non-Invasive Blood Pressure Alarm - Low': 'Non-Invasive Blood Pressure mean',
'Ionized Calcium': 'Ionized Calcium',
'LDH': 'LDH',
'LOS pre-ICU admission': 'LOS pre-ICU admission',
'Lactic Acid': 'Lactic Acid',
'Lipase': 'Lipase',
'Magnesium': 'Magnesium', 
'Fibrinogen': 'Fibrinogen',
'Filter Pressure': 'Filter Pressure',
'Pulmonary Artery Pressure Alarm - High': 'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure Alarm - Low': 'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure diastolic': 'Pulmonary Artery Pressure mean',
'Pulmonary Artery Pressure mean': 'Pulmonary Artery Pressure mean',       
'O2 Saturation Pulseoxymetry Alarm - High': 'O2 Saturation Pulseoxymetry pulseoxymetry',
'O2 Saturation Pulseoxymetry Alarm - Low': 'O2 Saturation Pulseoxymetry pulseoxymetry',
'O2 saturation pulseoxymetry': 'O2 saturation pulseoxymetry',                                 
'Gentamicin (Peak)': 'Gentamicin (Random)',
'Gentamicin (Random)': 'Gentamicin (Random)',
'Gentamicin (Trough)': 'Gentamicin (Random)',
'Temperature Fahrenheit': 'Temperature Celsius',
'Temperature F': 'Temperature Celsius', 
'Temperature F (calc)': 'Temperature Celsius', 
'Blood Temperature CCO (C)': 'Temperature Celsius', 
'Skin [Temperature]': 'Temperature Celsius', 
'Temperature C': 'Temperature Celsius', 
'Temperature C (calc)': 'Temperature Celsius', 
'Temp Axillary [F]': 'Temperature Celsius', 
'Temp Rectal [F]': 'Temperature Celsius',
'Temperature Celsius': 'Temperature Celsius',
'NBP [Systolic]': 'ART BP Systolic', 
'Arterial BP [Systolic]' :'ART BP Systolic', 
'BP Cuff [Systolic]': 'ART BP Systolic', 
'Manual BP [Systolic]': 'ART BP Systolic', 
'BP UAC [Systolic]': 'ART BP Systolic',
'BP Right Leg [Systolic]': 'ART BP Systolic', 
'Pulmonary Artery Pressure systolic': 'ART BP Systolic', 
'Non Invasive Blood Pressure systolic': 'ART BP Systolic',
'Arterial Blood Pressure systolic': 'ART BP Systolic',                                 
'ART BP Systolic': 'ART BP Systolic',
'Heart Rate Alarm - Low': 'Heart Rate',
'Heart rate Alarm - High': 'Heart Rate',
'Heart Rate': 'Heart Rate',
'Potassium  (3.5-5.3)': 'Potassium (whole blood)', 
'Potassium (3.5-5.3)': 'Potassium (whole blood)', 
'Potassium (serum)': 'Potassium (whole blood)',
'Potassium (whole blood)': 'Potassium (whole blood)',   
'Sodium': 'Sodium (serum)', 
'Sodium  (135-148)': 'Sodium (serum)', 
'Sodium (135-148)': 'Sodium (serum)', 
'Sodium (whole blood)': 'Sodium (serum)',
'Sodium (serum)': 'Sodium (serum)',
'WBC   (4-11,000)': 'WBC',
'WBC (4-11,000)': 'WBC', 
'WBC 4.0-11.0': 'WBC',                                      
'WBC': 'WBC',   
'BUN    (6-20)': 'BUN',
'BUN (6-20)': 'BUN',
'BUN': 'BUN',
'Arterial O2 pressure': 'Inspired O2 Fraction', 
'Arterial PaO2': 'Inspired O2 Fraction', 
'FIO2': 'Inspired O2 Fraction',
'FIO2 Alarm [High]': 'Inspired O2 Fraction', 
'FIO2 Alarm [Low]': 'Inspired O2 Fraction', 
'FIO2 Alarm-High': 'Inspired O2 Fraction', 
'FIO2 [Meas]': 'Inspired O2 Fraction',                                   
'Inspired O2 Fraction': 'Inspired O2 Fraction',                                     
'Total Bili': 'Total Bilirubin', 
'Direct Bili (0-0.3)': 'Total Bilirubin',
'Total Bilirubin': 'Total Bilirubin',                                    
'GCS - Eye Opening':'GCS - Eye Opening', 
'GCS - Motor Response':'GCS - Motor Response', 
'GCS - Verbal Response': 'GCS - Verbal Response',    
'GCS Total':'GCS Total', 
'HCO3 (serum)': 'HCO3 (serum)',
'Differential-Atyps': 'Differential-Atyps',
'Differential-Bands': 'Differential-Bands',
'Differential-Basos': 'Differential-Basos',
'Differential-Eos': 'Differential-Eos',
'Differential-Lymphs': 'Differential-Lymphs',
'Differential-Monos': 'Differential-Monos',
'Differential-Neuts': 'Differential-Neuts',
'BunScore_ApacheIV': 'BunScore_ApacheIV',
'Respiratory Rate': 'Respiratory Rate',
'Respiratory Rate (Set)':  'Respiratory Rate',
'Respiratory Rate (Total)':  'Respiratory Rate',
'Respiratory Rate (spontaneous)':  'Respiratory Rate',
})

In [70]:
charts=charts.pivot_table(values='VALUENUM', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME'], columns='LABEL')

In [71]:
charts

LABEL                                                     ABI Ankle BP L  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID CHARTTIME                             
23         124321  234044.0   220045 2157-10-21 12:06:00             NaN   
                                     2157-10-21 12:08:00             NaN   
                                     2157-10-21 13:00:00             NaN   
                                     2157-10-21 14:00:00             NaN   
                                     2157-10-21 15:00:00             NaN   
...                                                                  ...   
49496      112342  269261.0   224154 2124-03-12 18:00:00             NaN   
                                     2124-03-12 19:00:00             NaN   
                                     2124-03-12 20:00:00             NaN   
                                     2124-03-09 20:37:00             NaN   
                                     2124-03-12 10:30:00             NaN   

LABEL                                                     ABI Brachial BP L  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID CHARTTIME                                
23         124321  234044.0   220045 2157-10-21 12:06:00                NaN   
                                     2157-10-21 12:08:00                NaN   
                                     2157-10-21 13:00:00                NaN   
                                     2157-10-21 14:00:00                NaN   
                                     2157-10-21 15:00:00                NaN   
...                                                                     ...   
49496      112342  269261.0   224154 2124-03-12 18:00:00                NaN   
                                     2124-03-12 19:00:00                NaN   
                                     2124-03-12 20:00:00                NaN   
                                     2124-03-09 20:37:00                NaN   
                                     2124-03-12 10:30:00                NaN   

LABEL                                                     ALT  APACHE II  APS  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID CHARTTIME                                  
23         124321  234044.0   220045 2157-10-21 12:06:00  NaN        NaN  NaN   
                                     2157-10-21 12:08:00  NaN        NaN  NaN   
                                     2157-10-21 13:00:00  NaN        NaN  NaN   
                                     2157-10-21 14:00:00  NaN        NaN  NaN   
                                     2157-10-21 15:00:00  NaN        NaN  NaN   
...                                                       ...        ...  ...   
49496      112342  269261.0   224154 2124-03-12 18:00:00  NaN        NaN  NaN   
                                     2124-03-12 19:00:00  NaN        NaN  NaN   
                                     2124-03-12 20:00:00  NaN        NaN  NaN   
                                     2124-03-09 20:37:00  NaN        NaN  NaN   
                                     2124-03-12 10:30:00  NaN        NaN  NaN   

LABEL                                                     ART BP Systolic  \
SUBJECT_ID HADM_ID ICUSTAY_ID ITEMID CHARTTIME                              
23         124321  234044.0   220045 2157-10-21 12:06:00              NaN   
                                     2157-10-21 12:08:00              NaN   
                                     2157-10-21 13:00:00              NaN   
                                     2157-10-21 14:00:00              NaN   
                                     2157-10-21 15:00:00              NaN   
...                                                                   ...   
49496      112342  269261.0   224154 2124-03-12 18:00:00              NaN   
                                     2124-03-12 19:00:00              NaN   
                                     2124-03-12 20:00:00              NaN   
                                     2124-03-09 20:37:00              NaN   
     

In [72]:
charts.columns

Index(['ABI Ankle BP L', 'ABI Brachial BP L', 'ALT', 'APACHE II', 'APS',
       'ART BP Systolic', 'ART BP mean', 'ART Lumen Volume', 'AST',
       'Admission Weight (Kg)', 'Albumin', 'AlbuminScore_ApacheIV',
       'Albumin_ApacheIV', 'Alkaline Phosphate', 'Ammonia', 'Anion gap',
       'Aortic Pressure Signal - Diastolic',
       'Aortic Pressure Signal - Systolic', 'Arterial Base Excess',
       'Arterial Blood Pressure mean', 'Arterial CO2 Pressure',
       'Arterial O2 Saturation', 'Assisted Systole', 'Augmented Diastole',
       'BUN', 'Brain Natiuretic Peptide (BNP)', 'BunScore_ApacheIV',
       'C Reactive Protein (CRP)', 'CFI (PiCCO)', 'CI (PiCCO)', 'CK (CPK)',
       'CK-MB', 'CO (Arterial)', 'CO (PiCCO)', 'CO2 production',
       'Calcium non-ionized', 'Dialysate Rate', 'Differential-Atyps',
       'Differential-Bands', 'Differential-Basos', 'Differential-Eos',
       'Differential-Lymphs', 'Differential-Monos', 'Differential-Neuts',
       'Fibrinogen', 'Filter Pressure', '

In [73]:
cols=['ABI Ankle BP L', 'ABI Brachial BP L', 'ALT', 'APACHE II', 'APS',
       'ART BP Systolic', 'ART BP mean', 'ART Lumen Volume', 'AST',
       'Admission Weight (Kg)', 'Albumin', 'AlbuminScore_ApacheIV',
       'Albumin_ApacheIV', 'Alkaline Phosphate', 'Ammonia', 'Anion gap',
       'Aortic Pressure Signal - Diastolic',
       'Aortic Pressure Signal - Systolic', 'Arterial Base Excess',
       'Arterial Blood Pressure mean', 'Arterial CO2 Pressure',
       'Arterial O2 Saturation', 'Assisted Systole', 'Augmented Diastole',
       'BUN', 'Brain Natiuretic Peptide (BNP)', 'BunScore_ApacheIV',
       'C Reactive Protein (CRP)', 'CFI (PiCCO)', 'CI (PiCCO)', 'CK (CPK)',
       'CK-MB', 'CO (Arterial)', 'CO (PiCCO)', 'CO2 production',
       'Calcium non-ionized', 'Dialysate Rate', 'Differential-Atyps',
       'Differential-Bands', 'Differential-Basos', 'Differential-Eos',
       'Differential-Lymphs', 'Differential-Monos', 'Differential-Neuts',
       'Fibrinogen', 'Filter Pressure', 'Gentamicin (Random)', 'HCO3 (serum)',
       'Heart Rate', 'Impaired Skin Depth #1', 'Impaired Skin Length #1',
       'Impaired Skin Width #1', 'Inspired O2 Fraction',
       'Intra Cranial Pressure', 'Ionized Calcium', 'LDH', 'LDL measured',
       'LOS pre-ICU admission', 'Lactic Acid', 'Lipase', 'MDI #1 Puff',
       'Magnesium', 'Non Invasive Blood Pressure mean',
       'Non-Invasive Blood Pressure mean', 'O2 saturation pulseoxymetry',
       'Potassium (whole blood)', 'Pulmonary Artery Pressure mean',
       'Respiratory Rate', 'Skeletal Traction #1 - Pounds', 'Sodium (serum)',
       'Temperature Celsius', 'Total Bilirubin', 'Venous CO2 Pressure',
       'Venous O2 Pressure', 'Ventilator Tank #1', 'WBC']

In [74]:
charts.to_csv(output_files+'charts.csv')

In [75]:
charts.columns

Index(['ABI Ankle BP L', 'ABI Brachial BP L', 'ALT', 'APACHE II', 'APS',
       'ART BP Systolic', 'ART BP mean', 'ART Lumen Volume', 'AST',
       'Admission Weight (Kg)', 'Albumin', 'AlbuminScore_ApacheIV',
       'Albumin_ApacheIV', 'Alkaline Phosphate', 'Ammonia', 'Anion gap',
       'Aortic Pressure Signal - Diastolic',
       'Aortic Pressure Signal - Systolic', 'Arterial Base Excess',
       'Arterial Blood Pressure mean', 'Arterial CO2 Pressure',
       'Arterial O2 Saturation', 'Assisted Systole', 'Augmented Diastole',
       'BUN', 'Brain Natiuretic Peptide (BNP)', 'BunScore_ApacheIV',
       'C Reactive Protein (CRP)', 'CFI (PiCCO)', 'CI (PiCCO)', 'CK (CPK)',
       'CK-MB', 'CO (Arterial)', 'CO (PiCCO)', 'CO2 production',
       'Calcium non-ionized', 'Dialysate Rate', 'Differential-Atyps',
       'Differential-Bands', 'Differential-Basos', 'Differential-Eos',
       'Differential-Lymphs', 'Differential-Monos', 'Differential-Neuts',
       'Fibrinogen', 'Filter Pressure', '

In [76]:
for col in cols:
    charts[col] = charts.groupby('HADM_ID')[col].transform('mean')

In [77]:
charts.drop_duplicates(inplace=True)

In [78]:
charts

,,,,LABEL,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,ART Lumen Volume,AST,Admission Weight (Kg),...,Pulmonary Artery Pressure mean,Respiratory Rate,Skeletal Traction #1 - Pounds,Sodium (serum),Temperature Celsius,Total Bilirubin,Venous CO2 Pressure,Venous O2 Pressure,Ventilator Tank #1,WBC
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,,,,,,,,,,,,,,,,,,,,,
23,124321,234044.0,220045,2157-10-21 12:06:00,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,NaN,NaN,66.739039,...,NaN,17.114286,NaN,144.000000,36.301587,NaN,NaN,NaN,NaN,15.600000
34,144319,290505.0,220045,2191-02-23 05:25:00,NaN,NaN,NaN,NaN,NaN,128.055556,NaN,NaN,NaN,74.382842,...,NaN,16.731707,NaN,140.500000,36.319444,NaN,NaN,NaN,NaN,7.500000
36,165660,241249.0,220045,2134-05-12 06:52:00,NaN,NaN,12.000000,NaN,NaN,127.950000,NaN,NaN,18.00,106.079589,...,NaN,17.906475,NaN,141.500000,37.264444,0.600000,NaN,NaN,2471.428571,11.760000
85,112077,291697.0,220045,2167-07-25 21:12:00,NaN,NaN,23.000000,NaN,NaN,111.125000,NaN,NaN,18.00,97.897257,...,NaN,28.340426,NaN,137.000000,37.156566,NaN,NaN,NaN,NaN,8.650000
107,174162,264253.0,220045,2122-05-14 21:07:00,NaN,NaN,22.000000,NaN,NaN,104.520000,NaN,NaN,21.00,83.882580,...,NaN,15.051724,NaN,137.666667,36.902778,0.100000,NaN,NaN,NaN,11.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49471,184788,227744.0,220045,2143-05-10 12:09:00,NaN,NaN,NaN,NaN,NaN,108.195122,NaN,NaN,NaN,75.920322,...,32.160714,20.620690,NaN,134.600000,36.141577,NaN,NaN,NaN,2000.000000,11.466667
49480,122348,228590.0,220045,2162-07-29 07:00:00,NaN,NaN,26.500000,NaN,NaN,136.814286,NaN,NaN,24.25,74.517254,...,NaN,19.081761,NaN,136.181818,36.439940,0.600000,NaN,NaN,NaN,6.620000
49482,191442,209686.0,220045,2155-07-08 13:00:00,NaN,NaN,NaN,NaN,NaN,100.800000,NaN,NaN,NaN,73.018827,...,NaN,15.000000,NaN,NaN,36.194444,NaN,NaN,NaN,NaN,NaN


In [79]:
charts.to_csv(output_files+'charts_record_per_admission.csv')

# OUTPUTEVENTS File

In [80]:
outputevents=pd.read_csv(mimic_path+"OUTPUTEVENTS.csv.gz")

In [81]:
outputevents

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,4340476,68375,123645.0,276251.0,2175-09-05 23:00:00,226588,30.0,mL,2175-09-05 22:48:00,16915,NaN,NaN,NaN
4349214,4340477,68375,123645.0,276251.0,2175-09-06 00:00:00,226588,0.0,mL,2175-09-06 00:03:00,20971,NaN,NaN,NaN
4349215,4340478,68375,123645.0,276251.0,2175-09-06 01:00:00,226588,40.0,mL,2175-09-06 01:13:00,20971,NaN,NaN,NaN
4349216,4340479,68375,123645.0,276251.0,2175-09-06 02:00:00,226588,20.0,mL,2175-09-06 02:14:00,20971,NaN,NaN,NaN


In [82]:
outputevents=item_merge(outputevents, d_c)
outputevents

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,3928870,42590,199643.0,291593.0,2112-08-18 15:00:00,226621,0.0,mL,2112-08-18 15:56:00,14542,NaN,NaN,NaN,Davol,Drains
4349214,3928871,42590,199643.0,291593.0,2112-08-18 20:00:00,226621,20.0,mL,2112-08-18 21:38:00,14576,NaN,NaN,NaN,Davol,Drains
4349215,3928872,42590,199643.0,291593.0,2112-08-19 00:00:00,226621,0.0,mL,2112-08-19 00:10:00,14576,NaN,NaN,NaN,Davol,Drains
4349216,3928873,42590,199643.0,291593.0,2112-08-19 04:00:00,226621,10.0,mL,2112-08-19 04:19:00,14576,NaN,NaN,NaN,Davol,Drains


In [83]:
outputevents.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID',
       'VALUE', 'VALUEUOM', 'STORETIME', 'CGID', 'STOPPED', 'NEWBOTTLE',
       'ISERROR', 'LABEL', 'CATEGORY'],
      dtype='object')

In [84]:
labels=outputevents['LABEL'].tolist()

In [85]:
unique_labels=np.unique(labels)

In [86]:
len(unique_labels)

1153

In [87]:
outputevents['LABEL'] = outputevents['LABEL'].astype('category')
outputevents['LABEL'].cat.set_categories(unique_labels,inplace=True)

In [88]:
outputevents=outputevents.dropna(how='any', subset=['VALUE']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [89]:
outputevents

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN,Urine Out Foley,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349213,3928870,42590,199643.0,291593.0,2112-08-18 15:00:00,226621,0.0,mL,2112-08-18 15:56:00,14542,NaN,NaN,NaN,Davol,Drains
4349214,3928871,42590,199643.0,291593.0,2112-08-18 20:00:00,226621,20.0,mL,2112-08-18 21:38:00,14576,NaN,NaN,NaN,Davol,Drains
4349215,3928872,42590,199643.0,291593.0,2112-08-19 00:00:00,226621,0.0,mL,2112-08-19 00:10:00,14576,NaN,NaN,NaN,Davol,Drains
4349216,3928873,42590,199643.0,291593.0,2112-08-19 04:00:00,226621,10.0,mL,2112-08-19 04:19:00,14576,NaN,NaN,NaN,Davol,Drains


In [90]:
file=open(output_files+'unique_labels_output_events.txt','w')
for item in unique_labels:
    file.write(item+'\n')

file.close()

In [91]:
outputevents[outputevents.LABEL.isin(['wound drain',
'wound drainage',
'wound drains',
'wound pouch',
'wound vac output.',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'wound drainage', 'wound drain', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'wound drains', 'wound drain', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'wound pouch', 'wound drain', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'wound vac output.', 'wound drain', inplace=True)
outputevents[outputevents.LABEL.isin(['wound drain'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2595939,144104,22835,143282.0,231550.0,2173-03-07 10:00:00,41212,210.0,ml,2173-03-07 10:39:00,20814,NaN,NaN,NaN,wound drain,NaN
2595940,144105,22835,143282.0,231550.0,2173-03-07 18:00:00,41212,300.0,ml,2173-03-07 18:29:00,20814,NaN,NaN,NaN,wound drain,NaN
2595941,144106,22835,143282.0,231550.0,2173-03-08 06:00:00,41212,250.0,ml,2173-03-08 06:30:00,15443,NaN,NaN,NaN,wound drain,NaN
2595942,144107,22835,143282.0,231550.0,2173-03-08 18:00:00,41212,250.0,ml,2173-03-08 18:43:00,19065,NaN,NaN,NaN,wound drain,NaN
2595943,144108,22835,143282.0,231550.0,2173-03-09 06:00:00,41212,600.0,ml,2173-03-09 07:01:00,18784,NaN,NaN,NaN,wound drain,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784455,2556057,27874,193876.0,200140.0,2197-01-29 12:00:00,46415,150.0,ml,2197-01-29 12:27:00,20816,NaN,NaN,NaN,wound drain,NaN
2784456,2556058,27874,193876.0,200140.0,2197-01-29 18:00:00,46415,100.0,ml,2197-01-29 18:24:00,20816,NaN,NaN,NaN,wound drain,NaN
2784457,2556059,27874,193876.0,200140.0,2197-01-29 23:00:00,46415,100.0,ml,2197-01-29 22:41:00,20581,NaN,NaN,NaN,wound drain,NaN
2784458,2549796,27874,193876.0,200140.0,2197-02-05 06:00:00,46415,100.0,ml,2197-02-05 06:32:00,14772,NaN,NaN,NaN,wound drain,NaN


In [92]:
outputevents[outputevents.LABEL.isin(['vomit',
'vomitus',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'vomitus', 'vomit', inplace=True)
outputevents[outputevents.LABEL.isin(['vomit'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2627163,242485,20936,157548.0,291919.0,2151-07-30 10:00:00,41722,220.0,ml,2151-07-30 12:10:00,21570,NaN,NaN,NaN,vomit,NaN
2627164,242486,20936,157548.0,291919.0,2151-07-31 14:00:00,41722,200.0,ml,2151-07-31 14:11:00,17741,NaN,NaN,NaN,vomit,NaN
2627165,242487,20936,157548.0,291919.0,2151-08-01 13:00:00,41722,100.0,ml,2151-08-01 13:40:00,14133,NaN,NaN,NaN,vomit,NaN
2627166,765298,3234,129980.0,215432.0,2191-12-13 19:00:00,41722,300.0,ml,2191-12-13 19:40:00,17707,NaN,NaN,NaN,vomit,NaN
2627167,1927208,26523,137437.0,242297.0,2192-10-27 01:00:00,41722,250.0,ml,2192-10-27 01:04:00,19937,NaN,NaN,NaN,vomit,NaN
2781948,1616224,12961,148651.0,280029.0,2173-02-01 18:00:00,44930,200.0,ml,2173-02-01 18:37:00,21058,NaN,NaN,NaN,vomit,Free Form Intake


In [93]:
outputevents[outputevents.LABEL.isin(['vent drain',
'vent. drain',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'vent. drain', 'vent drain', inplace=True)
outputevents[outputevents.LABEL.isin(['vent drain'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2778653,979294,24085,114359.0,223185.0,2112-06-30 02:00:00,43859,2.0,ml,2112-06-30 04:14:00,15023,NaN,NaN,NaN,vent drain,NaN
2778654,979295,24085,114359.0,223185.0,2112-06-30 04:00:00,43859,3.0,ml,2112-06-30 04:14:00,15023,NaN,NaN,NaN,vent drain,NaN
2778655,979296,24085,114359.0,223185.0,2112-06-30 06:00:00,43859,4.0,ml,2112-06-30 05:56:00,15023,NaN,NaN,NaN,vent drain,NaN
2778656,979297,24085,114359.0,223185.0,2112-06-30 10:00:00,43859,9.0,ml,2112-06-30 10:39:00,15029,NaN,NaN,NaN,vent drain,NaN
2778657,979298,24085,114359.0,223185.0,2112-06-30 13:00:00,43859,12.0,ml,2112-06-30 13:28:00,15029,NaN,NaN,NaN,vent drain,NaN
2778658,979299,24085,114359.0,223185.0,2112-06-30 15:00:00,43859,10.0,ml,2112-06-30 14:50:00,15029,NaN,NaN,NaN,vent drain,NaN
2778659,979300,24085,114359.0,223185.0,2112-06-30 16:00:00,43859,9.0,ml,2112-06-30 16:23:00,15029,NaN,NaN,NaN,vent drain,NaN
2778660,979301,24085,114359.0,223185.0,2112-06-30 18:00:00,43859,8.0,ml,2112-06-30 17:45:00,15029,NaN,NaN,NaN,vent drain,NaN
2778661,979302,24085,114359.0,223185.0,2112-06-30 20:00:00,43859,14.0,ml,2112-06-30 20:22:00,15023,NaN,NaN,NaN,vent drain,NaN
2778662,979303,24085,114359.0,223185.0,2112-06-30 22:00:00,43859,8.0,ml,2112-06-30 21:59:00,15023,NaN,NaN,NaN,vent drain,NaN


In [94]:
outputevents[outputevents.LABEL.isin(['vaginal drng',
'vag drng',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'vaginal drng','vag drng', inplace=True)
outputevents[outputevents.LABEL.isin(['vaginal drng'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY


In [95]:
outputevents[outputevents.LABEL.isin(['chest tube',
'chest tube # 3',
'chest tube #3',
'chest tube BEFORE OR',
'chest tube dsg',
'chest tube in or',
'chest tube irrigant',
'chest tube site',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube # 3', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube #3', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube BEFORE OR', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube dsg', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube in or', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube irrigant', 'chest tube', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'chest tube site', 'chest tube', inplace=True)
outputevents[outputevents.LABEL.isin(['chest tube'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2587990,86623,8714,164454.0,224083.0,2159-05-07 08:00:00,41003,100.0,ml,2159-05-07 11:47:00,17457,NaN,NaN,NaN,chest tube,Free Form Intake
2587992,86625,8714,164454.0,224083.0,2159-05-07 16:00:00,41003,100.0,ml,2159-05-07 16:10:00,17457,NaN,NaN,NaN,chest tube,Free Form Intake
2587993,86626,8714,164454.0,224083.0,2159-05-07 17:00:00,41003,100.0,ml,2159-05-07 17:11:00,17457,NaN,NaN,NaN,chest tube,Free Form Intake
2587998,86631,8714,164454.0,224083.0,2159-05-08 08:00:00,41003,100.0,ml,2159-05-08 08:52:00,14607,NaN,NaN,NaN,chest tube,Free Form Intake
2587999,86632,8714,164454.0,224083.0,2159-05-08 16:00:00,41003,100.0,ml,2159-05-08 16:12:00,14607,NaN,NaN,NaN,chest tube,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2783458,2082777,2176,145484.0,294014.0,2183-01-16 06:00:00,45664,10.0,ml,2183-01-16 07:03:00,20704,NaN,NaN,NaN,chest tube,NaN
2783459,2081448,2176,145484.0,294014.0,2183-01-17 14:00:00,45664,5.0,ml,2183-01-17 14:26:00,17457,NaN,NaN,NaN,chest tube,NaN
2783460,2748270,2176,145484.0,294014.0,2183-01-20 11:00:00,45664,0.0,NaN,2183-01-20 11:00:00,21570,NaN,NaN,NaN,chest tube,NaN
2784389,2346231,27543,113974.0,289072.0,2187-11-01 01:00:00,46237,1500.0,ml,2187-11-01 02:14:00,19611,NaN,NaN,NaN,chest tube,NaN


In [96]:
outputevents[outputevents.LABEL.isin(['24hr Urine cc/kg/hr','24hr urine output',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='24hr Urine cc/kg/hr','24hr urine output', inplace=True)
outputevents[outputevents.LABEL.isin(['24hr urine output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2777256,843453,6975,192431.0,211378.0,2149-04-29 00:00:00,43372,5.80,ml,2149-04-29 03:10:00,15658,NaN,NaN,NaN,24hr urine output,NaN
2777257,846983,6975,192431.0,211378.0,2149-05-23 14:00:00,43372,0.00,NaN,2149-05-23 14:00:00,21570,NaN,NaN,NaN,24hr urine output,NaN
2777653,857167,9550,167003.0,298740.0,2196-11-25 09:00:00,43576,0.00,NaN,2196-11-25 09:00:00,21570,NaN,NaN,NaN,24hr urine output,NaN
2777654,857182,9550,167003.0,298740.0,2196-11-16 09:00:00,43576,2.76,ml,2196-11-16 13:23:00,15156,NaN,NaN,NaN,24hr urine output,NaN


In [97]:
outputevents[outputevents.LABEL.isin(['ANGIO FOLEY URINE',
'ANGIO OUT',
'ANGIO OUTPUT',
'ANGIO URINE',
'ANGIO URINE O/P',
'ANGIO URINE OUT',
'ANGIO URINE OUTPUT',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='ANGIO FOLEY URINE','ANGIO URINE', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'ANGIO OUT','ANGIO URINE', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'ANGIO OUTPUT','ANGIO URINE', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'ANGIO URINE O/P','ANGIO URINE', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'ANGIO URINE OUT','ANGIO URINE', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'ANGIO URINE OUTPUT','ANGIO URINE', inplace=True)
outputevents[outputevents.LABEL.isin(['ANGIO URINE'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2628791,303985,15041,123035.0,276548.0,2170-08-31 02:00:00,42042,1000.0,ml,2170-08-31 02:23:00,14597,NaN,NaN,NaN,ANGIO URINE,NaN
2628792,303986,15041,123035.0,276548.0,2170-08-31 03:00:00,42042,0.0,NaN,2170-08-31 03:00:00,21570,NaN,NaN,NaN,ANGIO URINE,NaN
2628793,527420,16221,199877.0,285958.0,2195-10-17 01:00:00,42042,100.0,ml,2195-10-17 05:15:00,20868,NaN,NaN,NaN,ANGIO URINE,NaN
2628794,527421,16221,199877.0,285958.0,2195-10-17 05:00:00,42042,0.0,NaN,2195-10-17 05:00:00,21570,NaN,NaN,NaN,ANGIO URINE,NaN
2628795,1014792,4656,149339.0,242611.0,2164-08-16 16:00:00,42042,625.0,ml,2164-08-16 16:10:00,17122,NaN,NaN,NaN,ANGIO URINE,NaN
2628796,2773057,4656,149339.0,242611.0,2164-08-18 06:00:00,42042,0.0,NaN,2164-08-18 06:00:00,21570,NaN,NaN,NaN,ANGIO URINE,NaN
2637783,323791,14906,116644.0,293241.0,2154-09-06 23:00:00,42111,800.0,ml,2154-09-07 02:16:00,17246,NaN,NaN,NaN,ANGIO URINE,NaN
2637784,323792,14906,116644.0,293241.0,2154-09-07 07:00:00,42111,0.0,NaN,2154-09-07 07:00:00,21570,NaN,NaN,NaN,ANGIO URINE,NaN
2641606,328629,25270,120136.0,272710.0,2105-10-16 04:00:00,42128,680.0,ml,2105-10-16 05:03:00,17194,NaN,NaN,NaN,ANGIO URINE,NaN
2641607,328630,25270,120136.0,272710.0,2105-10-16 05:00:00,42128,0.0,NaN,2105-10-16 05:00:00,21570,NaN,NaN,NaN,ANGIO URINE,NaN


In [98]:
outputevents[outputevents.LABEL.isin(['Cath Lab',
'Cath Lab Out',
'Cath Lab Output',
'Cath Lab Urine Out',
'Cath lab output',
'CathLab Output',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='Cath Lab Out','Cath Lab', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Cath Lab Output','Cath Lab', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Cath Lab Urine Out','Cath Lab', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Cath lab output','Cath Lab', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='CathLab Output','Cath Lab', inplace=True)
outputevents[outputevents.LABEL.isin(['Cath Lab'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2558222,18802,1340,169611.0,294331.0,2193-12-17 14:00:00,40580,0.0,ml,2193-12-17 14:31:00,15654,NaN,NaN,NaN,Cath Lab,Free Form Intake
2558223,18803,1340,169611.0,294331.0,2193-12-17 15:00:00,40580,0.0,ml,2193-12-17 15:15:00,15654,NaN,NaN,NaN,Cath Lab,Free Form Intake
2558224,18804,1340,169611.0,294331.0,2193-12-17 16:00:00,40580,0.0,ml,2193-12-17 16:18:00,15654,NaN,NaN,NaN,Cath Lab,Free Form Intake
2558225,18805,1340,169611.0,294331.0,2193-12-17 17:00:00,40580,0.0,ml,2193-12-17 16:51:00,15654,NaN,NaN,NaN,Cath Lab,Free Form Intake
2558226,18806,1340,169611.0,294331.0,2193-12-17 18:00:00,40580,0.0,ml,2193-12-17 18:04:00,15654,NaN,NaN,NaN,Cath Lab,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4334463,4345934,79069,115164.0,226424.0,2200-10-20 16:30:00,226632,500.0,mL,2200-10-20 16:23:00,14214,NaN,NaN,NaN,Cath Lab,Output
4334464,4345935,79069,115164.0,226424.0,2200-10-20 17:00:00,226632,0.0,mL,2200-10-20 18:58:00,14214,NaN,NaN,NaN,Cath Lab,Output
4334465,4345936,79069,115164.0,226424.0,2200-10-20 18:00:00,226632,0.0,mL,2200-10-20 18:58:00,14214,NaN,NaN,NaN,Cath Lab,Output
4336384,2668744,27780,163984.0,234755.0,2121-07-15 08:00:00,46748,0.0,NaN,2121-07-15 08:00:00,21570,NaN,NaN,NaN,Cath Lab,NaN


In [99]:
outputevents[outputevents.LABEL.isin(['Cardiac Output',
'Cardiac output',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='Cardiac output','Cardiac Output', inplace=True)
outputevents[outputevents.LABEL.isin(['Cardiac Output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2781921,1611301,6873,186838.0,200438.0,2192-12-27 03:15:00,44920,40.0,ml,2192-12-27 03:40:00,16707,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2781922,1611302,6873,186838.0,200438.0,2192-12-27 04:00:00,44920,40.0,ml,2192-12-27 04:00:00,16707,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2781923,1611303,6873,186838.0,200438.0,2192-12-27 05:00:00,44920,40.0,ml,2192-12-27 04:35:00,16707,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2781924,1611304,6873,186838.0,200438.0,2192-12-27 06:00:00,44920,40.0,ml,2192-12-27 05:59:00,16707,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2781925,1611305,6873,186838.0,200438.0,2192-12-27 07:00:00,44920,40.0,ml,2192-12-27 06:41:00,16707,NaN,NaN,NaN,Cardiac Output,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782136,2210731,3868,107365.0,261153.0,2125-06-18 03:00:00,44970,40.0,ml,2125-06-18 05:12:00,15803,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2782137,2210732,3868,107365.0,261153.0,2125-06-18 04:00:00,44970,40.0,ml,2125-06-18 05:12:00,15803,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2782138,2210733,3868,107365.0,261153.0,2125-06-18 05:00:00,44970,40.0,ml,2125-06-18 05:12:00,15803,NaN,NaN,NaN,Cardiac Output,Free Form Intake
2782139,2210734,3868,107365.0,261153.0,2125-06-18 06:00:00,44970,40.0,ml,2125-06-18 05:54:00,15803,NaN,NaN,NaN,Cardiac Output,Free Form Intake


In [100]:
outputevents[outputevents.LABEL.isin(['HEMODIALYSIS',
'HEMODIALYSIS O/P',
'HEMODIALYSIS OFF',
'HEMODIALYSIS OUT',
'HEMODIALYSIS OUTPUT',
'HEMODIALYSIS.',
'Hemo dialysis out',
'HemoDialysis',
'Hemodialysis',
'Hemodialysis OUT',
'Hemodialysis Out',
'Hemodialysis out',
'Hemodialysis removal',
'Hemodialysis.',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS O/P', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS OFF', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS OUT', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS OUTPUT', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HEMODIALYSIS.', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Hemo dialysis out', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='HemoDialysis', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Hemodialysis OUT', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Hemodialysis Out', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Hemodialysis out', 'Hemodialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='Hemodialysis.', 'Hemodialysis', inplace=True)
outputevents[outputevents.LABEL.isin(['Hemodialysis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2584798,61871,12564,115411.0,225836.0,2144-11-19 20:00:00,40881,2000.0,ml,2144-11-19 20:23:00,17155,NaN,NaN,NaN,Hemodialysis,NaN
2584799,108208,17436,103268.0,263452.0,2156-08-21 11:30:00,40881,4400.0,ml,2156-08-21 13:02:00,15815,NaN,NaN,NaN,Hemodialysis,NaN
2584800,104531,25668,180457.0,275876.0,2192-06-28 17:35:00,40881,1500.0,ml,2192-06-28 17:56:00,14996,NaN,NaN,NaN,Hemodialysis,NaN
2584801,104532,25668,180457.0,275876.0,2192-06-30 13:45:00,40881,3000.0,ml,2192-06-30 14:17:00,20675,NaN,NaN,NaN,Hemodialysis,NaN
2584802,110820,17436,103268.0,263452.0,2156-08-23 11:00:00,40881,4000.0,ml,2156-08-23 11:35:00,18576,NaN,NaN,NaN,Hemodialysis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780604,1396119,10465,114142.0,223849.0,2162-07-31 16:00:00,44567,1900.0,ml,2162-07-31 16:21:00,18234,NaN,NaN,NaN,Hemodialysis,NaN
2783303,2153752,16490,151602.0,239488.0,2149-06-05 18:30:00,45828,3000.0,ml,2149-06-05 18:45:00,17476,NaN,NaN,NaN,Hemodialysis,NaN
2783304,2155399,16490,151602.0,239488.0,2149-06-06 10:00:00,45828,0.3,ml,2149-06-06 09:47:00,17476,NaN,NaN,NaN,Hemodialysis,NaN
2783305,2163700,16490,151602.0,239488.0,2149-06-07 11:30:00,45828,2500.0,ml,2149-06-07 11:38:00,19528,NaN,NaN,NaN,Hemodialysis,NaN


In [101]:
outputevents[outputevents.LABEL.isin(['Dialysis',
'Dialysis Out',
'Dialysis Output.',
'Dialysis Removed',
'Dialysis out',
'Dialysis output',
'Dialysis.',
'Dialysys out',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis.', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis out', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis Out', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis Output.', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis Removed', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysis output', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Dialysys out', 'Dialysis', inplace=True)
outputevents[outputevents.LABEL.isin(['Dialysis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2329467,7571,6920,158965.0,261140.0,2157-09-29 17:00:00,40507,4000.0,ml,2157-09-29 17:30:00,15218,NaN,NaN,NaN,Dialysis,NaN
2329468,7572,6920,158965.0,261140.0,2157-09-30 12:00:00,40507,3000.0,ml,2157-09-30 11:57:00,15823,NaN,NaN,NaN,Dialysis,NaN
2329469,1214627,19213,114282.0,225773.0,2204-01-15 04:00:00,40507,2750.0,ml,2204-01-15 04:42:00,16012,NaN,NaN,NaN,Dialysis,NaN
2329470,1214628,19213,114282.0,225773.0,2204-01-15 08:00:00,40507,2000.0,ml,2204-01-15 08:08:00,15805,NaN,NaN,NaN,Dialysis,NaN
2329471,1214629,19213,114282.0,225773.0,2204-01-15 12:00:00,40507,2500.0,ml,2204-01-15 12:28:00,15805,NaN,NaN,NaN,Dialysis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779114,1148087,6334,115026.0,247862.0,2168-01-11 13:00:00,44193,2400.0,ml,2168-01-11 12:51:00,19689,NaN,NaN,NaN,Dialysis,NaN
2779115,1157822,6334,115026.0,247862.0,2168-01-13 13:00:00,44193,500.0,ml,2168-01-13 13:02:00,19689,NaN,NaN,NaN,Dialysis,NaN
2779116,1177101,6334,115026.0,247862.0,2168-01-18 08:00:00,44193,0.0,NaN,2168-01-18 08:00:00,21570,NaN,NaN,NaN,Dialysis,NaN
2781956,1619244,6392,182162.0,234103.0,2191-01-21 17:00:00,44901,2500.0,ml,2191-01-21 17:15:00,17877,NaN,NaN,NaN,Dialysis,NaN


In [102]:
outputevents[outputevents.LABEL.isin(['Dialysis','DIALYSIS',
'DIALYSIS OFF',
'DIALYSIS OUT',
'DIALYSIS OUTPUT',
'DIALYSIS REMOVED',
'DIALYSIS TOTAL OUT',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS OFF', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS OUT', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS OUTPUT', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS REMOVED', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS TOTAL OUT', 'Dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'DIALYSIS', 'Dialysis', inplace=True)
outputevents[outputevents.LABEL.isin(['Dialysis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2329467,7571,6920,158965.0,261140.0,2157-09-29 17:00:00,40507,4000.0,ml,2157-09-29 17:30:00,15218,NaN,NaN,NaN,Dialysis,NaN
2329468,7572,6920,158965.0,261140.0,2157-09-30 12:00:00,40507,3000.0,ml,2157-09-30 11:57:00,15823,NaN,NaN,NaN,Dialysis,NaN
2329469,1214627,19213,114282.0,225773.0,2204-01-15 04:00:00,40507,2750.0,ml,2204-01-15 04:42:00,16012,NaN,NaN,NaN,Dialysis,NaN
2329470,1214628,19213,114282.0,225773.0,2204-01-15 08:00:00,40507,2000.0,ml,2204-01-15 08:08:00,15805,NaN,NaN,NaN,Dialysis,NaN
2329471,1214629,19213,114282.0,225773.0,2204-01-15 12:00:00,40507,2500.0,ml,2204-01-15 12:28:00,15805,NaN,NaN,NaN,Dialysis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779115,1157822,6334,115026.0,247862.0,2168-01-13 13:00:00,44193,500.0,ml,2168-01-13 13:02:00,19689,NaN,NaN,NaN,Dialysis,NaN
2779116,1177101,6334,115026.0,247862.0,2168-01-18 08:00:00,44193,0.0,NaN,2168-01-18 08:00:00,21570,NaN,NaN,NaN,Dialysis,NaN
2780061,1092527,24601,193151.0,225189.0,2181-10-11 14:00:00,44085,2500.0,ml,2181-10-11 15:13:00,18078,NaN,NaN,NaN,Dialysis,NaN
2781956,1619244,6392,182162.0,234103.0,2191-01-21 17:00:00,44901,2500.0,ml,2191-01-21 17:15:00,17877,NaN,NaN,NaN,Dialysis,NaN


In [103]:
outputevents[outputevents.LABEL.isin(['Angio OUT',
'Angio Out',
'Angio Output',
'Angio output',
'Angiography Output',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'Angio output','Angiography Output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Angio Output','Angiography Output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Angio Out','Angiography Output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Angio OUT','Angiography Output', inplace=True)
outputevents[outputevents.LABEL.isin(['Angiography Output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2649016,718868,12801,182897.0,296640.0,2186-08-24 01:00:00,42957,500.0,ml,2186-08-24 04:42:00,14413,NaN,NaN,NaN,Angiography Output,NaN
2649017,718869,12801,182897.0,296640.0,2186-08-24 04:00:00,42957,0.0,NaN,2186-08-24 04:00:00,21570,NaN,NaN,NaN,Angiography Output,NaN
2779112,1070427,2197,186714.0,242326.0,2164-09-14 10:00:00,44046,2200.0,ml,2164-09-14 11:27:00,14367,NaN,NaN,NaN,Angiography Output,NaN
2779113,1070428,2197,186714.0,242326.0,2164-09-14 11:00:00,44046,0.0,NaN,2164-09-14 11:00:00,21570,NaN,NaN,NaN,Angiography Output,NaN
2780487,1178354,25315,177247.0,237007.0,2199-02-03 15:00:00,44231,1000.0,ml,2199-02-03 16:07:00,14367,NaN,NaN,NaN,Angiography Output,NaN
2780488,1178355,25315,177247.0,237007.0,2199-02-03 16:00:00,44231,0.0,NaN,2199-02-03 16:00:00,21570,NaN,NaN,NaN,Angiography Output,NaN
2782792,1965913,16663,140770.0,279045.0,2136-11-16 17:00:00,45413,225.0,ml,2136-11-16 18:06:00,16866,NaN,NaN,NaN,Angiography Output,NaN
2784464,2548119,31190,159351.0,292101.0,2124-03-01 15:30:00,46604,1500.0,ml,2124-03-01 18:38:00,17189,NaN,NaN,NaN,Angiography Output,NaN
2784465,2548120,31190,159351.0,292101.0,2124-03-01 18:00:00,46604,0.0,NaN,2124-03-01 18:00:00,21570,NaN,NaN,NaN,Angiography Output,NaN
2784466,2559769,31190,159351.0,292101.0,2124-03-03 13:45:00,46604,1300.0,ml,2124-03-03 13:40:00,16866,NaN,NaN,NaN,Angiography Output,NaN


In [104]:
outputevents[outputevents.LABEL.isin(['Paracentesis',
'Paracentesis Output',
'Paracentesis Site',
'Paracentesis out',
'Paracentesis output',
'Paracentesis site.',
'Parencentesis Out',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis Output','Paracentesis',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis Site','Paracentesis',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis out','Paracentesis',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis output','Paracentesis',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis site','Paracentesis',inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Paracentesis Out','Paracentesis',inplace=True)
outputevents[outputevents.LABEL.isin(['Paracentesis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2251126,2461,1284,118883.0,279148.0,2158-10-17 17:00:00,40451,550.0,ml,2158-10-17 20:46:00,15815,NaN,NaN,NaN,Paracentesis,NaN
2251127,2462,1284,118883.0,279148.0,2158-10-19 09:00:00,40451,0.0,NaN,2158-10-19 09:00:00,21570,NaN,NaN,NaN,Paracentesis,NaN
2251128,76329,25674,198147.0,204016.0,2142-12-18 18:00:00,40451,0.0,NaN,2142-12-18 18:00:00,21570,NaN,NaN,NaN,Paracentesis,NaN
2251129,193677,24560,154385.0,210953.0,2148-11-23 08:00:00,40451,0.0,ml,2148-11-23 08:45:00,20485,NaN,NaN,NaN,Paracentesis,NaN
2251130,193678,24560,154385.0,210953.0,2148-11-23 10:00:00,40451,0.0,ml,2148-11-23 11:57:00,20485,NaN,NaN,NaN,Paracentesis,NaN
2251131,193679,24560,154385.0,210953.0,2148-11-23 12:00:00,40451,0.0,ml,2148-11-23 11:57:00,20485,NaN,NaN,NaN,Paracentesis,NaN
2251132,193680,24560,154385.0,210953.0,2148-11-23 14:00:00,40451,0.0,ml,2148-11-23 14:21:00,20485,NaN,NaN,NaN,Paracentesis,NaN
2251133,193681,24560,154385.0,210953.0,2148-11-23 16:00:00,40451,0.0,ml,2148-11-23 16:44:00,20485,NaN,NaN,NaN,Paracentesis,NaN
2251134,193682,24560,154385.0,210953.0,2148-11-25 18:00:00,40451,6000.0,ml,2148-11-25 17:52:00,17248,NaN,NaN,NaN,Paracentesis,NaN
2251135,195598,24560,154385.0,210953.0,2148-11-18 19:00:00,40451,475.0,ml,2148-11-18 19:33:00,17825,NaN,NaN,NaN,Paracentesis,NaN


In [105]:
outputevents[outputevents.LABEL.isin(['angio Urine output',
'angio out',
'angio output',
'angio urine',
'angio urine out',
'angio urine output',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'angio urine out','angio urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'angio out','angio urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'angio output','angio urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'angio Urine output','angio urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'angio urine','angio urine output', inplace=True)
outputevents[outputevents.LABEL.isin(['angio urine output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2612424,153416,6555,172991.0,293686.0,2106-08-19 15:00:00,41384,1050.0,ml,2106-08-19 15:22:00,17194,NaN,NaN,NaN,angio urine output,NaN
2612425,153417,6555,172991.0,293686.0,2106-08-19 16:00:00,41384,0.0,NaN,2106-08-19 16:00:00,21570,NaN,NaN,NaN,angio urine output,NaN
2612426,621831,20012,148002.0,252493.0,2150-04-10 20:00:00,41384,1300.0,ml,2150-04-10 20:07:00,20780,NaN,NaN,NaN,angio urine output,NaN
2612427,743364,10361,195393.0,224225.0,2177-12-10 23:10:00,41384,1200.0,ml,2177-12-10 23:30:00,17190,NaN,NaN,NaN,angio urine output,NaN
2612428,743365,10361,195393.0,224225.0,2177-12-10 23:11:00,41384,0.0,NaN,2177-12-10 23:11:00,21570,NaN,NaN,NaN,angio urine output,NaN
2612429,811449,10037,107919.0,278481.0,2170-01-30 18:15:00,41384,500.0,ml,2170-01-30 18:31:00,17190,NaN,NaN,NaN,angio urine output,NaN
2612430,811450,10037,107919.0,278481.0,2170-01-30 21:00:00,41384,0.0,NaN,2170-01-30 21:00:00,21570,NaN,NaN,NaN,angio urine output,NaN
2612431,1236346,14300,126566.0,261860.0,2194-04-10 13:00:00,41384,1300.0,ml,2194-04-10 13:47:00,15046,NaN,NaN,NaN,angio urine output,NaN
2612432,2401401,32786,165351.0,275017.0,2152-11-16 15:00:00,41384,0.0,NaN,2152-11-16 15:00:00,21570,NaN,NaN,NaN,angio urine output,NaN
2636930,306530,13262,105239.0,216340.0,2131-10-11 17:00:00,42068,5575.0,ml,2131-10-11 17:08:00,21570,NaN,NaN,NaN,angio urine output,NaN


In [106]:
outputevents[outputevents.LABEL.isin(['cardiac output',
'cardiac output fluid',
'cardiac output ml',])]
outputevents['LABEL'].mask(outputevents['LABEL']=='cardiac output fluid', 'cardiac output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='cardiac output ml', 'cardiac output', inplace=True)
outputevents[outputevents.LABEL.isin(['cardiac output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2587831,69500,2628,105769.0,252616.0,2125-12-23 18:00:00,40909,40.0,ml,2125-12-23 18:12:00,20814,NaN,NaN,NaN,cardiac output,NaN
2587832,69501,2628,105769.0,252616.0,2125-12-24 08:00:00,40909,0.0,NaN,2125-12-24 08:00:00,21570,NaN,NaN,NaN,cardiac output,NaN
2587833,373151,14472,136494.0,284812.0,2176-11-21 01:00:00,40909,30.0,ml,2176-11-21 00:36:00,15449,NaN,NaN,NaN,cardiac output,NaN
2587834,373152,14472,136494.0,284812.0,2176-11-21 02:00:00,40909,30.0,ml,2176-11-21 02:00:00,15841,NaN,NaN,NaN,cardiac output,NaN
2587835,373153,14472,136494.0,284812.0,2176-11-21 04:00:00,40909,30.0,ml,2176-11-21 03:58:00,15449,NaN,NaN,NaN,cardiac output,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613154,1106851,1266,189402.0,251265.0,2106-12-06 01:00:00,41562,30.0,ml,2106-12-06 02:15:00,20131,NaN,NaN,NaN,cardiac output,Free Form Intake
2613155,1106852,1266,189402.0,251265.0,2106-12-06 02:00:00,41562,30.0,ml,2106-12-06 02:15:00,20131,NaN,NaN,NaN,cardiac output,Free Form Intake
2613156,1102909,1266,189402.0,251265.0,2106-12-04 21:00:00,41562,30.0,ml,2106-12-04 21:44:00,20131,NaN,NaN,NaN,cardiac output,Free Form Intake
2613157,1102910,1266,189402.0,251265.0,2106-12-04 22:00:00,41562,30.0,ml,2106-12-04 23:25:00,20131,NaN,NaN,NaN,cardiac output,Free Form Intake


In [107]:
outputevents[outputevents.LABEL.isin(['thoarcentesis',
'thoracentesis',
'thoracentesis fluid',
'thoracentesis left',
'thoracentesis out',
'thoracentesis tap',
'thorcentesis',
'thorencentesis',
'thorocentesis',
'thorocentesis fluid',
'thorocentesis output',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'thoarcentesis','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thoracentesis fluid','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thoracentesis left','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thoracentesis out','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thoracentesis tap','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thorcentesis','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thorencentesis','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thorocentesis','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thorocentesis fluid','thoracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'thorocentesis output','thoracentesis', inplace=True)

outputevents[outputevents.LABEL.isin(['thoracentesis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2558490,18811,15609,194043.0,283806.0,2104-03-01 14:00:00,40403,800.0,ml,2104-03-01 15:12:00,15654,NaN,NaN,NaN,thoracentesis,NaN
2558491,18812,15609,194043.0,283806.0,2104-03-05 20:00:00,40403,0.0,NaN,2104-03-05 20:00:00,21570,NaN,NaN,NaN,thoracentesis,NaN
2558492,28465,20779,154357.0,201462.0,2149-05-14 12:00:00,40403,700.0,ml,2149-05-14 13:01:00,21570,NaN,NaN,NaN,thoracentesis,NaN
2558493,34738,13973,193773.0,242073.0,2179-04-09 18:00:00,40403,900.0,ml,2179-04-09 18:02:00,21570,NaN,NaN,NaN,thoracentesis,NaN
2558494,32056,6464,121677.0,214810.0,2152-04-05 15:30:00,40403,800.0,ml,2152-04-05 15:39:00,15029,NaN,NaN,NaN,thoracentesis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4336368,2675144,27781,120626.0,240062.0,2127-08-13 14:00:00,46759,1400.0,ml,2127-08-13 16:58:00,20482,NaN,NaN,NaN,thoracentesis,NaN
4336369,2675145,27781,120626.0,240062.0,2127-08-13 16:30:00,46759,1650.0,ml,2127-08-13 16:58:00,20482,NaN,NaN,NaN,thoracentesis,NaN
4336370,2677135,27781,120626.0,240062.0,2127-08-14 21:00:00,46759,0.0,NaN,2127-08-14 21:00:00,21570,NaN,NaN,NaN,thoracentesis,NaN
4336389,2684477,27077,154353.0,214710.0,2128-08-13 17:30:00,46742,2000.0,ml,2128-08-13 17:41:00,18121,NaN,NaN,NaN,thoracentesis,NaN


In [108]:
outputevents[outputevents.LABEL.isin(['Chest Tube #1',
'Chest Tube #2',
'Chest Tube #3',
'Chest Tube In',
'Chest Tube R #3',
'Chest Tubes CTICU CT 1',
'Chest Tubes CTICU CT 2',
'Chest Tubes Chest Tube',
'Chest Tubes Left Pleural 1',
'Chest Tubes Left Pleural 2',
'Chest Tubes Mediastinal',
'Chest Tubes Other',
'Chest Tubes Right Pleural 1',
'Chest Tubes Right Pleural 2',
'Chest tube flush',
'Chest tube irrigate',
'Chest tube pleural',])]
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube #3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube In', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tube R #3', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes CTICU CT 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes CTICU CT 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Chest Tube', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Left Pleural 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Left Pleural 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Mediastinal', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Right Pleural 2', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube flush', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest Tubes Right Pleural 1', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube irrigate', 'Chest Tubes Other', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Chest tube pleural', 'Chest Tubes Other', inplace=True)

outputevents[outputevents.LABEL.isin(['Chest Tubes Other'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2214943,1774,5896,199422.0,285436.0,2188-07-24 06:00:00,40050,110.0,ml,2188-07-24 06:43:00,15029,NaN,NaN,NaN,Chest Tubes Other,NaN
2214944,19601,2355,191024.0,292695.0,2158-01-28 07:00:00,40050,70.0,ml,2158-01-28 07:12:00,14442,NaN,NaN,NaN,Chest Tubes Other,NaN
2214945,26149,10168,184129.0,NaN,2111-03-08 04:00:00,40050,20.0,ml,2111-03-08 04:50:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
2214946,26150,10168,184129.0,NaN,2111-03-08 07:00:00,40050,30.0,ml,2111-03-08 07:13:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
2214947,26151,10168,184129.0,NaN,2111-03-08 16:00:00,40050,60.0,ml,2111-03-08 17:20:00,21570,NaN,NaN,NaN,Chest Tubes Other,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4292402,4344731,45141,178080.0,287521.0,2113-10-15 08:00:00,226589,90.0,mL,2113-10-15 08:50:00,16593,NaN,NaN,NaN,Chest Tubes Other,Output
4292403,4344732,45141,178080.0,287521.0,2113-10-15 15:00:00,226589,0.0,mL,2113-10-15 14:59:00,16593,NaN,NaN,NaN,Chest Tubes Other,Output
4292404,4344733,45141,178080.0,287521.0,2113-10-16 05:03:00,226589,200.0,mL,2113-10-16 05:03:00,19611,NaN,NaN,NaN,Chest Tubes Other,Output
4292405,4344734,45141,178080.0,287521.0,2113-10-16 09:00:00,226589,70.0,mL,2113-10-16 10:00:00,19174,NaN,NaN,NaN,Chest Tubes Other,Output


In [109]:
outputevents[outputevents.LABEL.isin(['urine',
'urine amnt',
'urine cc\'s/k/hr',
'urine cc/k/ghr',
'urine cc/k/hr',
'urine cc/kg',
'urine cc/kg/hr',
'urine flush',
'urine from floor',
'urine in er.',
'urine o/p',
'urine o/p cc/kg/hr',
'urine out in er',
'urine out or',
'urine out-angio',
'urine out: cc/k/hr',
'urine out:cc/k/hr',
'urine outpt cc/kg/hr',
'urine output',
'urine output cc/k/hr',
'urine output farr-10',
'urine output-angio',
'urine output/kg',
'urine output/kg/hr',
'urine/k/hr',
'urinecc/kg/hr',])]

outputevents['LABEL'].mask(outputevents['LABEL']=='urine','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='urine amnt', 'urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']=='urine cc\'s/k/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine cc/k/ghr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine cc/k/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine flush','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine from floor','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine in er.','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine o/p','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine o/p cc/kg/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine out in er','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine out or','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine out-angio','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine out: cc/k/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine out:cc/k/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine output farr-10','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine output-angio','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine output/kg','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine/k/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urine output/kg/hr','urine output', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'urinecc/kg/hr','urine output', inplace=True)

outputevents[outputevents.LABEL.isin(['urine output'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2339469,9264,24804,126652.0,267509.0,2160-12-18 12:00:00,40534,60.0,ml,2160-12-18 12:28:00,19942,NaN,NaN,NaN,urine output,Free Form Intake
2339470,9265,24804,126652.0,267509.0,2160-12-18 14:00:00,40534,60.0,ml,2160-12-18 13:49:00,19942,NaN,NaN,NaN,urine output,Free Form Intake
2339471,9266,24804,126652.0,267509.0,2160-12-18 15:00:00,40534,0.0,NaN,2160-12-18 15:00:00,21570,NaN,NaN,NaN,urine output,Free Form Intake
2628770,288701,9630,113377.0,217528.0,2191-04-19 18:00:00,41857,500.0,ml,2191-04-19 17:54:00,20848,NaN,NaN,NaN,urine output,NaN
2641600,330969,25787,127393.0,293790.0,2145-09-15 14:00:00,42119,1100.0,ml,2145-09-15 14:49:00,15805,NaN,NaN,NaN,urine output,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2781939,1609002,18217,158646.0,246155.0,2117-02-08 14:00:00,44911,0.0,NaN,2117-02-08 14:00:00,21570,NaN,NaN,NaN,urine output,NaN
2783772,2310151,30730,148481.0,228942.0,2115-09-10 11:00:00,46180,500.0,ml,2115-09-10 12:06:00,15805,NaN,NaN,NaN,urine output,NaN
2783773,2327736,30730,148481.0,228942.0,2115-09-14 08:00:00,46180,0.0,NaN,2115-09-14 08:00:00,21570,NaN,NaN,NaN,urine output,NaN
2785278,2661297,31489,187298.0,270175.0,2110-06-30 10:00:00,46727,0.0,NaN,2110-06-30 10:00:00,21570,NaN,NaN,NaN,urine output,NaN


In [110]:
outputevents[outputevents.LABEL.isin(['dialysis',
'dialysis  out',
'dialysis fluid off',
'dialysis flush',
'dialysis net',
'dialysis off',
'dialysis out',
'dialysis output',
'dialysis removed',
'dialysis- fluid off',
'diayalsis out.',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis out','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis fluid off','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis net','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis flush','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis net','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis off','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis out','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis output','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis removed','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis fluid off','dialysis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'dialysis out.','dialysis', inplace=True)

outputevents[outputevents.LABEL.isin(['dialysis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2213053,1166,1900,116403.0,287745.0,2155-09-08 13:00:00,40426,3500.0,ml,2155-09-08 13:16:00,18557,NaN,NaN,NaN,dialysis,NaN
2213054,14630,7605,107327.0,276732.0,2147-01-19 03:00:00,40426,3000.0,ml,2147-01-19 13:13:00,18566,NaN,NaN,NaN,dialysis,NaN
2213055,14631,7605,107327.0,276732.0,2147-01-19 15:00:00,40426,900.0,ml,2147-01-19 15:47:00,18566,NaN,NaN,NaN,dialysis,NaN
2213056,14632,7605,107327.0,276732.0,2147-01-21 19:00:00,40426,0.0,NaN,2147-01-21 19:00:00,21570,NaN,NaN,NaN,dialysis,NaN
2213057,39625,16700,110554.0,252199.0,2130-07-08 16:00:00,40426,700.0,ml,2130-07-08 17:19:00,20814,NaN,NaN,NaN,dialysis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777716,2738479,15775,108010.0,271189.0,2125-04-18 02:00:00,43115,0.0,ml,2125-04-18 05:49:00,15824,NaN,NaN,NaN,dialysis,NaN
2779111,1064880,4869,122893.0,232304.0,2196-11-20 11:00:00,44027,2700.0,ml,2196-11-20 11:41:00,18121,NaN,NaN,NaN,dialysis,NaN
2783973,2340447,26828,194307.0,201074.0,2112-08-23 10:00:00,46232,100.0,ml,2112-08-23 11:01:00,19263,NaN,NaN,NaN,dialysis,Free Form Intake
2783974,2340448,26828,194307.0,201074.0,2112-08-23 11:00:00,46232,0.0,NaN,2112-08-23 11:00:00,21570,NaN,NaN,NaN,dialysis,Free Form Intake


In [111]:
outputevents[outputevents.LABEL.isin(['Total Blood Out',
'Total blood out',
'blood out',
'blood removed...',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'Total blood out','Total Blood Out', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'blood out', 'Total Blood Out', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'blood removed...','Total Blood Out', inplace=True)

outputevents[outputevents.LABEL.isin(['Total Blood Out'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2777488,851020,23702,127293.0,257640.0,2138-04-18 10:50:00,43538,55.000,ml,2138-04-18 11:16:00,21232,NaN,NaN,NaN,Total Blood Out,NaN
2777632,864170,10253,133940.0,276996.0,2123-04-23 23:00:00,43606,20.000,ml,2123-04-24 03:08:00,14376,NaN,NaN,NaN,Total Blood Out,NaN
2777633,864602,10253,133940.0,276996.0,2123-04-27 10:00:00,43606,0.000,NaN,2123-04-27 10:00:00,21570,NaN,NaN,NaN,Total Blood Out,NaN
2777634,879811,20887,183921.0,265127.0,2111-02-06 16:00:00,43606,0.147,ml,2111-02-06 17:08:00,15064,NaN,NaN,NaN,Total Blood Out,NaN
2777635,879812,20887,183921.0,265127.0,2111-02-07 04:00:00,43606,0.300,ml,2111-02-07 06:26:00,19599,NaN,NaN,NaN,Total Blood Out,NaN
2777636,879813,20887,183921.0,265127.0,2111-02-07 10:00:00,43606,0.300,ml,2111-02-07 10:58:00,15064,NaN,NaN,NaN,Total Blood Out,NaN
2777637,879814,20887,183921.0,265127.0,2111-02-08 00:00:00,43606,0.300,ml,2111-02-08 02:30:00,19599,NaN,NaN,NaN,Total Blood Out,NaN
2777638,879815,20887,183921.0,265127.0,2111-02-15 12:30:00,43606,0.500,ml,2111-02-15 17:31:00,17986,NaN,NaN,NaN,Total Blood Out,NaN
2777639,879816,20887,183921.0,265127.0,2111-02-15 16:30:00,43606,1.500,ml,2111-02-15 17:31:00,17986,NaN,NaN,NaN,Total Blood Out,NaN
2777640,881612,20887,107749.0,233682.0,2111-01-18 22:00:00,43606,0.800,ml,2111-01-18 23:46:00,15658,NaN,NaN,NaN,Total Blood Out,NaN


In [112]:
outputevents[outputevents.LABEL.isin(['vaginal discharge',
'vaginal fistula',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'vaginal fistula', 'vaginal discharge', inplace=True)
outputevents[outputevents.LABEL.isin(['vaginal discharge'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2783410,2176159,2635,169533.0,222468.0,2168-05-27 14:00:00,45844,500.0,ml,2168-05-27 14:43:00,20781,NaN,NaN,NaN,vaginal discharge,NaN
2783415,2172714,2635,169533.0,222468.0,2168-05-28 08:00:00,45844,0.0,ml,2168-05-28 10:11:00,16012,NaN,NaN,NaN,vaginal discharge,NaN


In [113]:
outputevents[outputevents.LABEL.isin(['ABDOMINAL DRAINAGE'
'Abd drain output',
'Abd wound drains',
'Abdominal drainage',
'abd drain',
'abd wd drainage',
'abd. cyst drain',
'abdominal drain',])]


outputevents['LABEL'].mask(outputevents['LABEL']== 'ABDOMINAL DRAINAGE', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Abd drain output', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'Abd wound drains', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'abd drain', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'abd wd drainage', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'abd. cyst drain', 'Abdominal drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'abdominal drain', 'Abdominal drainage', inplace=True)

outputevents[outputevents.LABEL.isin(['Abdominal drainage'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2595831,133481,1798,179432.0,297430.0,2194-02-02 15:30:00,41178,600.0,ml,2194-02-02 18:12:00,17457,NaN,NaN,NaN,Abdominal drainage,NaN
2595832,133482,1798,179432.0,297430.0,2194-02-03 05:00:00,41178,30.0,ml,2194-02-03 05:11:00,21290,NaN,NaN,NaN,Abdominal drainage,NaN
2595833,133483,1798,179432.0,297430.0,2194-02-03 19:00:00,41178,80.0,ml,2194-02-03 19:12:00,19150,NaN,NaN,NaN,Abdominal drainage,NaN
2595834,133484,1798,179432.0,297430.0,2194-02-03 21:00:00,41178,1575.0,ml,2194-02-04 12:44:00,20482,NaN,NaN,NaN,Abdominal drainage,NaN
2595835,133485,1798,179432.0,297430.0,2194-02-04 05:30:00,41178,350.0,ml,2194-02-04 05:52:00,14987,NaN,NaN,NaN,Abdominal drainage,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2783477,2126496,9773,136342.0,273354.0,2102-04-10 18:00:00,45742,100.0,ml,2102-04-10 19:07:00,14973,NaN,NaN,NaN,Abdominal drainage,NaN
2783478,2127067,9773,136342.0,273354.0,2102-04-11 07:00:00,45742,150.0,ml,2102-04-11 07:18:00,15803,NaN,NaN,NaN,Abdominal drainage,NaN
2783479,2129926,9773,136342.0,273354.0,2102-04-08 19:00:00,45742,200.0,ml,2102-04-08 19:26:00,19089,NaN,NaN,NaN,Abdominal drainage,NaN
2783480,2129927,9773,136342.0,273354.0,2102-04-09 00:00:00,45742,100.0,ml,2102-04-09 00:16:00,15814,NaN,NaN,NaN,Abdominal drainage,NaN


In [114]:
outputevents[outputevents.LABEL.isin(['vac abd drain',
'vac drain',
'vac drainage',
'vac drainage.',
'vac out',
'vac output',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'vac abd drain', 'vac drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'vac drain', 'vac drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'vac drainage.', 'vac drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'vac out', 'vac drainage', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'vac output', 'vac drainage', inplace=True)

outputevents[outputevents.LABEL.isin(['vac drainage'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2648929,733423,20066,150208.0,288840.0,2108-09-28 16:00:00,42982,275.0,ml,2108-09-28 19:09:00,19947,NaN,NaN,NaN,vac drainage,NaN
2648930,733424,20066,150208.0,288840.0,2108-09-29 08:00:00,42982,300.0,ml,2108-09-29 12:15:00,19947,NaN,NaN,NaN,vac drainage,NaN
2648931,733425,20066,150208.0,288840.0,2108-09-29 16:00:00,42982,50.0,ml,2108-09-29 16:18:00,19947,NaN,NaN,NaN,vac drainage,NaN
2648932,804653,26308,190417.0,283211.0,2145-02-16 22:00:00,42982,300.0,ml,2145-02-17 00:22:00,14562,NaN,NaN,NaN,vac drainage,NaN
2648933,755729,20066,147919.0,271698.0,2108-10-28 19:00:00,42982,0.0,NaN,2108-10-28 19:00:00,21570,NaN,NaN,NaN,vac drainage,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2784946,2484557,30470,185471.0,239012.0,2194-02-24 07:00:00,46466,100.0,ml,2194-02-24 06:46:00,19611,NaN,NaN,NaN,vac drainage,NaN
2784948,2484559,30470,185471.0,239012.0,2194-02-24 12:00:00,46466,50.0,ml,2194-02-24 11:56:00,15900,NaN,NaN,NaN,vac drainage,NaN
2784949,2484560,30470,185471.0,239012.0,2194-02-24 18:00:00,46466,75.0,ml,2194-02-24 18:13:00,15900,NaN,NaN,NaN,vac drainage,NaN
2784950,2483564,30470,185471.0,239012.0,2194-02-22 07:00:00,46466,30.0,ml,2194-02-22 07:06:00,19611,NaN,NaN,NaN,vac drainage,NaN


In [115]:
outputevents[outputevents.LABEL.isin([
'paracenteis drainag.',
'paracentesis',
'paracentesis asicits',
'paracentesis bag',
'paracentesis fluid',
'paracentesis out',
'paracentesis output',
'paracentesis removal',
'paracentesis site',
'paracentesis tap out',
'paracentesis.',
'paracentesis/OR',
'paracentesis: 2020ml',
'parascentesis',
'parencentesis',])]

outputevents['LABEL'].mask(outputevents['LABEL']== 'paracenteis drainag.', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis asicits', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis bag', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis fluid', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis out', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis output', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis removal', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis site', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis tap out', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis.', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis/OR', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'paracentesis: 2020ml', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'parascentesis', 'paracentesis', inplace=True)
outputevents['LABEL'].mask(outputevents['LABEL']== 'parencentesis', 'paracentesis', inplace=True)

outputevents[outputevents.LABEL.isin(['paracentesis'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL,CATEGORY
2271892,4213,19246,150542.0,244311.0,2126-08-21 17:00:00,40470,3700.0,ml,2126-08-21 18:08:00,17269,NaN,NaN,NaN,paracentesis,NaN
2271893,4214,19246,150542.0,244311.0,2126-08-23 00:00:00,40470,0.0,NaN,2126-08-23 00:00:00,21570,NaN,NaN,NaN,paracentesis,NaN
2271894,11303,6800,142356.0,265008.0,2177-12-30 04:00:00,40470,90.0,ml,2177-12-30 04:50:00,15029,NaN,NaN,NaN,paracentesis,NaN
2271895,11304,6800,142356.0,265008.0,2177-12-30 16:00:00,40470,5000.0,ml,2177-12-30 18:22:00,15352,NaN,NaN,NaN,paracentesis,NaN
2271896,34737,7675,156854.0,212869.0,2103-04-25 14:00:00,40470,1500.0,ml,2103-04-25 14:02:00,21570,NaN,NaN,NaN,paracentesis,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2783534,2171873,16365,141257.0,236635.0,2134-04-20 17:00:00,45792,100.0,ml,2134-04-20 17:28:00,16037,NaN,NaN,NaN,paracentesis,NaN
2783535,2752489,16365,141257.0,236635.0,2134-04-18 00:00:00,45792,0.0,ml,2134-04-17 23:55:00,19152,NaN,NaN,NaN,paracentesis,NaN
2783536,2752490,16365,141257.0,236635.0,2134-04-19 18:00:00,45792,100.0,ml,2134-04-19 19:26:00,15667,NaN,NaN,NaN,paracentesis,NaN
2783537,2124363,16365,141257.0,210835.0,2134-03-10 16:00:00,45715,2020.0,ml,2134-03-10 19:21:00,20106,NaN,NaN,NaN,paracentesis,NaN


In [116]:
outputevents['LABEL'] = outputevents['LABEL'].map({
'Chest Tube #1':'Chest Tubes Other',
'Chest Tube #2':'Chest Tubes Other',
'Chest Tube #3':'Chest Tubes Other',
'Chest Tube In':'Chest Tubes Other',
'Chest Tube R #3':'Chest Tubes Other',
'Chest Tubes CTICU CT 1':'Chest Tubes Other',
'Chest Tubes CTICU CT 2':'Chest Tubes Other',
'Chest Tubes Chest Tube':'Chest Tubes Other',
'Chest Tubes Left Pleural 1':'Chest Tubes Other',
'Chest Tubes Left Pleural 2':'Chest Tubes Other',
'Chest Tubes Mediastinal':'Chest Tubes Other',
'Chest Tubes Other':'Chest Tubes Other',
'Chest Tubes Right Pleural 1':'Chest Tubes Other',
'Chest Tubes Right Pleural 2':'Chest Tubes Other',
'Chest tube flush':'Chest Tubes Other',
'Chest tube irrigate':'Chest Tubes Other',
'Chest tube pleural':'Chest Tubes Other'
})

<ipython-input-116-fa0c5a82720e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputevents['LABEL'] = outputevents['LABEL'].map({


In [117]:
outputevents=outputevents.pivot_table(values='VALUE', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME'], columns='LABEL')

In [118]:
outputevents

LABEL                                                      Chest Tubes Other
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                             
13         143045.0 263738.0   40076  2167-01-09 16:45:00               65.0
                                      2167-01-09 17:00:00               35.0
                                      2167-01-09 17:15:00               16.0
                                      2167-01-09 18:00:00               24.0
                                      2167-01-09 18:30:00               40.0
...                                                                      ...
99982      151454.0 221194.0   226588 2156-11-29 13:11:00               20.0
                                      2156-11-29 15:00:00               40.0
                                      2156-11-29 17:00:00               60.0
                                      2156-11-29 20:21:00               40.0
                                      2156-11-29 22:00:00               20.0

[325691 rows x 1 columns]

In [119]:
outputevents['Chest Tubes Other'] = outputevents.groupby('HADM_ID')['Chest Tubes Other'].transform('mean')

In [120]:
outputevents.drop_duplicates(inplace=True)

In [121]:
outputevents.columns

Index(['Chest Tubes Other'], dtype='object', name='LABEL')

In [122]:
outputevents

LABEL                                                      Chest Tubes Other
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                             
13         143045.0 263738.0   40076  2167-01-09 16:45:00          31.071429
17         161087.0 257980.0   40076  2135-05-09 15:30:00          35.857143
           194023.0 277042.0   40076  2134-12-27 19:15:00           8.653846
20         157681.0 264490.0   40076  2183-04-28 18:00:00          36.000000
23         152223.0 227807.0   40076  2153-09-03 13:30:00          31.461538
...                                                                      ...
99708      183397.0 279308.0   226588 2133-06-26 13:03:00          49.090909
99830      176834.0 211489.0   226588 2187-08-22 20:00:00          25.105882
99873      143544.0 206270.0   226588 2103-12-21 12:00:00          18.787879
99880      179507.0 215004.0   226588 2162-04-19 11:37:00          14.347826
99913      175989.0 292375.0   226588 2141-05-12 18:00:00          27.906977

[5516 rows x 1 columns]

In [123]:
charts

,,,,LABEL,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,ART Lumen Volume,AST,Admission Weight (Kg),...,Pulmonary Artery Pressure mean,Respiratory Rate,Skeletal Traction #1 - Pounds,Sodium (serum),Temperature Celsius,Total Bilirubin,Venous CO2 Pressure,Venous O2 Pressure,Ventilator Tank #1,WBC
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,,,,,,,,,,,,,,,,,,,,,
23,124321,234044.0,220045,2157-10-21 12:06:00,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,NaN,NaN,66.739039,...,NaN,17.114286,NaN,144.000000,36.301587,NaN,NaN,NaN,NaN,15.600000
34,144319,290505.0,220045,2191-02-23 05:25:00,NaN,NaN,NaN,NaN,NaN,128.055556,NaN,NaN,NaN,74.382842,...,NaN,16.731707,NaN,140.500000,36.319444,NaN,NaN,NaN,NaN,7.500000
36,165660,241249.0,220045,2134-05-12 06:52:00,NaN,NaN,12.000000,NaN,NaN,127.950000,NaN,NaN,18.00,106.079589,...,NaN,17.906475,NaN,141.500000,37.264444,0.600000,NaN,NaN,2471.428571,11.760000
85,112077,291697.0,220045,2167-07-25 21:12:00,NaN,NaN,23.000000,NaN,NaN,111.125000,NaN,NaN,18.00,97.897257,...,NaN,28.340426,NaN,137.000000,37.156566,NaN,NaN,NaN,NaN,8.650000
107,174162,264253.0,220045,2122-05-14 21:07:00,NaN,NaN,22.000000,NaN,NaN,104.520000,NaN,NaN,21.00,83.882580,...,NaN,15.051724,NaN,137.666667,36.902778,0.100000,NaN,NaN,NaN,11.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49471,184788,227744.0,220045,2143-05-10 12:09:00,NaN,NaN,NaN,NaN,NaN,108.195122,NaN,NaN,NaN,75.920322,...,32.160714,20.620690,NaN,134.600000,36.141577,NaN,NaN,NaN,2000.000000,11.466667
49480,122348,228590.0,220045,2162-07-29 07:00:00,NaN,NaN,26.500000,NaN,NaN,136.814286,NaN,NaN,24.25,74.517254,...,NaN,19.081761,NaN,136.181818,36.439940,0.600000,NaN,NaN,NaN,6.620000
49482,191442,209686.0,220045,2155-07-08 13:00:00,NaN,NaN,NaN,NaN,NaN,100.800000,NaN,NaN,NaN,73.018827,...,NaN,15.000000,NaN,NaN,36.194444,NaN,NaN,NaN,NaN,NaN


In [70]:
def _merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], right_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])

In [125]:
charts_outputevents=_merge(charts,outputevents)

In [126]:
charts_outputevents

,,LABEL,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,ART Lumen Volume,AST,Admission Weight (Kg),...,Respiratory Rate,Skeletal Traction #1 - Pounds,Sodium (serum),Temperature Celsius,Total Bilirubin,Venous CO2 Pressure,Venous O2 Pressure,Ventilator Tank #1,WBC,Chest Tubes Other
SUBJECT_ID,HADM_ID,ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,NaN,NaN,66.739039,...,17.114286,NaN,144.000000,36.301587,NaN,NaN,NaN,NaN,15.600000,NaN
34,144319.0,290505.0,NaN,NaN,NaN,NaN,NaN,128.055556,NaN,NaN,NaN,74.382842,...,16.731707,NaN,140.500000,36.319444,NaN,NaN,NaN,NaN,7.500000,NaN
36,165660.0,241249.0,NaN,NaN,12.000000,NaN,NaN,127.950000,NaN,NaN,18.00,106.079589,...,17.906475,NaN,141.500000,37.264444,0.600000,NaN,NaN,2471.428571,11.760000,NaN
85,112077.0,291697.0,NaN,NaN,23.000000,NaN,NaN,111.125000,NaN,NaN,18.00,97.897257,...,28.340426,NaN,137.000000,37.156566,NaN,NaN,NaN,NaN,8.650000,NaN
107,174162.0,264253.0,NaN,NaN,22.000000,NaN,NaN,104.520000,NaN,NaN,21.00,83.882580,...,15.051724,NaN,137.666667,36.902778,0.100000,NaN,NaN,NaN,11.033333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49471,184788.0,227744.0,NaN,NaN,NaN,NaN,NaN,108.195122,NaN,NaN,NaN,75.920322,...,20.620690,NaN,134.600000,36.141577,NaN,NaN,NaN,2000.000000,11.466667,37.166667
49480,122348.0,228590.0,NaN,NaN,26.500000,NaN,NaN,136.814286,NaN,NaN,24.25,74.517254,...,19.081761,NaN,136.181818,36.439940,0.600000,NaN,NaN,NaN,6.620000,NaN
49482,191442.0,209686.0,NaN,NaN,NaN,NaN,NaN,100.800000,NaN,NaN,NaN,73.018827,...,15.000000,NaN,NaN,36.194444,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
charts_outputevents.to_csv(output_files+'charts_outputevents.csv')

# INPUTEVENTS_CV File

In [3]:
inputevents_cv=pd.read_csv(mimic_path+"INPUTEVENTS_CV.csv.gz")

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,9,17,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
inputevents_cv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17421052,8786,132435.0,258871.0,2152-09-13 00:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527931,17421053,8786,132435.0,258871.0,2152-09-13 01:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527932,17421054,8786,132435.0,258871.0,2152-09-13 02:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake
17527933,17421055,8786,132435.0,258871.0,2152-09-13 03:00:00,45347,0.0,NaN,NaN,NaN,...,D/C'd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake


In [16]:
inputevents_cv=item_merge(inputevents_cv, d_c)
inputevents_cv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL_x,CATEGORY_x,LABEL_y,CATEGORY_y
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,...,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN,Po Intake,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,...,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN,Po Intake,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,...,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN,Po Intake,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,...,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN,Po Intake,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,...,NaN,ml,Oral,NaN,NaN,NaN,Po Intake,NaN,Po Intake,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527930,17421052,8786,132435.0,258871.0,2152-09-13 00:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake,Banana Bag,Free Form Intake
17527931,17421053,8786,132435.0,258871.0,2152-09-13 01:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake,Banana Bag,Free Form Intake
17527932,17421054,8786,132435.0,258871.0,2152-09-13 02:00:00,45347,150.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake,Banana Bag,Free Form Intake
17527933,17421055,8786,132435.0,258871.0,2152-09-13 03:00:00,45347,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Banana Bag,Free Form Intake,Banana Bag,Free Form Intake


In [18]:
labels_incv=inputevents_cv['LABEL_x'].tolist()

In [19]:
labels_incv_unique=np.unique(labels_incv)

In [ ]:
file=open(output_files+'unique_labels_input_CV.txt','w')
for item in labels_incv_unique:
    file.write(item+'\n')

file.close()

In [20]:
inputevents_cv['LABEL_x'] = inputevents_cv['LABEL_x'].astype('category')
inputevents_cv['LABEL_x'].cat.set_categories(labels_incv_unique,inplace=True)
inputevents_cv=inputevents_cv.dropna(how='any', subset=['AMOUNT']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [23]:
inputevents_cv.columns

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID',
       'AMOUNT', 'AMOUNTUOM', 'RATE', 'RATEUOM', 'STORETIME', 'CGID',
       'ORDERID', 'LINKORDERID', 'STOPPED', 'NEWBOTTLE', 'ORIGINALAMOUNT',
       'ORIGINALAMOUNTUOM', 'ORIGINALROUTE', 'ORIGINALRATE', 'ORIGINALRATEUOM',
       'ORIGINALSITE', 'LABEL_x', 'CATEGORY_x', 'CATEGORY_y'],
      dtype='object')

In [24]:
inputevents_cv.rename(columns = {'LABEL_x':'LABEL'}, inplace = True)

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
inputevents_cv[inputevents_cv.LABEL.isin(['Protonix',
'Protonix 80mg/250cc',
'Protonix Drip',
'Protonix GTT',
'Protonix IV',
'Protonix gtt',
'Protonix gtt 8mg/hr',
'Protonix mg/hr'])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix 80mg/250cc', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix Drip', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix GTT', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix IV', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix gtt', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix gtt 8mg/hr', 'Protonix', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Protonix mg/hr', 'Protonix', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Protonix'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17441240,17431226,28868,145654.0,229203.0,2141-09-26 02:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17441241,17431227,28868,145654.0,229203.0,2141-09-26 03:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17441242,17431228,28868,145654.0,229203.0,2141-09-26 04:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17441243,17431229,28868,145654.0,229203.0,2141-09-26 05:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17441244,17431230,28868,145654.0,229203.0,2141-09-26 06:00:00,42204,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527662,17458900,389,134048.0,220654.0,2104-04-18 22:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17527663,17458901,389,134048.0,220654.0,2104-04-18 23:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17527664,17458902,389,134048.0,220654.0,2104-04-19 00:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake
17527665,17458903,389,134048.0,220654.0,2104-04-19 02:00:00,45024,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Protonix,Free Form Intake,Free Form Intake


In [26]:
inputevents_cv[inputevents_cv.LABEL.isin(['Prosobee',
'Prosobee 1/2s PO',
'Prosobee 1/4s PO',
'Prosobee 20cal GT',
'Prosobee 24cal GT',
'Prosobee FS PO',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Prosobee 1/2s PO', 'Prosobee', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Prosobee 1/4s PO', 'Prosobee', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Prosobee 20cal GT', 'Prosobee', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Prosobee 24cal GT', 'Prosobee', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Prosobee FS PO', 'Prosobee', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Prosobee'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17176592,8738555,3752,140442.0,289134.0,2187-03-19 13:15:00,30334,25.0,ml,NaN,NaN,...,NaN,NaN,cc,By Mouth,NaN,NaN,NaN,Prosobee,NaN,NaN
17176593,8738556,3752,140442.0,289134.0,2187-03-19 15:30:00,30334,30.0,ml,NaN,NaN,...,NaN,NaN,cc,By Mouth,NaN,NaN,NaN,Prosobee,NaN,NaN
17176594,8738557,3752,140442.0,289134.0,2187-03-19 18:30:00,30334,5.0,ml,NaN,NaN,...,NaN,NaN,cc,By Mouth,NaN,NaN,NaN,Prosobee,NaN,NaN
17176595,8738558,3752,140442.0,289134.0,2187-03-19 21:00:00,30334,15.0,ml,NaN,NaN,...,NaN,NaN,cc,By Mouth,NaN,NaN,NaN,Prosobee,NaN,NaN
17176596,8738559,3752,140442.0,289134.0,2187-03-20 00:00:00,30334,90.0,ml,NaN,NaN,...,NaN,NaN,cc,By Mouth,NaN,NaN,NaN,Prosobee,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17516345,17521766,18374,138701.0,263080.0,2142-04-04 02:00:00,43476,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prosobee,Free Form Intake,Free Form Intake
17516346,17521767,18374,138701.0,263080.0,2142-04-04 03:00:00,43476,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prosobee,Free Form Intake,Free Form Intake
17516347,17521768,18374,138701.0,263080.0,2142-04-04 04:00:00,43476,25.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prosobee,Free Form Intake,Free Form Intake
17516348,17521769,18374,138701.0,263080.0,2142-04-04 06:00:00,43476,97.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prosobee,Free Form Intake,Free Form Intake


In [27]:
inputevents_cv[inputevents_cv.LABEL.isin(['FREE H20',
'FREE H20 BOLUS',
'FREE H20 BOLUSES',
'FREE H20 NG',
'FREE H2O',
'FREE WATER',
'FREE WATER BOLUSES',
'FREE WATER INTAKE',
'FREE WATER/OR D5W',
'Free H20',
'Free H20 bolus',
'Free H20 intake',
'Free H2O',
'Free Water',
'Free Water Bolus',
'Free water',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE H20 BOLUS', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE H20 BOLUSES', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE H20 NG', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE H2O', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE WATER', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE WATER BOLUSES', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE WATER INTAKE', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'FREE WATER/OR D5W', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free H20', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free H20 bolus', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free H20 intake', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free H2O', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free Water Bolus', 'Free Water', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Free water', 'Free Water', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Free Water'])]


C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
7116438,5869991,4589,101114.0,294200.0,2112-01-07 12:00:00,30058,250.0,ml,NaN,NaN,...,NaN,NaN,ml,Nasogastric,NaN,NaN,NaN,Free Water,NaN,NaN
7116439,5869992,4589,101114.0,294200.0,2112-01-07 20:00:00,30058,250.0,ml,NaN,NaN,...,NaN,NaN,ml,Nasogastric,NaN,NaN,NaN,Free Water,NaN,NaN
7116440,5869993,4589,101114.0,294200.0,2112-01-08 00:00:00,30058,250.0,ml,NaN,NaN,...,NaN,NaN,ml,Nasogastric,NaN,NaN,NaN,Free Water,NaN,NaN
7116441,5869994,4589,101114.0,294200.0,2112-01-08 22:00:00,30058,100.0,ml,NaN,NaN,...,NaN,NaN,ml,Nasogastric,NaN,NaN,NaN,Free Water,NaN,NaN
7116442,5869995,4589,101114.0,294200.0,2112-01-09 00:00:00,30058,70.0,ml,NaN,NaN,...,NaN,NaN,ml,Nasogastric,NaN,NaN,NaN,Free Water,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527901,17469281,18892,190436.0,235302.0,2188-01-28 09:00:00,41457,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free Water,Free Form Intake,Free Form Intake
17527902,17469282,18892,190436.0,235302.0,2188-01-28 11:00:00,41457,500.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free Water,Free Form Intake,Free Form Intake
17527903,17469283,18892,190436.0,235302.0,2188-01-28 13:00:00,41457,500.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free Water,Free Form Intake,Free Form Intake
17527904,17469284,18892,190436.0,235302.0,2188-01-28 15:00:00,41457,500.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Free Water,Free Form Intake,Free Form Intake


In [28]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Fentanyl',
'Fentanyl (Conc)',
'Fentanyl Base',
'Fentanyl Drip',
'Fentanyl bolus:',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl (Conc)', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl Base', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl Drip', 'Fentanyl', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Fentanyl bolus:', 'Fentanyl', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Fentanyl'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
7281469,6010210,28972,134622.0,229355.0,2127-11-29 16:00:00,30150,2.00,ml,NaN,NaN,...,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN,NaN
7281470,6010211,28972,134622.0,229355.0,2127-11-29 17:00:00,30150,2.00,ml,NaN,NaN,...,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN,NaN
7281471,6010212,28972,134622.0,229355.0,2127-11-29 18:00:00,30150,2.00,ml,NaN,NaN,...,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN,NaN
7281472,6010213,28972,134622.0,229355.0,2127-11-29 19:00:00,30150,2.00,ml,NaN,NaN,...,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN,NaN
7281473,6010214,28972,134622.0,229355.0,2127-11-29 20:00:00,30150,2.00,ml,NaN,NaN,...,NaN,50.0,ml,Intravenous Push,2.000,ml/hr,NaN,Fentanyl,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16874951,13060846,31892,163351.0,240409.0,2154-02-04 23:00:00,30308,1.70,mcg,NaN,NaN,...,NaN,500.0,mcg,Intravenous,NaN,NaN,NaN,Fentanyl,NaN,NaN
16874952,13060850,27321,165983.0,274907.0,2159-12-10 23:00:00,30308,3.50,mcg,NaN,NaN,...,NaN,500.0,mcg,Intravenous,0.231,ml/hr,Percutaneous Cath,Fentanyl,NaN,NaN
16874953,13060897,31892,163351.0,240409.0,2154-02-05 07:00:00,30308,1.40,mcg,NaN,NaN,...,NaN,500.0,mcg,Intravenous,NaN,NaN,NaN,Fentanyl,NaN,NaN
16874954,13060899,27321,165983.0,274907.0,2159-12-11 07:00:00,30308,3.00,mcg,NaN,NaN,...,NaN,500.0,mcg,Intravenous,0.231,ml/hr,Percutaneous Cath,Fentanyl,NaN,NaN


In [29]:
inputevents_cv[inputevents_cv.LABEL.isin(['KCL',
'KCL 10 MEQ',
'KCL 10meq/100ml',
'KCL 10mmol in 100sw',
'KCL 20mEq bolus',
'KCL 40 MEQ IN 500MLS',
'KCL CVVHDF',
'KCL for CVVH',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10 MEQ', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10meq/100ml', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 10mmol in 100sw', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 20mEq bolus', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL 40 MEQ IN 500MLS', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL CVVHDF', 'KCL', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'KCL for CVVH', 'KCL', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['KCL'])]


C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
9815085,8925648,21842,174955.0,250436.0,2140-11-24 17:00:00,30026,0.000000,mEq,NaN,NaN,...,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN,NaN
9815086,8925659,21857,173189.0,269052.0,2116-11-05 18:00:00,30026,8.346456,mEq,NaN,NaN,...,NaN,100.0,mEq,Intravenous Push,63.0,ml/hr,NaN,KCL,NaN,NaN
9815087,8943326,9175,148433.0,218471.0,2194-12-20 10:00:00,30026,40.000000,mEq,NaN,NaN,...,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN,NaN
9815088,8943328,18605,173380.0,264352.0,2188-12-13 10:00:00,30026,4.000000,mEq,NaN,NaN,...,NaN,20.0,mEq,Intravenous Push,80.0,ml/hr,NaN,KCL,NaN,NaN
9815089,8943345,6960,178048.0,298934.0,2120-11-24 00:00:00,30026,40.000000,mEq,NaN,NaN,...,NaN,40.0,mEq,Intravenous Push,NaN,NaN,NaN,KCL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527220,17454480,29130,106375.0,212954.0,2200-08-03 10:00:00,46013,0.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,Free Form Intake,Free Form Intake
17527221,17454481,29130,106375.0,212954.0,2200-08-03 11:00:00,46013,0.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,Free Form Intake,Free Form Intake
17527222,17454482,29130,106375.0,212954.0,2200-08-03 12:00:00,46013,0.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,Free Form Intake,Free Form Intake
17527223,17444362,29130,106375.0,212954.0,2200-08-02 22:00:00,46013,0.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCL,Free Form Intake,Free Form Intake


In [30]:
inputevents_cv[inputevents_cv.LABEL.isin(['Kphos',
'Kphos 15 mmol',
'Kphos 15mmol',
'Kphos 15mmol. 250cc',
'Kphos 30mmol',
'Kphos 30mmol/44 mEq',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Kphos 15 mmol','Kphos', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Kphos 15mmol','Kphos', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Kphos 15mmol. 250cc','Kphos', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Kphos 30mmol','Kphos', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Kphos 30mmol/44 mEq','Kphos', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Kphos'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17490223,17476977,101,175533.0,233111.0,2196-09-28 23:00:00,40880,83.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17490224,17476978,101,175533.0,233111.0,2196-09-29 00:00:00,40880,83.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17490225,17476979,101,175533.0,233111.0,2196-09-29 01:00:00,40880,83.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17490226,17476980,101,175533.0,233111.0,2196-09-29 02:00:00,40880,83.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17490227,17476981,101,175533.0,233111.0,2196-09-29 03:00:00,40880,83.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17526889,17400950,30191,130885.0,232219.0,2183-06-12 08:00:00,46680,31.700001,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17526890,17400951,30191,130885.0,232219.0,2183-06-12 09:00:00,46680,31.700001,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17526894,17400955,30191,130885.0,232219.0,2183-06-12 13:00:00,46680,42.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17526895,17400956,30191,130885.0,232219.0,2183-06-12 14:00:00,46680,42.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake


In [31]:
inputevents_cv[inputevents_cv.LABEL.isin(['LR',
'LR BOLUS',
'LR Bolus',
'LR bolus',
'LR W/ 20 KCL',
'LR w/ 40 mEq',
'LR w/40 kcl',
'LR w/50meqNAacetate',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR BOLUS', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR Bolus', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR bolus', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR W/ 20 KCL', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR w/ 40 mEq', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR w/40 kcl', 'LR', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'LR w/50meqNAacetate', 'LR', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['LR'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17452906,17449408,9957,105364.0,208354.0,2160-08-28 04:00:00,45321,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17452907,17449409,9957,105364.0,208354.0,2160-08-28 05:00:00,45321,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17452908,17449410,9957,105364.0,208354.0,2160-08-28 06:00:00,45321,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17452909,17449411,9957,105364.0,208354.0,2160-08-28 07:00:00,45321,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17452910,17449412,9957,105364.0,208354.0,2160-08-28 08:00:00,45321,50.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527374,17462447,4413,141166.0,242340.0,2173-01-19 16:00:00,42288,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17527375,17462448,4413,141166.0,242340.0,2173-01-19 17:00:00,42288,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17527376,17462449,4413,141166.0,242340.0,2173-01-19 18:00:00,42288,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake
17527377,17462450,4413,141166.0,242340.0,2173-01-19 20:00:00,42288,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LR,Free Form Intake,Free Form Intake


In [32]:
inputevents_cv[inputevents_cv.LABEL.isin(['SODIUM PHOS',
'SODIUM PHOS IN NS',
'SODIUM PHOSPHATE',
'SODIUM PO4',
'SOIDUM PHOSPHATE',
'SOD PHOSPHATE',
'Sodium phos 15mmol',
'Sodium phos 30mmol',
'Sodium phosphate',
'Soduim Phospate',
'Soduim Phosphate',
'Sodium PHosphate',
'Sodium Phos 30mmol',
'Sodium Phosphate',
'Sodium Phosphate IV',
'Sodium Phosphate30mm',
'na phos',
'na phosp 15 mmol',
'na phosphate',
'na phosphate in ns',
'NA PHOS',
'NA PHOS  0mmol',
'NA PHOS (15MMOL)',
'NA PHOS 15MMOL',
'NA PHOS 30 MMO;',
'NA PHOS 30MMOL',
'NA PHOS IN NSS',
'NA PHOSPHATE',
'NA PHOSPHATE 15MMOL',
'NA PHOSPHATE 30MMOL',
'NA POSPHATE',
'NA Phos',
'NA Phosphate',
'NA Phosphate 30mmol',
'NA phos',
'NA phosphate',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SODIUM PHOS','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SODIUM PHOS IN NS','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SODIUM PHOSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SODIUM PO4','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SOIDUM PHOSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SOD PHOSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium phos 15mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium phos 30mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium phosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Soduim Phospate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Soduim Phosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'SOD PHOSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium PHosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium Phos 30mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium Phosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium Phosphate IV','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Sodium Phosphate30mm','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'na phos','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'na phosp 15 mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'na phosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'na phosphate in ns','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS  0mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS (15MMOL)','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS 15MMOL','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS 30 MMO;','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS 30MMOL','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOS IN NSS','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOSPHATE 15MMOL','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA PHOSPHATE 30MMOL','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA POSPHATE','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA Phos','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA Phosphate','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA Phosphate 30mmol','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA phos','Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'NA phosphate','Sodium Phosphate', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Sodium Phosphate'])]


C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17442031,17456623,14227,111310.0,277166.0,2104-01-03 12:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442032,17456624,14227,111310.0,277166.0,2104-01-03 13:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442033,17456625,14227,111310.0,277166.0,2104-01-03 14:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442034,17456626,14227,111310.0,277166.0,2104-01-03 15:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442040,17411674,12768,182007.0,224267.0,2154-03-11 10:00:00,45047,41.700001,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527612,17433484,17795,179589.0,245295.0,2201-01-28 11:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527613,17433485,17795,179589.0,245295.0,2201-01-28 12:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527614,17433486,17795,179589.0,245295.0,2201-01-28 13:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527615,17433487,17795,179589.0,245295.0,2201-01-28 14:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake


In [33]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Kphos',
'K 10MEQ',
'K PHOS',
'K PHOS 15 MMOL',
'K PHOS 15MM0L',
'K PHOS 15MMOL',
'K PHOS 30MMOL',
'K PHOS OVER 6 HRS',
'K PHOS, 30 MMOL',
'K PHOS-30',
'K PHOSPHATE',
'K Phos',
'K Phos 15mmol',
'K phos',
'KPHOS',
'KPHOS 15MMOL',
'KPHOS 30MMOL',
'KPO4',
'KPO4 30 MM',
'KPhos',
'KPhos 15mmol/250D5W',
'KPo4 30mmol /NS',])]


inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K 10MEQ','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS 15 MMOL','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS 15MM0L','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS 30MMOL','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS OVER 6 HRS','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS, 30 MMOL','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOS-30','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K PHOSPHATE','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K Phos','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K Phos 15mmol','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='K phos','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPHOS','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPHOS 15MMOL','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPHOS 30MMOL','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPO4','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPO4 30 MM','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPhos','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPhos 15mmol/250D5W','Kphos',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='KPo4 30mmol /NS','Kphos',inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Kphos'])]


C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17435848,17516435,18520,147760.0,221298.0,2166-12-06 10:00:00,44162,0.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17435849,17516436,18520,147760.0,221298.0,2166-12-06 11:00:00,44162,85.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17435850,17516437,18520,147760.0,221298.0,2166-12-06 12:00:00,44162,85.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17435851,17516438,18520,147760.0,221298.0,2166-12-06 13:00:00,44162,85.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17435852,17516439,18520,147760.0,221298.0,2166-12-06 14:00:00,44162,85.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527786,17458038,27939,197384.0,247039.0,2177-08-22 19:00:00,46116,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17527787,17458039,27939,197384.0,247039.0,2177-08-22 20:00:00,46116,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17527788,17458040,27939,197384.0,247039.0,2177-08-22 21:00:00,46116,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake
17527789,17458041,27939,197384.0,247039.0,2177-08-22 22:00:00,46116,80.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kphos,Free Form Intake,Free Form Intake


In [34]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Insulin',
'Insulin Carrier',
'Insulin Drip',
'Insulin carrier',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Insulin Carrier', 'Insulin', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Insulin Drip', 'Insulin', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']== 'Insulin carrier', 'Insulin', inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Insulin'])]


C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
13987004,8926225,20796,166936.0,200406.0,2134-01-12 04:00:00,30045,7.0,U,NaN,NaN,...,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN,NaN
13987005,8926226,12879,199845.0,240198.0,2186-01-24 04:00:00,30045,12.0,U,NaN,NaN,...,NaN,100.0,U,Intravenous Push,2.0,ml/hr,NaN,Insulin,NaN,NaN
13987006,8926227,8449,102933.0,222892.0,2105-12-29 04:00:00,30045,4.0,U,NaN,NaN,...,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN,NaN
13987007,8926230,18630,173492.0,245846.0,2124-11-21 04:00:00,30045,1.0,U,NaN,NaN,...,NaN,100.0,U,Intravenous Push,NaN,NaN,NaN,Insulin,NaN,NaN
13987008,8926231,13744,181330.0,277942.0,2101-01-25 04:00:00,30045,2.0,U,NaN,NaN,...,NaN,100.0,U,Intravenous Push,1.0,ml/hr,NaN,Insulin,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17481100,17401403,20815,150497.0,252193.0,2147-05-31 17:00:00,44518,5.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insulin,Free Form Intake,Free Form Intake
17481101,17401404,20815,150497.0,252193.0,2147-05-31 18:00:00,44518,5.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insulin,Free Form Intake,Free Form Intake
17481102,17401405,20815,150497.0,252193.0,2147-05-31 19:00:00,44518,5.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insulin,Free Form Intake,Free Form Intake
17481103,17401406,20815,150497.0,252193.0,2147-05-31 20:00:00,44518,5.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Insulin,Free Form Intake,Free Form Intake


In [35]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Sodium Phosphate',
'NA PHOS',
'NA PHOS  0mmol',
'NA PHOS (15MMOL)',
'NA PHOS 15MMOL',
'NA PHOS 30 MMO;',
'NA PHOS 30MMOL',
'NA PHOS IN NSS',
'NA PHOSPHATE',
'NA PHOSPHATE 15MMOL',
'NA PHOSPHATE 30MMOL',
'NA POSPHATE',
'NA Phos',
'NA Phosphate',
'NA Phosphate 30mmol',
'NA phos',
'NA phosphate',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS 0mmol', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS (15MMOL)', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS 30 MMO;', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS 30MMOL', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOS IN NSS', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOSPHATE', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOSPHATE 15MMOL', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA PHOSPHATE 30MMOL', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA POSPHATE', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA Phos', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA Phosphate', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA Phosphate 30mmol', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA phos', 'Sodium Phosphate', inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='NA phosphate', 'Sodium Phosphate', inplace=True)


inputevents_cv[inputevents_cv.LABEL.isin(['Sodium Phosphate'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17442031,17456623,14227,111310.0,277166.0,2104-01-03 12:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442032,17456624,14227,111310.0,277166.0,2104-01-03 13:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442033,17456625,14227,111310.0,277166.0,2104-01-03 14:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442034,17456626,14227,111310.0,277166.0,2104-01-03 15:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442040,17411674,12768,182007.0,224267.0,2154-03-11 10:00:00,45047,41.700001,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527612,17433484,17795,179589.0,245295.0,2201-01-28 11:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527613,17433485,17795,179589.0,245295.0,2201-01-28 12:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527614,17433486,17795,179589.0,245295.0,2201-01-28 13:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527615,17433487,17795,179589.0,245295.0,2201-01-28 14:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake


In [36]:
inputevents_cv[inputevents_cv.LABEL.isin(['albumin',
'albumin 12.5%'])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='albumin 12.5%','albumin',inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['albumin'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17471161,17516686,9328,192998.0,240264.0,2139-06-30 12:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471168,17516693,9328,192998.0,240264.0,2139-06-30 20:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471169,17516694,9328,192998.0,240264.0,2139-06-30 21:00:00,42832,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471170,17516695,9328,192998.0,240264.0,2139-07-01 02:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471172,17516697,9328,192998.0,240264.0,2139-07-01 05:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471173,17516698,9328,192998.0,240264.0,2139-07-01 12:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471175,17516700,9328,192998.0,240264.0,2139-07-01 18:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471176,17516701,9328,192998.0,240264.0,2139-07-02 00:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471177,17516702,9328,192998.0,240264.0,2139-07-02 06:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake
17471179,17516704,9328,192998.0,240264.0,2139-07-02 12:00:00,42832,250.0,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,albumin,Free Form Intake,Free Form Intake


In [37]:
inputevents_cv[inputevents_cv.LABEL.isin([
'Sodium Phosphate',
'sodium phos',
'sodium phos 15mmol',
'sodium phos 30mmol',
'sodium phosphate',
'sodium phosphate 15m',])]

inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='sodium phos','Sodium Phosphate',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='sodium phos 15mmol','Sodium Phosphate',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='sodium phos 30mmol','Sodium Phosphate',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='sodium phosphate','Sodium Phosphate',inplace=True)
inputevents_cv['LABEL'].mask(inputevents_cv['LABEL']=='sodium phosphate 15m','Sodium Phosphate',inplace=True)

inputevents_cv[inputevents_cv.LABEL.isin(['Sodium Phosphate'])]

C:\Users\Mariam Safieldin\anaconda3\lib\site-packages\pandas\core\generic.py:9286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,...,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE,LABEL,CATEGORY_x,CATEGORY_y
17442031,17456623,14227,111310.0,277166.0,2104-01-03 12:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442032,17456624,14227,111310.0,277166.0,2104-01-03 13:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442033,17456625,14227,111310.0,277166.0,2104-01-03 14:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442034,17456626,14227,111310.0,277166.0,2104-01-03 15:00:00,45047,60.000000,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17442040,17411674,12768,182007.0,224267.0,2154-03-11 10:00:00,45047,41.700001,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17527612,17433484,17795,179589.0,245295.0,2201-01-28 11:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527613,17433485,17795,179589.0,245295.0,2201-01-28 12:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527614,17433486,17795,179589.0,245295.0,2201-01-28 13:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake
17527615,17433487,17795,179589.0,245295.0,2201-01-28 14:00:00,45535,41.299999,ml,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sodium Phosphate,Free Form Intake,Free Form Intake


In [38]:
inputevents_cv['LABEL'] = inputevents_cv['LABEL'].map({
'Protonix 80mg/250cc':'Protonix',
'Protonix':'Protonix',
'Protonix Drip':'Protonix',
'Protonix GTT':'Protonix',
'Protonix IV':'Protonix',
'Protonix gtt':'Protonix',
'Protonix gtt 8mg/hr':'Protonix',
'Protonix mg/hr':'Protonix',
'Prosobee 1/2s PO':'Prosobee',
'Prosobee':'Prosobee',
'Prosobee 1/4s PO':'Prosobee',
'Prosobee 20cal GT':'Prosobee',
'Prosobee 24cal GT':'Prosobee',
'Prosobee FS PO':'Prosobee',
'FREE H20':'Free Water',
'FREE H20 BOLUS':'Free Water',
'FREE H20 BOLUSES':'Free Water',
'FREE H20 NG':'Free Water',
'FREE H2O':'Free Water',
'FREE WATER':'Free Water',
'FREE WATER BOLUSES':'Free Water',
'FREE WATER INTAKE':'Free Water',
'FREE WATER/OR D5W':'Free Water',
'Free H20':'Free Water',
'Free H20 bolus':'Free Water',
'Free H20 intake':'Free Water',
'Free H2O':'Free Water',
'Free Water':'Free Water',
'Free Water Bolus':'Free Water',
'Free water':'Free Water',
'Fentanyl':'Fentanyl',
'Fentanyl (Conc)':'Fentanyl',
'Fentanyl Base':'Fentanyl',
'Fentanyl Drip':'Fentanyl',
'Fentanyl bolus:':'Fentanyl',
'KCL':'KCL',
'KCL 10 MEQ':'KCL',
'KCL 10meq/100ml':'KCL',
'KCL 10mmol in 100sw':'KCL',
'KCL 20mEq bolus':'KCL',
'KCL 40 MEQ IN 500MLS':'KCL',
'KCL CVVHDF':'KCL',
'KCL for CVVH':'KCL',
'Kphos':'Kphos',
'K 10MEQ':'Kphos',
'K PHOS':'Kphos',
'K PHOS 15 MMOL':'Kphos',
'K PHOS 15MM0L':'Kphos',
'K PHOS 15MMOL':'Kphos',
'K PHOS 30MMOL':'Kphos',
'K PHOS OVER 6 HRS':'Kphos',
'K PHOS, 30 MMOL':'Kphos',
'K PHOS-30':'Kphos',
'K PHOSPHATE':'Kphos',
'K Phos':'Kphos',
'K Phos 15mmol':'Kphos',
'K phos':'Kphos',
'KPHOS':'Kphos',
'KPHOS 15MMOL':'Kphos',
'KPHOS 30MMOL':'Kphos',
'KPO4':'Kphos',
'KPO4 30 MM':'Kphos',
'KPhos':'Kphos',
'KPhos 15mmol/250D5W':'Kphos',
'KPo4 30mmol /NS':'Kphos',
'Kphos':'Kphos',
'Kphos 15 mmol':'Kphos',
'Kphos 15mmol':'Kphos',
'Kphos 15mmol. 250cc':'Kphos',
'Kphos 30mmol':'Kphos',
'Kphos 30mmol/44 mEq':'Kphos',
'Soduim Phospate':'Sodium Phosphate',
'Soduim Phosphate':'Sodium Phosphate',
'Sodium PHosphate':'Sodium Phosphate',
'Sodium Phos 30mmol':'Sodium Phosphate',
'Sodium Phosphate':'Sodium Phosphate',
'Sodium Phosphate IV':'Sodium Phosphate',
'Sodium Phosphate30mm':'Sodium Phosphate',
'na phos':'Sodium Phosphate',
'na phosp 15 mmol':'Sodium Phosphate',
'na phosphate':'Sodium Phosphate',
'na phosphate in ns':'Sodium Phosphate',
'NA PHOS':'Sodium Phosphate',
'NA PHOS  0mmol':'Sodium Phosphate',
'NA PHOS (15MMOL)':'Sodium Phosphate',
'NA PHOS 15MMOL':'Sodium Phosphate',
'NA PHOS 30 MMO;':'Sodium Phosphate',
'NA PHOS 30MMOL':'Sodium Phosphate',
'NA PHOS IN NSS':'Sodium Phosphate',
'NA PHOSPHATE':'Sodium Phosphate',
'NA PHOSPHATE 15MMOL':'Sodium Phosphate',
'NA PHOSPHATE 30MMOL':'Sodium Phosphate',
'NA POSPHATE':'Sodium Phosphate',
'NA Phos':'Sodium Phosphate',
'NA Phosphate':'Sodium Phosphate',
'NA Phosphate 30mmol':'Sodium Phosphate',
'NA phos':'Sodium Phosphate',
'NA phosphate':'Sodium Phosphate',
'SODIUM PHOS':'Sodium Phosphate',
'SODIUM PHOS IN NS':'Sodium Phosphate',
'SODIUM PHOSPHATE':'Sodium Phosphate',
'SODIUM PO4':'Sodium Phosphate',
'SOIDUM PHOSPHATE':'Sodium Phosphate',
'SOD PHOSPHATE':'Sodium Phosphate',
'Sodium phos 15mmol':'Sodium Phosphate',
'Sodium phos 30mmol':'Sodium Phosphate',
'Sodium phosphate':'Sodium Phosphate',
'NA PHOS':'Sodium Phosphate',
'NA PHOS  0mmol':'Sodium Phosphate',
'NA PHOS (15MMOL)':'Sodium Phosphate',
'NA PHOS 15MMOL':'Sodium Phosphate',
'NA PHOS 30 MMO;':'Sodium Phosphate',
'NA PHOS 30MMOL':'Sodium Phosphate',
'NA PHOS IN NSS':'Sodium Phosphate',
'NA PHOSPHATE':'Sodium Phosphate',
'NA PHOSPHATE 15MMOL':'Sodium Phosphate',
'NA PHOSPHATE 30MMOL':'Sodium Phosphate',
'NA POSPHATE':'Sodium Phosphate',
'NA Phos':'Sodium Phosphate',
'NA Phosphate':'Sodium Phosphate',
'NA Phosphate 30mmol':'Sodium Phosphate',
'NA phos':'Sodium Phosphate',
'NA phosphate':'Sodium Phosphate',
'Insulin':'Insulin',
'Insulin Carrier':'Insulin',
'Insulin Drip':'Insulin',
'Insulin carrier':'Insulin',
'sodium phos':'Sodium Phosphate',
'sodium phos 15mmol':'Sodium Phosphate',
'sodium phos 30mmol':'Sodium Phosphate',
'sodium phosphate':'Sodium Phosphate',
'sodium phosphate 15m':'Sodium Phosphate'})

<ipython-input-38-ed33cd4b74f5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputevents_cv['LABEL'] = inputevents_cv['LABEL'].map({


In [39]:
inputevents_cv=inputevents_cv.pivot_table(values='AMOUNT', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME'], columns='LABEL')

In [40]:
inputevents_cv.columns

Index(['Fentanyl', 'Free Water', 'Insulin', 'KCL', 'Kphos', 'Prosobee',
       'Protonix', 'Sodium Phosphate'],
      dtype='object', name='LABEL')

In [42]:
cols=['Fentanyl', 'Free Water', 'Insulin', 'KCL', 'Kphos', 'Prosobee',
       'Protonix', 'Sodium Phosphate']

In [43]:
for col in cols:
    inputevents_cv[col] = inputevents_cv.groupby('HADM_ID')[col].transform('mean')

In [ ]:
inputevents_cv.rename(columns = {'LABEL_x':'LABEL'}, inplace = True)

In [45]:
inputevents_cv

LABEL                                                       Fentanyl  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                        
3          145834.0 211552.0   30118  2101-10-21 04:00:00   7.812500   
                                      2101-10-21 05:00:00   7.812500   
                                      2101-10-21 07:00:00   7.812500   
                                      2101-10-21 08:00:00   7.812500   
                                      2101-10-21 09:00:00   7.812500   
...                                                              ...   
32810      189751.0 204923.0   30026  2100-09-21 15:00:00  70.798023   
                                      2100-09-21 16:00:00  70.798023   
                                      2100-09-21 18:00:00  70.798023   
                                      2100-09-22 03:00:00  70.798023   
                                      2100-09-25 05:30:00  70.798023   

LABEL                                                      Free Water  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                         
3          145834.0 211552.0   30118  2101-10-21 04:00:00         NaN   
                                      2101-10-21 05:00:00         NaN   
                                      2101-10-21 07:00:00         NaN   
                                      2101-10-21 08:00:00         NaN   
                                      2101-10-21 09:00:00         NaN   
...                                                               ...   
32810      189751.0 204923.0   30026  2100-09-21 15:00:00         NaN   
                                      2100-09-21 16:00:00         NaN   
                                      2100-09-21 18:00:00         NaN   
                                      2100-09-22 03:00:00         NaN   
                                      2100-09-25 05:30:00         NaN   

LABEL                                                      Insulin        KCL  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                                 
3          145834.0 211552.0   30118  2101-10-21 04:00:00      NaN        NaN   
                                      2101-10-21 05:00:00      NaN        NaN   
                                      2101-10-21 07:00:00      NaN        NaN   
                                      2101-10-21 08:00:00      NaN        NaN   
                                      2101-10-21 09:00:00      NaN        NaN   
...                                                            ...        ...   
32810      189751.0 204923.0   30026  2100-09-21 15:00:00      NaN  23.333333   
                                      2100-09-21 16:00:00      NaN  23.333333   
                                      2100-09-21 18:00:00      NaN  23.333333   
                                      2100-09-22 03:00:00      NaN  23.333333   
                                      2100-09-25 05:30:00      NaN  23.333333   

LABEL                                                      Kphos  Prosobee  \
SUBJECT_ID HADM_ID  ICUSTAY_ID ITEMID CHARTTIME                              
3          145834.0 211552.0   30118  2101-10-21 04:00:00    NaN       NaN   
                                      2101-10-21 05:00:00    NaN       NaN   
                                      2101-10-21 07:00:00    NaN       NaN   
                                      2101-10-21 08:00:00    NaN       NaN   
                                      2101-10-21 09:00:00    NaN       NaN   
...                                                          ...       ...   
32810      189751.0 204923.0   30026  2100-09-21 15:00:00    NaN       NaN   
                                      2100-09-21 16:00:00    NaN       NaN   
                                      2100-09-21 18:00:00    NaN       NaN   
                                      2100-09-22 03:00:00    NaN       NaN   
                                      2100-09-25 05:30:00    NaN       NaN   

LABEL                                       

In [46]:
inputevents_cv.drop_duplicates(inplace=True)

In [ ]:
charts_outputevents_inputevents=_merge(charts_outputevents,inputevents_cv)

In [47]:
inputevents_cv.to_csv(output_files+"inputevents_cv.csv")

# INPUTEVENTS_MV File

In [44]:
inputevents_mv=pd.read_csv(mimic_path+"INPUTEVENTS_MV.csv")

In [49]:
inputevents_mv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,241,27063,139787,223259.0,2/5/2133 6:29,2/5/2133 8:45,225166,6.774532,mEq,NaN,...,ml,0,0,1,Rewritten,NaN,RN,2/5/2133 12:52,10.000000,0.050000
1,242,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 6:30,225944,28.132997,ml,30.142497,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817
2,243,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 6:30,225166,2.813300,mEq,NaN,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426
3,244,27063,139787,223259.0,2/3/2133 12:00,2/3/2133 12:01,225893,1.000000,dose,NaN,...,ml,0,0,2,Rewritten,RN,NaN,2/3/2133 17:06,1.000000,1.000000
4,245,27063,139787,223259.0,2/3/2133 12:00,2/3/2133 12:01,220949,100.000000,ml,NaN,...,ml,0,0,2,Rewritten,RN,NaN,2/3/2133 17:06,100.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1036987,70851,104704,208780.0,9/24/2130 10:07,9/24/2130 10:25,220949,7.500000,ml,25.000000,...,ml,0,0,0,Changed,NaN,NaN,NaN,250.000000,25.000000
1048571,1036988,70851,104704,208780.0,9/23/2130 8:45,9/23/2130 9:50,222051,9.782611,mg,2.009368,...,ml,0,0,0,Rewritten,NaN,NaN,NaN,9.782610,2.000000
1048572,1036989,70851,104704,208780.0,9/23/2130 8:45,9/23/2130 9:50,225158,48.913050,ml,45.150508,...,ml,0,0,0,Rewritten,NaN,NaN,NaN,48.913048,44.944000
1048573,1036990,70851,104704,208780.0,9/23/2130 17:14,9/23/2130 17:15,223258,10.000000,units,NaN,...,NaN,0,0,0,FinishedRunning,NaN,NaN,NaN,10.000000,10.000000


In [50]:
inputevents_mv=item_merge(inputevents_mv, d_c)
inputevents_mv

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
0,241,27063,139787,223259.0,2/5/2133 6:29,2/5/2133 8:45,225166,6.774532,mEq,NaN,...,0,1,Rewritten,NaN,RN,2/5/2133 12:52,10.000000,0.050000,Potassium Chloride,Medications
1,243,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 6:30,225166,2.813300,mEq,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426,Potassium Chloride,Medications
2,247,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 7:03,225166,4.433333,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,4.433333,0.050000,Potassium Chloride,Medications
3,249,27063,139787,223259.0,2/5/2133 9:43,2/5/2133 12:30,225166,8.360021,mEq,NaN,...,0,0,Changed,NaN,NaN,NaN,10.000001,0.050248,Potassium Chloride,Medications
4,251,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 5:35,225166,0.050060,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,2.753300,0.050248,Potassium Chloride,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,988517,94472,119136,251260.0,8/20/2105 17:04,8/21/2105 14:32,222190,38.147549,mg,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,38.147541,0.029626,Ranitidine,Nutrition - Parenteral
1048571,988142,94472,119136,251260.0,8/22/2105 18:17,8/22/2105 18:18,222190,0.029755,mg,NaN,...,0,0,Rewritten,NaN,NaN,NaN,29.814402,0.000000,Ranitidine,Nutrition - Parenteral
1048572,988430,94472,119136,251260.0,8/21/2105 14:32,8/22/2105 12:34,222190,40.000003,mg,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,40.000000,0.000000,Ranitidine,Nutrition - Parenteral
1048573,987984,94472,119136,251260.0,8/23/2105 12:15,8/24/2105 10:39,222190,40.000002,mg,NaN,...,0,2,Rewritten,RN,NaN,8/24/2105 11:27,40.000000,0.029762,Ranitidine,Nutrition - Parenteral


In [143]:
labels_inmv_unique=np.unique(inputevents_mv['LABEL'].tolist())
file=open(output_files+'unique_labels_input_MV.txt','w')
for item in labels_inmv_unique:
    file.write(item+'\n')

file.close()

In [51]:
labels_inmv_unique=np.unique(inputevents_mv['LABEL'].tolist())

In [52]:
inputevents_mv['LABEL'] = inputevents_mv['LABEL'].astype('category')
inputevents_mv['LABEL'].cat.set_categories(labels_inmv_unique,inplace=True)
inputevents_mv=inputevents_mv.dropna(how='any', subset=['AMOUNT']) # This removes rows with null values for VALUENUM column (a total of 190,141 records removed)

In [53]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Insulin - 70/30',
'Insulin - Glargine',
'Insulin - Humalog',
'Insulin - Humalog 75/25',
'Insulin - NPH',
'Insulin - Regular'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - 70/30', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Glargine', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Humalog', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - Humalog 75/25', 'Insulin - Regular', inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Insulin - NPH', 'Insulin - Regular', inplace=True)

inputevents_mv[inputevents_mv.LABEL.isin(['Insulin - Regular'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
389980,108,29969,129547,248410.0,2/20/2181 21:00,2/20/2181 22:00,223258,5.000000,units,5.000000,...,0,0,Changed,NaN,NaN,NaN,100.000000,5.000000,Insulin - Regular,Medications
389981,112,29969,129547,248410.0,2/21/2181 4:00,2/21/2181 7:24,223258,34.984998,units,10.289705,...,0,0,FinishedRunning,NaN,NaN,NaN,34.985001,10.294024,Insulin - Regular,Medications
389982,116,29969,129547,248410.0,2/21/2181 4:00,2/21/2181 4:01,223258,0.166597,units,9.995806,...,0,0,Rewritten,NaN,NaN,NaN,34.985321,10.000000,Insulin - Regular,Medications
389983,101,30354,120396,243526.0,12/25/2154 12:00,12/25/2154 12:01,223258,5.000000,units,NaN,...,0,2,Rewritten,RN,NaN,12/25/2154 18:12,5.000000,5.000000,Insulin - Regular,Medications
389984,47,30354,120396,243526.0,12/25/2154 12:00,12/25/2154 12:01,223258,5.000000,units,NaN,...,0,1,Rewritten,NaN,RN,12/25/2154 18:13,5.000000,0.000000,Insulin - Regular,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968548,1033494,32074,162201,256697.0,9/29/2197 17:28,9/29/2197 17:29,223257,18.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,18.000000,18.000000,Insulin - Regular,Medications
968549,1033496,32074,162201,256697.0,9/29/2197 8:00,9/29/2197 8:01,223257,18.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,18.000000,18.000000,Insulin - Regular,Medications
968550,1033503,32074,162201,256697.0,9/28/2197 8:32,9/28/2197 8:33,223257,18.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,18.000000,18.000000,Insulin - Regular,Medications
968551,1033505,32074,162201,256697.0,9/30/2197 17:45,9/30/2197 17:46,223257,18.000000,units,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,18.000000,18.000000,Insulin - Regular,Medications


In [54]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Potassium Chloride',
'KCL (Bolus)',
'KCl (CRRT)'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'KCL (Bolus)','KCl (CRRT)',inplace=True)
inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']== 'Potassium Chloride','KCl (CRRT)',inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['KCl (CRRT)'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
0,241,27063,139787,223259.0,2/5/2133 6:29,2/5/2133 8:45,225166,6.774532,mEq,NaN,...,0,1,Rewritten,NaN,RN,2/5/2133 12:52,10.000000,0.050000,KCl (CRRT),Medications
1,243,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 6:30,225166,2.813300,mEq,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426,KCl (CRRT),Medications
2,247,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 7:03,225166,4.433333,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,4.433333,0.050000,KCl (CRRT),Medications
3,249,27063,139787,223259.0,2/5/2133 9:43,2/5/2133 12:30,225166,8.360021,mEq,NaN,...,0,0,Changed,NaN,NaN,NaN,10.000001,0.050248,KCl (CRRT),Medications
4,251,27063,139787,223259.0,2/5/2133 5:34,2/5/2133 5:35,225166,0.050060,mEq,NaN,...,0,0,Rewritten,NaN,NaN,NaN,2.753300,0.050248,KCl (CRRT),Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015230,1024930,53347,194627,299415.0,10/28/2194 1:22,10/28/2194 2:20,227522,48.000001,ml,49.655174,...,0,0,FinishedRunning,NaN,NaN,NaN,48.000000,49.655170,KCl (CRRT),Medications
1015231,1022105,66641,153726,220356.0,9/14/2131 16:00,9/14/2131 17:00,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications
1015232,1024753,53347,194627,299415.0,10/27/2194 14:35,10/27/2194 15:35,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications
1015233,1026884,2090,172739,253288.0,10/16/2137 4:24,10/16/2137 5:24,227522,49.999999,ml,49.999999,...,0,0,FinishedRunning,NaN,NaN,NaN,50.000000,50.000000,KCl (CRRT),Medications


In [55]:
inputevents_mv[inputevents_mv.LABEL.isin([
'Fentanyl',
'Fentanyl (Concentrate)'])]

inputevents_mv['LABEL'].mask(inputevents_mv['LABEL']=='Fentanyl (Concentrate)', 'Fentanyl',inplace=True)
inputevents_mv[inputevents_mv.LABEL.isin(['Fentanyl'])]

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,...,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE,LABEL,CATEGORY
497264,3,30354,120396,243526.0,12/24/2154 12:00,12/24/2154 14:00,221744,0.050000,mg,25.000000,...,0,0,FinishedRunning,NaN,NaN,NaN,0.050000,25.000000,Fentanyl,Medications
497265,11,30354,120396,243526.0,12/24/2154 12:41,12/28/2154 16:41,221744,2.500000,mg,25.000001,...,0,2,Rewritten,RN,NaN,12/24/2154 14:04,2.500000,25.000000,Fentanyl,Medications
497266,31,30354,120396,243526.0,12/24/2154 15:40,12/24/2154 15:41,221744,500.000020,mcg,NaN,...,0,0,FinishedRunning,NaN,NaN,NaN,500.000000,500.000030,Fentanyl,Medications
497267,38,30354,120396,243526.0,12/24/2154 15:35,12/25/2154 5:01,221744,0.671780,mg,50.008468,...,0,0,Stopped,NaN,NaN,NaN,2.462917,50.000000,Fentanyl,Medications
497268,44,30354,120396,243526.0,12/24/2154 12:00,12/28/2154 16:00,221744,2.500000,mg,25.000001,...,0,2,Rewritten,RN,NaN,12/24/2154 14:05,2.500000,25.000000,Fentanyl,Medications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770542,1023854,49068,105351,279856.0,11/10/2190 17:50,11/10/2190 20:12,225942,0.070891,mg,29.954001,...,0,0,Changed,NaN,NaN,NaN,0.124808,30.000000,Fentanyl,Medications
770543,1023861,49068,105351,279856.0,11/6/2190 12:00,11/7/2190 7:05,225942,1.238000,mg,64.873374,...,0,0,FinishedRunning,NaN,NaN,NaN,1.238000,64.821556,Fentanyl,Medications
770544,1023868,49068,105351,279856.0,11/7/2190 21:46,11/9/2190 4:56,225942,0.779167,mg,25.000001,...,0,0,Changed,NaN,NaN,NaN,2.500000,25.000000,Fentanyl,Medications
770545,1024156,49068,105351,279856.0,11/11/2190 4:17,11/11/2190 13:12,225942,0.221946,mg,24.891145,...,0,0,Changed,NaN,NaN,NaN,2.199962,25.000000,Fentanyl,Medications


In [56]:
inputevents_mv['LABEL'] = inputevents_mv['LABEL'].map({
'Fentanyl':'Fentanyl',
'Fentanyl (Concentrate)':'Fentanyl',
'Insulin - 70/30':'Insulin - Regular',
'Insulin - Glargine':'Insulin - Regular',
'Insulin - Humalog':'Insulin - Regular',
'Insulin - Humalog 75/25':'Insulin - Regular',
'Insulin - NPH':'Insulin - Regular',
'Insulin - Regular':'Insulin - Regular',
'Pantoprazole (Protonix)':'Pantoprazole (Protonix)',
'Na Phos':'Na Phos',
'K Phos':'K Phos',
'Potassium Chloride':'KCl (CRRT)', 
'KCL (Bolus)':'KCl (CRRT)',   
'KCl (CRRT)':'KCl (CRRT)',   
'Free Water':'Free Water'})

In [57]:
inputevents_mv=inputevents_mv.pivot_table(values='AMOUNT', index=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID'], columns='LABEL')

In [59]:
inputevents_mv.columns

Index(['Fentanyl', 'Free Water', 'Insulin - Regular', 'K Phos', 'KCl (CRRT)',
       'Na Phos', 'Pantoprazole (Protonix)'],
      dtype='object', name='LABEL')

In [60]:
cols=['Fentanyl', 'Free Water', 'Insulin - Regular', 'K Phos', 'KCl (CRRT)',
       'Na Phos', 'Pantoprazole (Protonix)']

In [61]:
for col in cols:
    inputevents_mv[col] = inputevents_mv.groupby('HADM_ID')[col].transform('mean')

In [62]:
inputevents_mv.drop_duplicates(inplace=True)

In [86]:
inputevents_mv

,,,LABEL,Fentanyl,Free Water,Insulin,Kphos,KCL,Sodium Phosphate,Protonix
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,
23,124321,234044.0,223262,NaN,NaN,4.000000,NaN,NaN,NaN,NaN
109,126055,236124.0,225910,NaN,NaN,NaN,NaN,NaN,NaN,1.0
124,138376,256064.0,223262,6.036613,NaN,2.878788,NaN,37.692307,NaN,1.0
199,125590,235675.0,223262,0.426788,NaN,17.235129,NaN,35.000000,NaN,NaN
305,194340,217232.0,223258,NaN,NaN,6.333333,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
99894,127380,204236.0,225910,NaN,NaN,NaN,30.000002,40.000001,NaN,1.0
99936,107913,213906.0,225166,NaN,NaN,NaN,NaN,22.460000,NaN,NaN
99938,103815,296707.0,223262,NaN,NaN,12.729912,NaN,35.000000,NaN,1.0


# Merge Tables

In [81]:
inputevents_cv

,,,,LABEL,Fentanyl,Free Water,Insulin,KCL,Kphos,Protonix,Sodium Phosphate
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,,,,,,,
3,145834.0,211552.0,30118,2101-10-21 04:00:00,7.812500,NaN,NaN,NaN,NaN,NaN,NaN
11,194540.0,229441.0,30026,2178-04-17 00:00:00,NaN,NaN,NaN,0.321429,NaN,NaN,NaN
12,112213.0,232669.0,30026,2104-08-09 02:00:00,NaN,NaN,0.597826,20.000000,NaN,NaN,NaN
13,143045.0,263738.0,30026,2167-01-09 17:30:00,NaN,NaN,2.909091,20.000000,NaN,NaN,NaN
17,194023.0,277042.0,30026,2134-12-28 22:00:00,NaN,NaN,NaN,13.333333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
32798,187802.0,215493.0,30026,2108-03-05 20:00:00,NaN,NaN,1.239130,20.000000,NaN,NaN,NaN
32800,179038.0,203257.0,30118,2189-02-03 04:00:00,12.500000,NaN,1.306452,13.500000,NaN,NaN,NaN
32805,124834.0,252568.0,30118,2182-11-20 06:00:00,73.779481,46.153846,5.691023,NaN,NaN,NaN,NaN


In [64]:
inputevents_mv.rename(columns = {'Insulin - Regular':'Insulin'}, inplace = True)

In [65]:
inputevents_mv.rename(columns = {'K Phos':'Kphos'}, inplace = True)
inputevents_mv.rename(columns = {'KCl (CRRT)':'KCL'}, inplace = True)
inputevents_mv.rename(columns = {'Na Phos':'Sodium Phosphate'}, inplace = True)
inputevents_mv.rename(columns = {'Pantoprazole (Protonix)':'Protonix'}, inplace = True)

In [66]:
inputevents_mv

,,,LABEL,Fentanyl,Free Water,Insulin,Kphos,KCL,Sodium Phosphate,Protonix
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,,,,,,,
23,124321,234044.0,223262,NaN,NaN,4.000000,NaN,NaN,NaN,NaN
109,126055,236124.0,225910,NaN,NaN,NaN,NaN,NaN,NaN,1.0
124,138376,256064.0,223262,6.036613,NaN,2.878788,NaN,37.692307,NaN,1.0
199,125590,235675.0,223262,0.426788,NaN,17.235129,NaN,35.000000,NaN,NaN
305,194340,217232.0,223258,NaN,NaN,6.333333,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
99894,127380,204236.0,225910,NaN,NaN,NaN,30.000002,40.000001,NaN,1.0
99936,107913,213906.0,225166,NaN,NaN,NaN,NaN,22.460000,NaN,NaN
99938,103815,296707.0,223262,NaN,NaN,12.729912,NaN,35.000000,NaN,1.0


In [68]:
inputevents_cv.drop('Prosobee',inplace=True,axis=1)

In [69]:
inputevents_cv

,,,,LABEL,Fentanyl,Free Water,Insulin,KCL,Kphos,Protonix,Sodium Phosphate
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,,,,,,,
3,145834.0,211552.0,30118,2101-10-21 04:00:00,7.812500,NaN,NaN,NaN,NaN,NaN,NaN
11,194540.0,229441.0,30026,2178-04-17 00:00:00,NaN,NaN,NaN,0.321429,NaN,NaN,NaN
12,112213.0,232669.0,30026,2104-08-09 02:00:00,NaN,NaN,0.597826,20.000000,NaN,NaN,NaN
13,143045.0,263738.0,30026,2167-01-09 17:30:00,NaN,NaN,2.909091,20.000000,NaN,NaN,NaN
17,194023.0,277042.0,30026,2134-12-28 22:00:00,NaN,NaN,NaN,13.333333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
32798,187802.0,215493.0,30026,2108-03-05 20:00:00,NaN,NaN,1.239130,20.000000,NaN,NaN,NaN
32800,179038.0,203257.0,30118,2189-02-03 04:00:00,12.500000,NaN,1.306452,13.500000,NaN,NaN,NaN
32805,124834.0,252568.0,30118,2182-11-20 06:00:00,73.779481,46.153846,5.691023,NaN,NaN,NaN,NaN


In [ ]:
def _merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], right_on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])

In [71]:
charts_outputevents=pd.read_csv(output_files+'charts_outputevents.csv')

In [89]:
charts_outputevents

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,...,Respiratory Rate,Skeletal Traction #1 - Pounds,Sodium (serum),Temperature Celsius,Total Bilirubin,Venous CO2 Pressure,Venous O2 Pressure,Ventilator Tank #1,WBC,Chest Tubes Other
0,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,17.114286,NaN,144.000000,36.301587,NaN,NaN,NaN,NaN,15.600000,NaN
1,34,144319.0,290505.0,NaN,NaN,NaN,NaN,NaN,128.055556,NaN,...,16.731707,NaN,140.500000,36.319444,NaN,NaN,NaN,NaN,7.500000,NaN
2,36,165660.0,241249.0,NaN,NaN,12.000000,NaN,NaN,127.950000,NaN,...,17.906475,NaN,141.500000,37.264444,0.600000,NaN,NaN,2471.428571,11.760000,NaN
3,85,112077.0,291697.0,NaN,NaN,23.000000,NaN,NaN,111.125000,NaN,...,28.340426,NaN,137.000000,37.156566,NaN,NaN,NaN,NaN,8.650000,NaN
4,107,174162.0,264253.0,NaN,NaN,22.000000,NaN,NaN,104.520000,NaN,...,15.051724,NaN,137.666667,36.902778,0.100000,NaN,NaN,NaN,11.033333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610,49471,184788.0,227744.0,NaN,NaN,NaN,NaN,NaN,108.195122,NaN,...,20.620690,NaN,134.600000,36.141577,NaN,NaN,NaN,2000.000000,11.466667,37.166667
8611,49480,122348.0,228590.0,NaN,NaN,26.500000,NaN,NaN,136.814286,NaN,...,19.081761,NaN,136.181818,36.439940,0.600000,NaN,NaN,NaN,6.620000,NaN
8612,49482,191442.0,209686.0,NaN,NaN,NaN,NaN,NaN,100.800000,NaN,...,15.000000,NaN,NaN,36.194444,NaN,NaN,NaN,NaN,NaN,NaN
8613,49485,119002.0,251750.0,NaN,NaN,238.833333,NaN,NaN,105.339943,72.75,...,20.274510,NaN,135.590909,36.797138,0.483333,41.0,55.0,2058.333333,13.430769,NaN


In [84]:
df = inputevents_mv.combine_first(inputevents_cv)

In [85]:
df

,,,,LABEL,Fentanyl,Free Water,Insulin,KCL,Kphos,Protonix,Sodium Phosphate
SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,,,,,,,
3,145834.0,211552.0,30118,2101-10-21 04:00:00,7.812500,NaN,NaN,NaN,NaN,NaN,NaN
11,194540.0,229441.0,30026,2178-04-17 00:00:00,NaN,NaN,NaN,0.321429,NaN,NaN,NaN
12,112213.0,232669.0,30026,2104-08-09 02:00:00,NaN,NaN,0.597826,20.000000,NaN,NaN,NaN
13,143045.0,263738.0,30026,2167-01-09 17:30:00,NaN,NaN,2.909091,20.000000,NaN,NaN,NaN
17,194023.0,277042.0,30026,2134-12-28 22:00:00,NaN,NaN,NaN,13.333333,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
99894,127380.0,204236.0,225910,NaN,NaN,NaN,NaN,40.000001,30.000002,1.0,NaN
99936,107913.0,213906.0,225166,NaN,NaN,NaN,NaN,22.460000,NaN,NaN,NaN
99938,103815.0,296707.0,223262,NaN,NaN,NaN,12.729912,35.000000,NaN,1.0,NaN


In [87]:
merged_df_mvcv_charts=_merge(charts_outputevents, df)

In [88]:
merged_df_mvcv_charts

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,...,Ventilator Tank #1,WBC,Chest Tubes Other,Fentanyl,Free Water,Insulin,KCL,Kphos,Protonix,Sodium Phosphate
0,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,NaN,15.600000,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
1,34,144319.0,290505.0,NaN,NaN,NaN,NaN,NaN,128.055556,NaN,...,NaN,7.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36,165660.0,241249.0,NaN,NaN,12.000000,NaN,NaN,127.950000,NaN,...,2471.428571,11.760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,85,112077.0,291697.0,NaN,NaN,23.000000,NaN,NaN,111.125000,NaN,...,NaN,8.650000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,107,174162.0,264253.0,NaN,NaN,22.000000,NaN,NaN,104.520000,NaN,...,NaN,11.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8641,49471,184788.0,227744.0,NaN,NaN,NaN,NaN,NaN,108.195122,NaN,...,2000.000000,11.466667,37.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8642,49480,122348.0,228590.0,NaN,NaN,26.500000,NaN,NaN,136.814286,NaN,...,NaN,6.620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8643,49482,191442.0,209686.0,NaN,NaN,NaN,NaN,NaN,100.800000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8644,49485,119002.0,251750.0,NaN,NaN,238.833333,NaN,NaN,105.339943,72.75,...,2058.333333,13.430769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
merged_df_mvcv_charts.to_csv(output_files+'merged_df_mvcv_charts.csv')

In [91]:
adm_data=pd.read_csv(output_files+'icu_stays_demographics.csv')

In [93]:
adm_data.columns

Index(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'LOS',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'DIAGNOSIS', 'HAS_CHARTEVENTS_DATA', 'GENDER', 'AGE'],
      dtype='object')

In [94]:
adm_data.drop(['Unnamed: 0','HAS_CHARTEVENTS_DATA'], inplace=True, axis=1)

In [96]:
merged_df_mvcv_charts_adm=_merge(merged_df_mvcv_charts, adm_data)

In [98]:
merged_df_mvcv_charts_adm.to_csv(output_files+'merged_df_mvcv_charts_adm.csv')

# Merge with Diagnosis Table

In [103]:
diag=pd.read_csv(mimic_path+'DIAGNOSES_ICD.csv')

In [104]:
diag_titles=pd.read_csv(mimic_path+'D_ICD_DIAGNOSES.csv')

In [105]:
diag

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [106]:
diag_titles

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,1166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,1170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,1171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,1172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,1173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [107]:
def diag_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['ICD9_CODE'], right_on=['ICD9_CODE'])

In [108]:
diag_titled=diag_merge(diag,diag_titles)

In [110]:
diag_titled.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE', 'ROW_ID_y',
       'SHORT_TITLE', 'LONG_TITLE'],
      dtype='object')

In [111]:
diag_titled.drop(['ROW_ID_x','ROW_ID_y','SEQ_NUM'],axis=1, inplace=True)

In [112]:
diag_titled

,SUBJECT_ID,HADM_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,109,172335,40301,Mal hyp kid w cr kid V,"Hypertensive chronic kidney disease, malignant..."
1,109,172335,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
2,109,172335,58281,Chr nephritis in oth dis,Chronic glomerulonephritis in diseases classif...
3,109,172335,5855,Chron kidney dis stage V,"Chronic kidney disease, Stage V"
4,109,172335,4254,Prim cardiomyopathy NEC,Other primary cardiomyopathies
...,...,...,...,...,...
670444,97503,188195,20280,Oth lymp unsp xtrndl org,"Other malignant lymphomas, unspecified site, e..."
670445,97503,188195,V5869,Long-term use meds NEC,Long-term (current) use of other medications
670446,97503,188195,V1279,Prsnl hst ot spf dgst ds,Personal history of other diseases of digestiv...
670447,97503,188195,5275,Sialolithiasis,Sialolithiasis


In [114]:
def diag_charts_merge(table1, table2):
    return table1.merge(table2, how='left', left_on=['SUBJECT_ID','HADM_ID'], right_on=['SUBJECT_ID','HADM_ID'])

In [139]:
merged_df_mvcv_charts_adm_diag=diag_charts_merge(merged_df_mvcv_charts_adm, diag_titled)

In [140]:
merged_df_mvcv_charts_adm_diag

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ABI Ankle BP L,ABI Brachial BP L,ALT,APACHE II,APS,ART BP Systolic,ART BP mean,...,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,AGE,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,75.304110,2252,Ben neo cerebr meninges,Benign neoplasm of cerebral meninges
1,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,75.304110,3485,Cerebral edema,Cerebral edema
2,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,75.304110,78039,Convulsions NEC,Other convulsions
3,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,75.304110,4241,Aortic valve disorder,Aortic valve disorders
4,23,124321.0,234044.0,NaN,NaN,NaN,NaN,NaN,122.180000,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,BRAIN MASS,M,75.304110,4019,Hypertension NOS,Unspecified essential hypertension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129846,49496,112342.0,269261.0,NaN,NaN,15.333333,NaN,NaN,106.853261,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,METASTATIC RENAL CELL CARCINOMA\CHEMOTHERAPY I...,M,68.564384,6930,Drug dermatitis NOS,Dermatitis due to drugs and medicines taken in...
129847,49496,112342.0,269261.0,NaN,NaN,15.333333,NaN,NaN,106.853261,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,METASTATIC RENAL CELL CARCINOMA\CHEMOTHERAPY I...,M,68.564384,E9331,Adv eff antineoplastic,Antineoplastic and immunosuppressive drugs cau...
129848,49496,112342.0,269261.0,NaN,NaN,15.333333,NaN,NaN,106.853261,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,METASTATIC RENAL CELL CARCINOMA\CHEMOTHERAPY I...,M,68.564384,2859,Anemia NOS,"Anemia, unspecified"
129849,49496,112342.0,269261.0,NaN,NaN,15.333333,NaN,NaN,106.853261,NaN,...,ENGL,CATHOLIC,MARRIED,WHITE,METASTATIC RENAL CELL CARCINOMA\CHEMOTHERAPY I...,M,68.564384,2874,NaN,NaN


In [141]:
merged_df_mvcv_charts_adm_diag.to_csv(output_files+'merged_df_mvcv_charts_adm_diag.csv')

# Identifying subjects with delirium diagnosis

In [142]:
del_codes=['2930','2903','29281','2931','29041','78097','29011','2910']

In [143]:
diag_codes=merged_df_mvcv_charts_adm_diag['ICD9_CODE'].tolist()
subject_ids=merged_df_mvcv_charts_adm_diag['SUBJECT_ID'].tolist()
delirious_subjects=[]
k=0
delirium=[]
for i in range(0,len(diag_codes)):
    if diag_codes[i] in del_codes:
        k+=1
        delirious_subjects.append(subject_ids[i])
        delirium.append(1)
    else:
        delirium.append(0)

In [144]:
short_titles=merged_df_mvcv_charts_adm_diag['SHORT_TITLE'].tolist()
long_titles=merged_df_mvcv_charts_adm_diag['LONG_TITLE'].tolist()

In [145]:
j=0
delirium_labels=[]
for i in range(0,len(diag_codes)):
    if diag_codes[i] in del_codes or subject_ids[i] in delirious_subjects:
        j+=1
        delirium_labels.append(1)
    else:
        delirium_labels.append(0)

In [146]:
j

18031

In [147]:
merged_df_mvcv_charts_adm_diag['Delirium']=delirium_labels

In [148]:
merged_df_mvcv_charts_adm_diag.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ABI Ankle BP L',
       'ABI Brachial BP L', 'ALT', 'APACHE II', 'APS', 'ART BP Systolic',
       'ART BP mean',
       ...
       'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'AGE',
       'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE', 'Delirium'],
      dtype='object', length=103)

In [149]:
merged_df_mvcv_charts_adm_diag.drop(['ICD9_CODE','SHORT_TITLE','LONG_TITLE'],inplace=True,axis=1)

In [150]:
merged_df_mvcv_charts_adm_diag.drop('DIAGNOSIS',inplace=True,axis=1)

In [151]:
merged_df_mvcv_charts_adm_diag.drop_duplicates(inplace=True)

In [152]:
merged_df_mvcv_charts_adm_diag.to_csv(output_files+'merged_df_mvcv_charts_adm_diag_delirium_labels.csv')